In [1]:
###IMPORT MODULES AS APPROPRIATE###

In [2]:
import nltk; nltk.download('stopwords')

C:\Users\brady\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\brady\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\brady\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/relea

In [4]:
###INSERT STOP WORDS AS NEEDED###

In [5]:
#NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
###IMPORT DATA - THIS IS CURRENTLY UGLY BUT WHATEVER###

###try this later:
#import csv
#reviews = [row for row in csv.reader(open('reviews.csv'))]
#print reviews

In [7]:
df = [
"The American College of Medical Genetics and Genomics (ACMG) previously developed guidance for the interpretation of sequence variants. 1 In the past decade, sequencing technology has evolved -rapidly with the advent of high-throughput next-generation -sequencing. By adopting and leveraging next-generation sequencing, clinical laboratories are now performing an ever-increasing catalogue of genetic testing spanning genotyping, single genes, gene panels, exomes, genomes, transcriptomes, and epigenetic assays for genetic disorders. By virtue of increased complexity, this shift in genetic testing has been accompanied by new challenges in sequence interpretation. In this context the ACMG convened a workgroup in 2013 comprising representatives from the ACMG, the Association for Molecular Pathology (AMP), and the College of American Pathologists to revisit and revise the standards and guidelines for the interpretation of sequence variants. The group consisted of clinical laboratory directors and clinicians. This report represents expert opinion of the workgroup with input from ACMG, AMP, and College of American Pathologists stakeholders. These recommendations primarily apply to the breadth of genetic tests used in clinical laboratories, including genotyping, single genes, panels, exomes, and genomes. This report recommends the use of specific standard terminology-pathogenic, likely pathogenic, uncertain significance, likely benign, and benign-to describe variants identified in genes that cause Mendelian disorders. Moreover, this recommendation describes a process for classifying variants into these five categories based on criteria using typical types of variant evidence (e.g., population data, computational data, functional data, segregation data). Because of the increased complexity of analysis and interpretation of clinical genetic testing described in this report, the ACMG strongly recommends that clinical molecular genetic testing should be performed in a Clinical Laboratory Improvement Amendments-approved laboratory, with results interpreted by a board-certified clinical molecular geneticist or molecular genetic pathologist or the equivalent.", 
"There is substantial heterogeneity among primary prostate cancers, evident in the spectrum of molecular abnormalities and its variable clinical course. As part of The Cancer Genome Atlas (TCGA), we present a comprehensive molecular analysis of 333 primary prostate carcinomas. Our results revealed a molecular taxonomy in which 74% of these tumors fell into one of seven subtypes defined by specific gene fusions (ERG, ETV1/4, and FLI1) or mutations (SPOP, FOXA1, and IDH1). Epigenetic profiles showed substantial heterogeneity, including an IDH1 mutant subset with a methylator phenotype. Androgen receptor (AR) activity varied widely and in a subtype-specific manner, with SPOP and FOXA1 mutant tumors having the highest levels of AR-induced transcripts. 25% of the prostate cancers had a presumed actionable lesion in the PI3K or MAPK signaling pathways, and DNA repair genes were inactivated in 19%. Our analysis reveals molecular heterogeneity among primary prostate cancers, as well as potentially actionable molecular defects.", 
"PD-L1 and PD-1 (PD) pathway blockade is a highly promising therapy and has elicited durable antitumor responses and long-term remissions in a subset of patients with a broad spectrum of cancers. How to improve, widen, and predict the clinical response to anti-PD therapy is a central theme in the field of cancer immunology and immunotherapy. Oncologic, immunologic, genetic, and biological studies focused on the human cancer microenvironment have yielded substantial insight into this issue. Here, we focus on tumor microenvironment and evaluate several potential therapeutic response markers including the PD-L1 and PD-1 expression pattern, genetic mutations within cancer cells and neoantigens, cancer epigenetics and effector T cell landscape, and microbiota. We further clarify the mechanisms of action of these markers and their roles in shaping, being shaped, and/or predicting therapeutic responses. We also discuss a variety of combinations with PD pathway blockade and their scientific rationales for cancer treatment.", 
"Ensembl (ext-link-type=uri xlink:href=http://www.ensembl.org xlink:type=simple>http://www.ensembl.org) is a genomic interpretation system providing the most up-to-date annotations, querying tools and access methods for chordates and key model organisms. This year we released updated annotation (gene models, comparative genomics, regulatory regions and variation) on the new human assembly, GRCh38, although we continue to support researchers using the GRCh37.p13 assembly through a dedicated site (ext-link-type=uri xlink:href=http://grch37.ensembl.org xlink:type=simple>http://grch37.ensembl.org). Our Regulatory Build has been revamped to identify regulatory regions of interest and to efficiently highlight their activity across disparate epigenetic data sets. A number of new interfaces allow users to perform large-scale comparisons of their data against our annotations. The REST server (ext-link-type=uri xlink:href=http://rest.ensembl.org xlink:type=simple>http://rest.ensembl.org), which allows programs written in any language to query our databases, has moved to a full service alongside our upgraded website tools. Our online Variant Effect Predictor tool has been updated to process more variants and calculate summary statistics. Lastly, the WiggleTools package enables users to summarize large collections of data sets and view them as single tracks in Ensembl. The Ensembl code base itself is more accessible: it is now hosted on our GitHub organization page (ext-link-type=uri xlink:href=https://github.com/Ensembl xlink:type=simple>https://github.com/Ensembl) under an Apache 2.0 open source license.", 
"Over the past 20 years, breakthrough discoveries of chromatin-modifying enzymes and associated mechanisms that alter chromatin in response to physiological or pathological signals have transformed our knowledge of epigenetics from a collection of curious biological phenomena to a functionally dissected research field. Here, we provide a personal perspective on the development of epigenetics, from its historical origins to what we define as 'the modern era of epigenetic research'. We primarily highlight key molecular mechanisms of and conceptual advances in epigenetic control that have changed our understanding of normal and perturbed development.", 
"Single-molecule, real-time sequencing developed by Pacific BioSciences offers longer read lengths than the second-generation sequencing (SGS) technologies, making it well-suited for unsolved problems in genome, transcriptome, and epigenetics research. The highly-contiguous de novo assemblies using PacBio sequencing can close gaps in current reference assemblies and characterize structural variation (SV) in personal genomes. With longer reads, we can sequence through extended repetitive regions and detect mutations, many of which are associated with diseases. Moreover, PacBio transcriptome sequencing is advantageous for the identification of gene isoforms and facilitates reliable discoveries of novel genes and novel isoforms of annotated genes, due to its ability to sequence full-length transcripts or fragments with significant lengths. Additionally, PacBio's sequencing technique provides information that is useful for the direct detection of base modifications, such as methylation. In addition to using PacBio sequencing alone, many hybrid sequencing strategies have been developed to make use of more accurate short reads in conjunction with PacBio long reads. In general, hybrid sequencing strategies are more affordable and scalable especially for small-size laboratories than using PacBio Sequencing alone. The advent of PacBio sequencing has made available much information that could not be obtained via SGS alone.", 
"MicroRNAs (miRNAs) are endogenous, small non-coding RNAs that function in regulation of gene expression. Compelling evidences have demonstrated that miRNA expression is dysregulated in human cancer through various mechanisms, including amplification or deletion of miRNA genes, abnormal transcriptional control of miRNAs, dysregulated epigenetic changes and defects in the miRNA biogenesis machinery. MiRNAs may function as either oncogenes or tumor suppressors under certain conditions. The dysregulated miRNAs have been shown to affect the hallmarks of cancer, including sustaining proliferative signaling, evading growth suppressors, resisting cell death, activating invasion and metastasis, and inducing angiogenesis. An increasing number of studies have identified miRNAs as potential biomarkers for human cancer diagnosis, prognosis and therapeutic targets or tools, which needs further investigation and validation. In this review, we focus on how miRNAs regulate the development of human tumors by acting as tumor suppressors or oncogenes.", 
"'If G-quadruplexes form so readily in vitro, Nature will have found a way of using them in vivo' (Statement by Aaron Klug over 30 years ago). During the last decade, four-stranded helical structures called G-quadruplex (or G4) have emerged from being a structural curiosity observed in vitro, to being recognized as a possible nucleic acid based mechanism for regulating multiple biological processes in vivo. The sequencing of many genomes has revealed that they are rich in sequence motifs that have the potential to form G-quadruplexes and that their location is non-random, correlating with functionally important genomic regions. In this short review, we summarize recent evidence for the in vivo presence and function of DNA and RNA G-quadruplexes in various cellular pathways including DNA replication, gene expression and telomere maintenance. We also highlight remaining open questions that will have to be addressed in the future.", 
"The aberrant expression of transcription factor Basonuclin (BNC) had been reported in different kinds of tumors. Here, we investigated the expression and methylation status of two Basonuclin homologs, BNC1 and BNC2 in hepatocellular carcinoma (HCC). We found that the expression levels of both BNC1 and BNC2 were down-regulated in HCC cell lines and primary HCC tissues. The frequency and intensity of BNC1 promoter hypermethylation in tumor tissues was significantly higher than that in adjacent non-tumor tissues. 5-Aza-2'-Deoxycytidine treatment could significantly increase the BNC1 expression in the methylated HCC cell lines, which implicated that epigenetic modification contributed to the down-regulation of BNC1. In addition, BNC1 hypermethylation in tumor tissues was more likely to happen in female patients. No methylation of the BNC2 promoter was found in HCC tumor tissues. However, a frequent deletion of the BNC2 gene was observed, which indicated that the chromosomal loss of the BNC2 gene might be one important reason for its lower expression level in HCC. Our results suggested that BNC1 and BNC2 were down-regulated in HCC which may provide new insight into the tumorigenesis of HCC.", 
"An increasingly recognized resistance mechanism to androgen receptor (AR)-directed therapy in prostate cancer involves epithelial plasticity, in which tumor cells demonstrate low to absent AR expression and often have neuroendocrine features. The etiology and molecular basis for this 'alternative' treatment-resistant cell state remain incompletely understood. Here, by analyzing whole-exome sequencing data of metastatic biopsies from patients, we observed substantial genomic overlap between castration-resistant tumors that were histologically characterized as prostate adenocarcinomas (CRPC-Adeno) and neuroendocrine prostate cancer (CRPC-NE); analysis of biopsy samples from the same individuals over time points to a model most consistent with divergent clonal evolution. Genome-wide DNA methylation analysis revealed marked epigenetic differences between CRPC-NE tumors and CRPC-Adeno, and also designated samples of CRPC-Adeno with clinical features of AR independence as CRPC-NE, suggesting that epigenetic modifiers may play a role in the induction and/or maintenance of this treatment-resistant state. This study supports the emergence of an alternative, 'AR-indifferent' cell state through divergent clonal evolution as a mechanism of treatment resistance in advanced prostate cancer.", 
"TP53 is the most frequently mutated gene in human cancer. Functionally, p53 is activated by a host of stress stimuli and, in turn, governs an exquisitely complex anti-proliferative transcriptional program that touches upon a bewildering array of biological responses. Despite the many unveiled facets of the p53 network, a clear appreciation of how and in what contexts p53 exerts its diverse effects remains unclear. How can we interpret p53's disparate activities and the consequences of its dysfunction to understand how cell type, mutation profile, and epigenetic cell state dictate outcomes, and how might we restore its tumor-suppressive activities in cancer?", 
"The field of machine learning, which aims to develop computer algorithms that improve with experience, holds promise to enable computers to assist humans in the analysis of large, complex data sets. Here, we provide an overview of machine learning applications for the analysis of genome sequencing data sets, including the annotation of sequence elements and epigenetic, proteomic or metabolomic data. We present considerations and recurrent challenges in the application of supervised, semi-supervised and unsupervised machine learning methods, as well as of generative and discriminative modelling approaches. We provide general guidelines to assist in the selection of these machine learning methods and their practical application for the analysis of genetic and genomic data sets.", 
"Mammalian DNA methylation is a critical epigenetic mechanism orchestrating gene expression networks in many biological processes. However, investigation of the functions of specific methylation events remains challenging. Here, we demonstrate that fusion of Tet1 or Dnmt3a with a catalytically inactive Cas9 (dCas9) enables targeted DNA methylation editing. Targeting of the dCas9-Tet1 or -Dnmt3a fusion protein to methylated or unmethylated promoter sequences caused activation or silencing, respectively, of an endogenous reporter. Targeted demethylation of the BDNF promoter IV or the MyoD distal enhancer by dCas9-Tet1 induced BDNF expression in post-mitotic neurons or activated MyoD facilitating reprogramming of fibroblasts into myoblasts, respectively. Targeted de novo methylation of a CTCF loop anchor site by dCas9-Dnmt3a blocked CTCF binding and interfered with DNA looping, causing altered gene expression in the neighboring loop. Finally, we show that these tools can edit DNA methylation in mice, demonstrating their wide utility for functional studies of epigenetic regulation.", 
"Diverse classes of RNA, ranging from small to long non-coding RNAs, have emerged as key regulators of gene expression, genome stability and defence against foreign genetic elements. Small RNAs modify chromatin structure and silence transcription by guiding Argonaute-containing complexes to complementary nascent RNA scaffolds and then mediating the recruitment of histone and DNA methyltransferases. In addition, recent advances suggest that chromatin-associated long non-coding RNA scaffolds also recruit chromatin-modifying complexes independently of small RNAs. These co-transcriptional silencing mechanisms form powerful RNA surveillance systems that detect and silence inappropriate transcription events, and provide a memory of these events via self-reinforcing epigenetic loops.", 
"Next-generation sequencing has revealed that more than 50% of human cancers harbour mutations in enzymes that are involved in chromatin organization. Tumour cells not only are activated by genetic and epigenetic alterations, but also routinely use epigenetic processes to ensure their escape from chemotherapy and host immune surveillance. Hence, a growing emphasis of recent drug discovery efforts has been on targeting the epigenome, including DNA methylation and histone modifications, with several new drugs being tested and some already approved by the US Food and Drug Administration (FDA). The future will see the increasing success of combining epigenetic drugs with other therapies. As epigenetic drugs target the epigenome as a whole, these true 'genomic medicines' lessen the need for precision approaches to individualized therapies.", 
"One-carbon (1C) metabolism, mediated by the folate cofactor, supports multiple physiological processes. These include biosynthesis (purines and thymidine), amino acid homeostasis (glycine, serine, and methionine), epigenetic maintenance, and redox defense. Both within eukaryotic cells and across organs, 1C metabolic reactions are compartmentalized. Here we review the fundamentals of mammalian 1C metabolism, including the pathways active in different compartments, cell types, and biological states. Emphasis is given to recent discoveries enabled by modern genetics, analytical chemistry, and isotope tracing. An emerging theme is the biological importance of mitochondrial 1C reactions, both for producing 1C units that are exported to the cytosol and for making additional products, including glycine and NADPH. Increased clarity regarding differential folate pathway usage in cancer, stem cells, development, and adult physiology is reviewed and highlights new opportunities for selective therapeutic intervention.", 
"Acetyl-coenzyme A (acetyl-CoA) is a central metabolic intermediate. The abundance of acetyl-CoA in distinct subcellular compartments reflects the general energetic state of the cell. Moreover, acetyl-CoA concentrations influence the activity or specificity of multiple enzymes, either in an allosteric manner or by altering substrate availability. Finally, by influencing the acetylation profile of several proteins, including histones, acetyl-CoA controls key cellular processes, including energy metabolism, mitosis, and autophagy, both directly and via the epigenetic regulation of gene expression. Thus, acetyl-CoA determines the balance between cellular catabolism and anabolism by simultaneously operating as a metabolic intermediate and as a second messenger.", 
"Cancer growth and progression are associated with immune suppression. Cancer cells have the ability to activate different immune checkpoint pathways that harbor immunosuppressive functions. Monoclonal antibodies that target immune checkpoints provided an immense breakthrough in cancer therapeutics. Among the immune checkpoint inhibitors, PD-1/PD-L1 and CTLA-4 inhibitors showed promising therapeutic outcomes, and some have been approved for certain cancer treatments, while others are under clinical trials. Recent reports have shown that patients with various malignancies benefit from immune checkpoint inhibitor treatment. However, mainstream initiation of immune checkpoint therapy to treat cancers is obstructed by the low response rate and immune-related adverse events in some cancer patients. This has given rise to the need for developing sets of biomarkers that predict the response to immune checkpoint blockade and immune-related adverse events. In this review, we discuss different predictive biomarkers for anti-PD-1/PD-L1 and anti-CTLA-4 inhibitors, including immune cells, PD-L1 overexpression, neoantigens, and genetic and epigenetic signatures. Potential approaches for further developing highly reliable predictive biomarkers should facilitate patient selection for and decision-making related to immune checkpoint inhibitor-based therapies.", 
"Nucleosome positioning varies between cell types. By deep sequencing cell-free DNA (cfDNA), isolated from circulating blood plasma, we generated maps of genome-wide in vivo nucleosome occupancy and found that short cfDNA fragments harbor footprints of transcription factors. The cfDNA nucleosome occupancies correlate well with the nuclear architecture, gene structure, and expression observed in cells, suggesting that they could inform the cell type of origin. Nucleosome spacing inferred from cfDNA in healthy individuals correlates most strongly with epigenetic features of lymphoid and myeloid cells, consistent with hematopoietic cell death as the normal source of cfDNA. We build on this observation to show how nucleosome footprints can be used to infer cell types contributing to cfDNA in pathological states such as cancer. Since this strategy does not rely on genetic differences to distinguish between contributing tissues, it may enable the noninvasive monitoring of a much broader set of clinical conditions than currently possible.", 
"The Cas9 protein (CRISPR-associated protein 9), derived from type II CRISPR (clustered regularly interspaced short palindromic repeats) bacterial immune systems, is emerging as a powerful tool for engineering the genome in diverse organisms. As an RNA-guided DNA endonuclease, Cas9 can be easily programmed to target new sites by altering its guide RNA sequence, and its development as a tool has made sequence-specific gene editing several magnitudes easier. The nuclease-deactivated form of Cas9 further provides a versatile RNA-guided DNA-targeting platform for regulating and imaging the genome, as well as for rewriting the epigenetic status, all in a sequence-specific manner. With all of these advances, we have just begun to explore the possible applications of Cas9 in biomedical research and therapeutics. In this review, we describe the current models of Cas9 function and the structural and biochemical studies that support it. We focus on the applications of Cas9 for genome editing, regulation, and imaging, discuss other possible applications and some technical considerations, and highlight the many advantages that CRISPR/Cas9 technology offers.", 
"Carcinogenesis cannot be explained only by genetic alterations, but also involves epigenetic processes. Modification of histones by acetylation plays a key role in epigenetic regulation of gene expression and is controlled by the balance between histone deacetylases (HDAC) and histone acetyltransferases (HAT). HDAC inhibitors induce cancer cell cycle arrest, differentiation and cell death, reduce angiogenesis and modulate immune response. Mechanisms of anticancer effects of HDAC inhibitors are not uniform; they may be different and depend on the cancer type, HDAC inhibitors, doses, etc. HDAC inhibitors seem to be promising anti-cancer drugs particularly in the combination with other anti-cancer drugs and/or radiotherapy. HDAC inhibitors vorinostat, romidepsin and belinostat have been approved for some T-cell lymphoma and panobinostat for multiple myeloma. Other HDAC inhibitors are in clinical trials for the treatment of hematological and solid malignancies. The results of such studies are promising but further larger studies are needed. Because of the reversibility of epigenetic changes during cancer development, the potency of epigenetic therapies seems to be of great importance. Here, we summarize the data on different classes of HDAC inhibitors, mechanisms of their actions and discuss novel results of preclinical and clinical studies, including the combination with other therapeutic modalities.", 
"The finding that small non-coding RNAs (ncRNAs) are able to control gene expression in a sequence specific manner has had a massive impact on biology. Recent improvements in high throughput sequencing and computational prediction methods have allowed the discovery and classification of several types of ncRNAs. Based on their precursor structures, biogenesis pathways and modes of action, ncRNAs are classified as small interfering RNAs (siRNAs), microRNAs (miRNAs), PIWI-interacting RNAs (piRNAs), endogenous small interfering RNAs (endo-siRNAs or esiRNAs), promoter associate RNAs (pRNAs), small nucleolar RNAs (snoRNAs) and sno-derived RNAs. Among these, miRNAs appear as important cytoplasmic regulators of gene expression. miRNAs act as post-transcriptional regulators of their messenger RNA (mRNA) targets via mRNA degradation and/or translational repression. However, it is becoming evident that miRNAs also have specific nuclear functions. Among these, the most studied and debated activity is the miRNA-guided transcriptional control of gene expression. Although available data detail quite precisely the effectors of this activity, the mechanisms by which miRNAs identify their gene targets to control transcription are still a matter of debate. Here, we focus on nuclear functions of miRNAs and on alternative mechanisms of target recognition, at the promoter lavel, by miRNAs in carrying out transcriptional gene silencing.", 
"Since decades it has been known that non-protein-coding RNAs have important cellular functions. Deep sequencing recently facilitated the discovery of thousands of novel transcripts, now classified as long noncoding RNAs (lncRNAs), in many vertebrate and invertebrate species. LncRNAs are involved in a wide range of cellular mechanisms, from almost all aspects of gene expression to protein translation and stability. Recent findings implicate lncRNAs as key players of cellular differentiation, cell lineage choice, organogenesis and tissue homeostasis. Moreover, lncRNAs are involved in pathological conditions such as cancer and cardiovascular disease, and therefore provide novel biomarkers and pharmaceutical targets. Here we discuss examples illustrating the versatility of lncRNAs in gene control, development and differentiation, as well as in human disease.", 
"Identifying and validating molecular targets of interventions that extend the human health span and lifespan has been difficult, as most clinical biomarkers are not sufficiently representative of the fundamental mechanisms of ageing to serve as their indicators. In a recent breakthrough, biomarkers of ageing based on DNA methylation data have enabled accurate age estimates for any tissue across the entire life course. These 'epigenetic clocks' link developmental and maintenance processes to biological ageing, giving rise to a unified theory of life course. Epigenetic biomarkers may help to address long-standing questions in many fields, including the central question: why do we age?", 
"Epithelial-mesenchymal transition (EMT) is a process in which epithelial cells acquire mesenchymal features. In cancer, EMT is associated with tumor initiation, invasion, metastasis, and resistance to therapy. Recently, it has been demonstrated that EMT is not a binary process, but occurs through distinct cellular states. Here, we review the recent studies that demonstrate the existence of these different EMT states in cancer and the mechanisms regulating their functions. We discuss the different functional characteristics, such as proliferation, propagation, plasticity, invasion, and metastasis associated with the distinct EMT states. We summarize the role of the transcriptional and epigenetic landscapes, gene regulatory network and their surrounding niche in controlling the transition through the different EMT states.", 
"Methylation of DNA and of histone 3 at Lys 9 (H3K9) are highly correlated with gene silencing in eukaryotes from fungi to humans. Both of these epigenetic marks need to be established at specific regions of the genome and then maintained at these sites through cell division. Protein structural domains that specifically recognize methylated DNA and methylated histones are key for targeting enzymes that catalyse these marks to appropriate genome sites. Genetic, genomic, structural and biochemical data reveal connections between these two epigenetic marks, and these domains mediate much of the crosstalk.", 
"This year is the tenth anniversary of the publication in this journal of a model suggesting the existence of 'tumour progenitor genes'. These genes are epigenetically disrupted at the earliest stages of malignancies, even before mutations, and thus cause altered differentiation throughout tumour evolution. The past decade of discovery in cancer epigenetics has revealed a number of similarities between cancer genes and stem cell reprogramming genes, widespread mutations in epigenetic regulators, and the part played by chromatin structure in cellular plasticity in both development and cancer. In the light of these discoveries, we suggest here a framework for cancer epigenetics involving three types of genes: 'epigenetic mediators', corresponding to the tumour progenitor genes suggested earlier; 'epigenetic modifiers' of the mediators, which are frequently mutated in cancer; and 'epigenetic modulators' upstream of the modifiers, which are responsive to changes in the cellular environment and often linked to the nuclear architecture. We suggest that this classification is helpful in framing new diagnostic and therapeutic approaches to cancer.", 
"Specification of primordial germ cells (PGCs) marks the beginning of the totipotent state. However, without a tractable experimental model, the mechanism of human PGC (hPGC) specification remains unclear. Here, we demonstrate specification of hPGC-like cells (hPGCLCs) from germline competent pluripotent stem cells. The characteristics of hPGCLCs are consistent with the embryonic hPGCs and a germline seminoma that share a CD38 cell-surface marker, which collectively defines likely progression of the early human germline. Remarkably, SOX17 is the key regulator of hPGC-like fate, whereas BLIMP1 represses endodermal and other somatic genes during specification of hPGCLCs. Notable mechanistic differences between mouse and human PGC specification could be attributed to their divergent embryonic development and pluripotent states, which might affect other early cell-fate decisions. We have established a foundation for future studies on resetting of the epigenome in hPGCLCs and hPGCs for totipotency and the transmission of genetic and epigenetic information.", 
"The discovery of long non-coding RNA (lncRNA) has dramatically altered our understanding of cancer. Here, we describe a comprehensive analysis of lncRNA alterations at transcriptional, genomic, and epigenetic levels in 5,037 human tumor specimens across 13 cancer types from The Cancer Genome Atlas. Our results suggest that the expression and dysregulation of lncRNAs are highly cancer type specific compared with protein- coding genes. Using the integrative data generated by this analysis, we present a clinically guided small interfering RNA screening strategy and a co-expression analysis approach to identify cancer driver lncRNAs and predict their functions. This provides a resource for investigating lncRNAs in cancer and lays the groundwork for the development of new diagnostics and treatments.", 
"Plant genonnes encode various small RNAs that function in distinct, yet overlapping, genetic and epigenetic silencing pathways. However, the abundance and diversity of small-RNA classes varies among plant species, suggesting coevolution between environmental adaptations and gene-silencing mechanisms. Biogenesis of small RNAs in plants is well understood, but we are just beginning to uncover their intricate regulation and activity. Here, we discuss the biogenesis of plant small RNAs, such as nnicroRNAs, secondary siRNAs and heterochromatic siRNAs, and their diverse cellular and developmental functions, including in reproductive transitions, genomic imprinting and paramutation. We also discuss the diversification of small-RNA-directed silencing pathways through the expansion of RNA-dependent RNA polymerases, DICER proteins and ARGONAUTE proteins.", 
"Epigenetic modifications, including DNA methylation, represent a potential mechanism for environmental impacts on human disease. Maternal smoking in pregnancy remains an important public health problem that impacts child health in a myriad of ways and has potential lifelong consequences. The mechanisms are largely unknown, but epigenetics most likely plays a role. We formed the Pregnancy And Childhood Epigenetics (PACE) consortium and meta-analyzed, across 13 cohorts (n = 6,685), the association between maternal smoking in pregnancy and newborn blood DNA methylation at over 450,000 CpG sites (CpGs) by using the Illumina 450K BeadChip. Over 6,000 CpGs were differentially methylated in relation to maternal smoking at genome-wide statistical significance (false discovery rate, 5%), including 2,965 CpGs corresponding to 2,017 genes not previously related to smoking and methylation in either newborns or adults. Several genes are relevant to diseases that can be caused by maternal smoking (e.g., orofacial clefts and asthma) or adult smoking (e.g., certain cancers). A number of differentially methylated CpGs were associated with gene expression. We observed enrichment in pathways and processes critical to development. In older children (5 cohorts, n = 3,187), 100% of CpGs gave at least nominal levels of significance, far more than expected by chance (p value < 2.2 x 10(-16)). Results were robust to different normalization methods used across studies and cell type adjustment. In this large scale meta-analysis of methylation data, we identified numerous loci involved in response to maternal smoking in pregnancy with persistence into later childhood and provide insights into mechanisms underlying effects of this important exposure.", 
"In 1964, Vincent Allfrey and colleagues reported the identification of histone acetylation and with deep insight proposed a regulatory role for this protein modification in transcription regulation. Subsequently, histone acetyltransferases (HATs), histone deacetylases (HDACs) and acetyl-Lys-binding proteins were identified as transcription regulators, thereby providing compelling evidence for his daring hypothesis. During the past 15 years, reversible protein acetylation and its modifying enzymes have been implicated in many cellular functions beyond transcription regulation. Here, we review the progress accomplished during the past 50 years and discuss the future of protein acetylation.", 
"N-6-methyladenosine (m(6)A) is the most prevalent and internal modification that occurs in the messenger RNAs (mRNA) of most eukaryotes, although its functional relevance remained a mystery for decades. This modification is installed by the m(6)A methylation writers and can be reversed by demethylases that serve as erasers. In this review, we mainly summarize recent progress in the study of the m(6)A mRNA methylation machineries across eukaryotes and discuss their newly uncovered biological functions. The broad roles of m(6)A in regulating cell fates and embryonic development highlight the existence of another layer of epigenetic regulation at the RNA level, where mRNA is subjected to chemical modifications that affect protein expression.", 
"The Bromo- and Extra-Terminal (BET) proteins BRD2, BRD3, and BRD4 play important roles in transcriptional regulation, epigenetics, and cancer and are the targets of pan-BET selective bromodomain inhibitor JQ1. However, the lack of intra-BET selectivity limits the scope of current inhibitors as probes for target validation and could lead to unwanted side effects or toxicity in a therapeutic setting. We designed Proteolysis Targeted Chimeras (PROTACs) that tether JQ1 to a ligand for the E3 ubiquitin ligase VHL, aimed at triggering the intracellular destruction of BET proteins. Compound MZ1 potently and rapidly induces reversible, long-lasting, and unexpectedly selective removal of BRD4 over BRD2 and BRD3. The activity of MZ1 is dependent on binding to VHL but is achieved at a sufficiently low concentration not to induce stabilization of HIF-1 alpha. Gene expression profiles of selected cancer-related genes responsive to JQ1 reveal distinct and more limited transcriptional responses induced by MZ1, consistent with selective suppression of BRD4. Our discovery opens up new opportunities to elucidate the cellular phenotypes and therapeutic implications associated with selective targeting of BRD4.", 
"We define the chromatin accessibility and transcriptional landscapes in 13 human primary blood cell types that span the hematopoietic hierarchy. Exploiting the finding that the enhancer landscape better reflects cell identity than mRNA levels, we enable 'enhancer cytometry' for enumeration of pure cell types from complex populations. We identify regulators governing hematopoietic differentiation and further show the lineage ontogeny of genetic elements linked to diverse human diseases. In acute myeloid leukemia (AML), chromatin accessibility uncovers unique regulatory evolution in cancer cells with a progressively increasing mutation burden. Single AML cells exhibit distinctive mixed regulome profiles corresponding to disparate developmental stages. A method to account for this regulatory heterogeneity identified cancer-specific deviations and implicated HOX factors as key regulators of preleukemic hematopoietic stem cell characteristics. Thus, regulome dynamics can provide diverse insights into hematopoietic development and disease.", 
"Treatment of cancer has been revolutionized by immune checkpoint blockade therapies. Despite the high rate of response in advanced melanoma, the majority of patients succumb to disease. To identify factors associated with success or failure of checkpoint therapy, we profiled transcriptomes of 16,291 individual immune cells from 48 tumor samples of melanoma patients treated with checkpoint inhibitors. Two distinct states of CD8(+) T cells were defined by clustering and associated with patient tumor regression or progression. A single transcription factor, TCF7, was visualized within CD8(+) T cells in fixed tumor samples and predicted positive clinical outcome in an independent cohort of checkpoint-treated patients. We delineated the epigenetic landscape and clonality of these T cell states and demonstrated enhanced antitumor immunity by targeting novel combinations of factors in exhausted cells. Our study of immune cell transcriptomes from tumors demonstrates a strategy for identifying predictors, mechanisms, and targets for enhancing checkpoint immunotherapy.", 
"The extent of tumor heterogeneity is an emerging theme that researchers are only beginning to understand. How genetic and epigenetic heterogeneity affects tumor evolution and clinical progression is unknown. The precise nature of the environmental factors that influence this heterogeneity is also yet to be characterized. Nature Medicine, Nature Biotechnology and the Volkswagen Foundation organized a meeting focused on identifying the obstacles that need to be overcome to advance translational research in and tumor heterogeneity. Once these key questions were established, the attendees devised potential solutions. Their ideas are presented here.", 
"Single-stranded guanine-rich DNA sequences can fold into four-stranded DNA structures called G-quadruplexes (G4s) that arise from the self-stacking of two or more guanine quartets. There has been considerable recent progress in the detection and mapping of G4 structures in the human genome and in biologically relevant contexts. These advancements, many of which align with predictions made previously in computational studies, provide important new insights into the functions of G4 structures in, for example, the regulation of transcription and genome stability, and uncover their potential relevance for cancer therapy.", 
"Humans show strong sex differences in immunity to infection and autoimmunity, suggesting sex hormones modulate immune responses. Indeed, receptors for estrogens (ERs) regulate cells and pathways in the innate and adaptive immune system, as well as immune cell development. ERs are ligand-dependent transcription factors that mediate long-range chromatin interactions and form complexes at gene regulatory elements, thus promoting epigenetic changes and transcription. ERs also participate in membrane-initiated steroid signaling to generate rapid responses. Estradiol and ER activity show profound dose- and context-dependent effects on innate immune signaling pathways and myeloid cell development. While estradiol most often promotes the production of type I interferon, innate pathways leading to pro-inflammatory cytokine production may be enhanced or dampened by ER activity. Regulation of innate immune cells and signaling by ERs may contribute to the reported sex differences in innate immune pathways. Here we review the recent literature and highlight several molecular mechanisms by which ERs regulate the development or functional responses of innate immune cells. (C) 2015 Elsevier Inc. All rights reserved.", 
"Epigenetic studies relied so far on correlations between epigenetic marks and gene expression pattern. Technologies developed for epigenome editing now enable direct study of functional relevance of precise epigenetic modifications and gene regulation. The reversible nature of epigenetic modifications, including DNA methylation, has been already exploited in cancer therapy for remodeling the aberrant epigenetic landscape. However, this was achieved non-selectively using epigenetic inhibitors. Epigenetic editing at specific loci represents a novel approach that might selectively and heritably alter gene expression. Here, we developed a CRISPR-Cas9-based tool for specific DNA methylation consisting of deactivated Cas9 (dCas9) nuclease and catalytic domain of the DNA methyltransferase DNMT3A targeted by co-expression of a guide RNA to any 20 bp DNA sequence followed by the NGG trinucleotide. We demonstrated targeted CpG methylation in a similar to 35 bp wide region by the fusion protein. We also showed that multiple guide RNAs could target the dCas9-DNMT3A construct to multiple adjacent sites, which enabled methylation of a larger part of the promoter. DNA methylation activity was specific for the targeted region and heritable across mitotic divisions. Finally, we demonstrated that directed DNA methylation of a wider promoter region of the target loci IL6ST and BACH2 decreased their expression.", 
"The DNA methyltransferase (DNMT) family comprises a conserved set of DNA-modifying enzymes that have a central role in epigenetic gene regulation. Recent studies have shown that the functions of the canonical DNMT enzymes-DNMT1, DNMT3A and DNMT3B-go beyond their traditional roles of establishing and maintaining DNA methylation patterns. This Review analyses how molecular interactions and changes in gene copy numbers modulate the activity of DNMTs in diverse gene regulatory functions, including transcriptional silencing, transcriptional activation and post-transcriptional regulation by DNMT2-dependent tRNA methylation. This mechanistic diversity enables the DNMT family to function as a versatile toolkit for epigenetic regulation.", 
"Estimates of biological age based on DNA methylation patterns, often referred to as epigenetic age, DNAm age, have been shown to be robust biomarkers of age in humans. We previously demonstrated that independent of chronological age, epigenetic age assessed in blood predicted all-cause mortality in four human cohorts. Here, we expanded our original observation to 13 different cohorts for a total sample size of 13,089 individuals, including three racial/ethnic groups. In addition, we examined whether incorporating information on blood cell composition into the epigenetic age metrics improves their predictive power for mortality. All considered measures of epigenetic age acceleration were predictive of mortality (p <= 8.2x10(-9)), independent of chronological age, even after adjusting for additional risk factors (p<5.4x10(-4)), and within the racial/ethnic groups that we examined (non-Hispanic whites, Hispanics, African Americans). Epigenetic age estimates that incorporated information on blood cell composition led to the smallest p-values for time to death (p=7.5x10(-43)). Overall, this study a) strengthens the evidence that epigenetic age predicts all-cause mortality above and beyond chronological age and traditional risk factors, and b) demonstrates that epigenetic age estimates that incorporate information on blood cell counts lead to highly significant associations with all-cause mortality.", 
"Childhood trauma confers higher risk of adulthood physical and mental illness; however, the biological mechanism mediating this association remains largely unknown. Recent research has suggested dysregulation of the immune system as a possible biological mediator. The present paper conducted a meta-analysis to establish whether early-life adversity contributes to potentially pathogenic pro-inflammatory phenotypes in adult individuals. A systematic search of Pubmed, PsycINFO, EMBASE, Scopus and Medline identified 25 articles for the meta-analysis, including 18 studies encompassing a sample of 16 870 individuals for C-reactive protein (CRP), 15 studies including 3751 individuals for interleukin-6 (IL-6) and 10 studies including 881 individuals for tumour necrosis factor-alpha (TNF-alpha). Random-effects meta-analysis showed that individuals exposed to childhood trauma had significantly elevated baseline peripheral levels of CRP (Fisher's z = 0.10, 95% confidence interval (CI) = 0.05-0.14), IL-6 (z = 0.08, 95% CI = 0.03-0.14) and TNF-alpha (z = 0.23, 95% CI = 0.14-0.32). Subgroup analyses for specific types of trauma (sexual, physical or emotional abuse) revealed that these impact differentially the single inflammatory markers. Moreover, meta-regression revealed greater effect sizes in clinical samples for the association between childhood trauma and CRP but not for IL-6 or TNF-alpha. Age, body mass index (BMI) and gender had no moderating effects. The analysis demonstrates that childhood trauma contributes to a pro-inflammatory state in adulthood, with specific inflammatory profiles depending on the specific type of trauma.", 
"The heart must continuously pump blood to supply the body with oxygen and nutrients. To maintain the high energy consumption required by this role, the heart is equipped with multiple complex biological systems that allow adaptation to changes of systemic demand. The processes of growth (hypertrophy), angiogenesis, and metabolic plasticity are critically involved in maintenance of cardiac homeostasis. Cardiac hypertrophy is classified as physiological when it is associated with normal cardiac function or as pathological when associated with cardiac dysfunction. Physiological hypertrophy of the heart occurs in response to normal growth of children or during pregnancy, as well as in athletes. In contrast, pathological hypertrophy is induced by factors such as prolonged and abnormal hemodynamic stress, due to hypertension, myocardial infarction etc. Pathological hypertrophy is associated with fibrosis, capillary rarefaction, increased production of pro-inflammatory cytokines, and cellular dysfunction (impairment of signaling, suppression of autophagy, and abnormal cardiomyocyte/non-cardiomyocyte interactions), as well as undesirable epigenetic changes, with these complex responses leading to maladaptive cardiac remodeling and heart failure. This review describes the key molecules and cellular responses involved in physiological/pathological cardiac hypertrophy. (C) 2016 Elsevier Ltd. All rights reserved.", 
"The process of aging results in a host of changes at the cellular and molecular levels, which include senescence, telomere shortening, and changes in gene expression. Epigenetic patterns also change over the lifespan, suggesting that epigenetic changes may constitute an important component of the aging process. The epigenetic mark that has been most highly studied is DNA methylation, the presence of methyl groups at CpG dinucleotides. These dinucleotides are often located near gene promoters and associate with gene expression levels. Early studies indicated that global levels of DNA methylation increase over the first few years of life and then decrease beginning in late adulthood. Recently, with the advent of microarray and next-generation sequencing technologies, increases in variability of DNA methylation with age have been observed, and a number of site-specific patterns have been identified. It has also been shown that certain CpG sites are highly associated with age, to the extent that prediction models using a small number of these sites can accurately predict the chronological age of the donor. Together, these observations point to the existence of two phenomena that both contribute to age-related DNA methylation changes: epigenetic drift and the epigenetic clock. In this review, we focus on healthy human aging throughout the lifetime and discuss the dynamics of DNA methylation as well as how interactions between the genome, environment, and the epigenome influence aging rates. We also discuss the impact of determining 'epigenetic age' for human health and outline some important caveats to existing and future studies.", 
"The ability to rapidly restore the integrity of a broken skin barrier is critical and is the ultimate goal of therapies for hard-to-heal-ulcers. Unfortunately effective treatments to enhance healing and reduce scarring are still lacking. A deeper understanding of the physiology of normal repair and of the pathology of delayed healing is a prerequisite for the development of more effective therapeutic interventions. Transition from the inflammatory to the proliferative phase is a key step during healing and accumulating evidence associates a compromised transition with wound healing disorders. Thus, targeting factors that impact this phase transition may offer a rationale for therapeutic development. This review summarizes mechanisms regulating the inflammation-proliferation transition at cellular and molecular levels. We propose that identification of such mechanisms will reveal promising targets for development of more effective therapies.", 
"Adult somatic stem cells in various organs maintain homeostatic tissue regeneration and enhance plasticity. Since its initial discovery five decades ago, investigations of adult neurogenesis and neural stem cells have led to an established and expanding field that has significantly influenced many facets of neuroscience, developmental biology, and regenerative medicine. Here we review recent progress and focus on questions related to adult mammalian neural stem cells that also apply to other somatic stem cells. We further discuss emerging topics that are guiding the field toward better understanding adult neural stem cells and ultimately applying these principles to improve human health.", 
"The dogma that adaptive immunity is the only arm of the immune response with memory capacity has been recently challenged by several studies demonstrating evidence for memory-like innate immune training. However, the underlying mechanisms and location for generating such innate memory responses in vivo remain unknown. Here, we show that access of Bacillus Calmette-Guerin (BCG) to the bone marrow (BM) changes the transcriptional landscape of hematopoietic stem cells (HSCs) and multipotent progenitors (MPPs), leading to local cell expansion and enhanced myelopoiesis at the expense of lymphopoiesis. Importantly, BCG-educated HSCs generate epigenetically modified macrophages that provide significantly better protection against virulent M. tuberculosis infection than naive macrophages. By using parabiotic and chimeric mice, as well as adoptive transfer approaches, we demonstrate that training of the monocyte/macrophage lineage via BCG-induced HSC reprogramming is sustainable in vivo. Our results indicate that targeting the HSC compartment provides a novel approach for vaccine development.", 
"Experimental modelling of human disorders enables the definition of the cellular and molecular mechanisms underlying diseases and the development of therapies for treating them. The availability of human pluripotent stem cells (PSCs), which are capable of self-renewal and have the potential to differentiate into virtually any cell type, can now help to overcome the limitations of animal models for certain disorders. The ability to model human diseases using cultured PSCs has revolutionized the ways in which we study monogenic, complex and epigenetic disorders, as well as early- and late-onset diseases. Several strategies are used to generate such disease models using either embryonic stem cells (ES cells) or patient-specific induced PSCs (iPSCs), creating new possibilities for the establishment of models and their use in drug screening.", 
"Resetting of the epigenome in human primordial germ cells (hPGCs) is critical for development. We show that the transcriptional program of hPGCs is distinct from that in mice, with co-expression of somatic specifiers and naive pluripotency genes TFCP2L1 and KLF4. This unique gene regulatory network, established by SOX17 and BLIMP1, drives comprehensive germline DNA demethylation by repressing DNA methylation pathways and activating TET-mediated hydroxymethylation. Base-resolution methylome analysis reveals progressive DNA demethylation to basal levels in week 5-7 in vivo hPGCs. Concurrently, hPGCs undergo chromatin reorganization, X reactivation, and imprint erasure. Despite global hypomethylation, evolutionarily young and potentially hazardous retroelements, like SVA, remain methylated. Remarkably, some loci associated with metabolic and neurological disorders are also resistant to DNA demethylation, revealing potential for transgenerational epigenetic inheritance that may have phenotypic consequences. We provide comprehensive insight on early human germline transcriptional network and epigenetic reprogramming that subsequently impacts human development and disease.", 
"The cBioPortal for Cancer Genomics (http://cbioportal.org) provides a Web resource for exploring, visualizing, and analyzing multidimensional cancer genomics data. The portal reduces molecular profiling data from cancer tissues and cell lines into readily understandable genetic, epigenetic, gene expression, and proteomic events. The query interface combined with customized data storage enables researchers to interactively explore genetic alterations across samples, genes, and pathways and, when available in the underlying data, to link these to clinical outcomes. The portal provides graphical summaries of gene-level data from multiple platforms, network visualization and analysis, survival analysis, patient-centric queries, and software programmatic access. The intuitive Web interface of the portal makes complex cancer genomics profiles accessible to researchers and clinicians without requiring bioinformatics expertise, thus facilitating biological discoveries. Here, we provide a practical guide to the analysis and visualization features of the cBioPortal for Cancer Genomics.", 
"Aging is characterized by a progressive loss of physiological integrity, leading to impaired function and increased vulnerability to death. This deterioration is the primary risk factor for major human pathologies, including cancer, diabetes, cardiovascular disorders, and neurodegenerative diseases. Aging research has experienced an unprecedented advance over recent years, particularly with the discovery that the rate of aging is controlled, at least to some extent, by genetic pathways and biochemical processes conserved in evolution. This Review enumerates nine tentative hallmarks that represent common denominators of aging in different organisms, with special emphasis on mammalian aging. These hallmarks are: genomic instability, telomere attrition, epigenetic alterations, loss of proteostasis, deregulated nutrient sensing, mitochondrial dysfunction, cellular senescence, stem cell exhaustion, and altered intercellular communication. A major challenge is to dissect the interconnectedness between the candidate hallmarks and their relative contributions to aging, with the final goal of identifying pharmaceutical targets to improve human health during aging, with minimal side effects.", 
"DNA methylation is frequently described as a 'silencing' epigenetic mark, and indeed this function of 5-methylcytosine was originally proposed in the 1970s. Now, thanks to improved genome-scale mapping of methylation, we can evaluate DNA methylation in different genomic contexts: transcriptional start sites with or without CpG islands, in gene bodies, at regulatory elements and at repeat sequences. The emerging picture is that the function of DNA methylation seems to vary with context, and the relationship between DNA methylation and transcription is more nuanced than we realized at first. Improving our understanding of the functions of DNA methylation is necessary for interpreting changes in this mark that are observed in diseases such as cancer.", 
"The Cancer Genome Atlas (TCGA) Research Network has profiled and analyzed large numbers of human tumors to discover molecular aberrations at the DNA, RNA, protein and epigenetic levels. The resulting rich data provide a major opportunity to develop an integrated picture of commonalities, differences and emergent themes across tumor lineages. The Pan-Cancer initiative compares the first 12 tumor types profiled by TCGA. Analysis of the molecular aberrations and their functional roles across tumor types will teach us how to extend therapies effective in one cancer type to others with a similar genomic profile.", 
"The central dogma of gene expression is that DNA is transcribed into messenger RNAs, which in turn serve as the template for protein synthesis. The discovery of extensive transcription of large RNA transcripts that do not code for proteins, termed long noncoding RNAs (lncRNAs), provides an important new perspective on the centrality of RNA in gene regulation. Here, we discuss genome-scale strategies to discover and characterize lncRNAs. An emerging theme from multiple model systems is that lncRNAs form extensive networks of ribonucleoprotein (RNP) complexes with numerous chromatin regulators and then target these enzymatic activities to appropriate locations in the genome. Consistent with this notion, lncRNAs can function as modular scaffolds to specify higher-order organization in RNP complexes and in chromatin states. The importance of these modes of regulation is underscored by the newly recognized roles of long RNAs for proper gene control across all kingdoms of life.", 
"Metastases represent the end products of a multistep cell-biological process termed the invasion-metastasis cascade, which involves dissemination of cancer cells to anatomically distant organ sites and their subsequent adaptation to foreign tissue microenvironments. Each of these events is driven by the acquisition of genetic and/or epigenetic alterations within tumor cells and the co-option of nonneoplastic stromal cells, which together endow incipient metastatic cells with traits needed to generate macroscopic metastases. Recent advances provide provocative insights into these cell-biological and molecular changes, which have implications regarding the steps of the invasion-metastasis cascade that appear amenable to therapeutic targeting.", 
"Background: It is not yet known whether DNA methylation levels can be used to accurately predict age across a broad spectrum of human tissues and cell types, nor whether the resulting age prediction is a biologically meaningful measure. Results: I developed a multi-tissue predictor of age that allows one to estimate the DNA methylation age of most tissues and cell types. The predictor, which is freely available, was developed using 8,000 samples from 82 Illumina DNA methylation array datasets, encompassing 51 healthy tissues and cell types. I found that DNA methylation age has the following properties: first, it is close to zero for embryonic and induced pluripotent stem cells; second, it correlates with cell passage number; third, it gives rise to a highly heritable measure of age acceleration; and, fourth, it is applicable to chimpanzee tissues. Analysis of 6,000 cancer samples from 32 datasets showed that all of the considered 20 cancer types exhibit significant age acceleration, with an average of 36 years. Low age-acceleration of cancer tissue is associated with a high number of somatic mutations and TP53 mutations, while mutations in steroid receptors greatly accelerate DNA methylation age in breast cancer. Finally, I characterize the 353 CpG sites that together form an aging clock in terms of chromatin states and tissue variance. Conclusions: I propose that DNA methylation age measures the cumulative effect of an epigenetic maintenance system. This novel epigenetic clock can be used to address a host of questions in developmental biology, cancer and aging research.", 
"Cytosine DNA methylation is a stable epigenetic mark that is crucial for diverse biological processes, including gene and transposon silencing, imprinting and X chromosome inactivation. Recent findings in plants and animals have greatly increased our understanding of the pathways used to accurately target, maintain and modify patterns of DNA methylation and have revealed unanticipated mechanistic similarities between these organisms. Key roles have emerged for small RNAs, proteins with domains that bind methylated DNA and DNA glycosylases in these processes. Drawing on insights from both plants and animals should deepen our understanding of the regulation and biological significance of DNA methylation.", 
"A flexible statistical framework is developed for the analysis of read counts from RNA-Seq gene expression studies. It provides the ability to analyse complex experiments involving multiple treatment conditions and blocking variables while still taking full account of biological variation. Biological variation between RNA samples is estimated separately from the technical variation associated with sequencing technologies. Novel empirical Bayes methods allow each gene to have its own specific variability, even when there are relatively few biological replicates from which to estimate such variability. The pipeline is implemented in the edgeR package of the Bioconductor project. A case study analysis of carcinoma data demonstrates the ability of generalized linear model methods (GLMs) to detect differential expression in a paired design, and even to detect tumour-specific expression changes. The case study demonstrates the need to allow for gene-specific variability, rather than assuming a common dispersion across genes or a fixed relationship between abundance and variability. Genewise dispersions de-prioritize genes with inconsistent results and allow the main analysis to focus on changes that are consistent between biological replicates. Parallel computational approaches are developed to make non-linear model fitting faster and more reliable, making the application of GLMs to genomic data more convenient and practical. Simulations demonstrate the ability of adjusted profile likelihood estimators to return accurate estimators of biological variability in complex situations. When variation is gene-specific, empirical Bayes estimators provide an advantageous compromise between the extremes of assuming common dispersion or separate genewise dispersion. The methods developed here can also be applied to count data arising from DNA-Seq applications, including ChIP-Seq for epigenetic marks and DNA methylation analyses.", 
"Cancer-associated IDH mutations are characterized by neomorphic enzyme activity and resultant 2-hydroxyglutarate (2HG) production. Mutational and epigenetic profiling of a large acute myeloid leukemia (AML) patient cohort revealed that IDH1/2-mutant AMLs display global DNA hypermethylation and a specific hypermethylation signature. Furthermore, expression of 2HG-producing IDH alleles in cells induced global DNA hypermethylation. In the AML cohort, IDH1/2 mutations were mutually exclusive with mutations in the alpha-ketoglutarate-dependent enzyme TET2, and TET2 loss-of-function mutations were associated with similar epigenetic defects as IDH1/2 mutants. Consistent with these genetic and epigenetic data, expression of IDH mutants impaired TET2 catalytic function in cells. Finally, either expression of mutant IDH1/2 or Tet2 depletion impaired hematopoietic differentiation and increased stem/progenitor cell marker expression, suggesting a shared proleukemogenic effect.", 
"The epigenetic regulation of DNA-templated processes has been intensely studied over the last 15 years. DNA methylation, histone modification, nucleosome remodeling, and RNA-mediated targeting regulate many biological processes that are fundamental to the genesis of cancer. Here, we present the basic principles behind these epigenetic pathways and highlight the evidence suggesting that their misregulation can culminate in cancer. This information, along with the promising clinical and preclinical results seen with epigenetic drugs against chromatin regulators, signifies that it is time to embrace the central role of epigenetics in cancer.", 
"DNA methylation is among the best studied epigenetic modifications and is essential to mammalian development. Although the methylation status of most CpG dinucteotides in the genome is stably propagated through mitosis, improvements to methods for measuring methylation have identified numerous regions in which it is dynamically regulated. In this Review, we discuss key concepts in the function of DNA methylation in mammals, stemming from more than two decades of research, including many recent studies that have elucidated when and where DNA methylation has a regulatory role in the genome. We include insights from early development, embryonic stem cells and adult Lineages, particularly haematopoiesis, to highlight the general features of this modification as it participates in both global and localized epigenetic regulation..", 
"Methylation of the N-6 position of adenosine (m(6)A) is a posttranscriptional modification of RNA with poorly understood prevalence and physiological relevance. The recent discovery that FTO, an obesity risk gene, encodes an m(6)A demethylase implicates m(6)A as an important regulator of physiological processes. Here, we present a method for transcriptome-wide m(6)A localization, which combines m(6)A-specific methylated RNA immunoprecipitation with next-generation sequencing (MeRIP-Seq). We use this method to identify mRNAs of 7,676 mammalian genes that contain m(6)A, indicating that m(6)A is a common base modification of mRNA. The m(6)A modification exhibits tissue-specific regulation and is markedly increased throughout brain development. We find that m(6)A sites are enriched near stop codons and in 3' UTRs, and we uncover an association between m 6 A residues and microRNA-binding sites within 3' UTRs. These findings provide a resource for identifying transcripts that are substrates for adenosine methylation and reveal insights into the epigenetic regulation of the mammalian transcriptome.", 
"Chromatin regulators have become attractive targets for cancer therapy, but it is unclear why inhibition of these ubiquitous regulators should have gene-specific effects in tumor cells. Here, we investigate how inhibition of the widely expressed transcriptional coactivator BRD4 leads to selective inhibition of the MYC oncogene in multiple myeloma (MM). BRD4 and Mediator were found to co-occupy thousands of enhancers associated with active genes. They also co-occupied a small set of exceptionally large super-enhancers associated with genes that feature prominently in MM biology, including the MYC oncogene. Treatment of MM tumor cells with the BET-bromodomain inhibitor JQ1 led to preferential loss of BRD4 at super-enhancers and consequent transcription elongation defects that preferentially impacted genes with super-enhancers, including MYC. Super-enhancers were found at key oncogenic drivers in many other tumor cells. These observations have implications for the discovery of cancer therapeutics directed at components of super-enhancers in diverse tumor types.", 
"The ability to measure human aging from molecular profiles has practical implications in many fields, including disease prevention and treatment, forensics, and extension of life. Although chronological age has been linked to changes in DNA methylation, the methylome has not yet been used to measure and compare human aging rates. Here, we build a quantitative model of aging using measurements at more than 450,000 CpG markers from the whole blood of 656 human individuals, aged 19 to 101. This model measures the rate at which an individual's methylome ages, which we show is impacted by gender and genetic variants. We also show that differences in aging rates help explain epigenetic drift and are reflected in the transcriptome. Moreover, we show how our aging model is upheld in other human tissues and reveals an advanced aging rate in tumor tissue. Our model highlights specific components of the aging process and provides a quantitative readout for studying the role of methylation in age-related disease.", 
"Organisms require an appropriate balance of stability and reversibility in gene expression programmes to maintain cell identity or to enable responses to stimuli; epigenetic regulation is integral to this dynamic control. Post-translational modification of histones by methylation is an important and widespread type of chromatin modification that is known to influence biological processes in the context of development and cellular responses. To evaluate how histone methylation contributes to stable or reversible control, we provide a broad overview of how histone methylation is regulated and leads to biological outcomes. The importance of appropriately maintaining or reprogramming histone methylation is illustrated by its links to disease and ageing and possibly to transmission of traits across generations.", 
"Chromosomes are the physical realization of genetic information and thus form the basis for its readout and propagation. Here we present a high-resolution chromosomal contact map derived from a modified genome-wide chromosome conformation capture approach applied to Drosophila embryonic nuclei. The data show that the entire genome is linearly partitioned into well-demarcated physical domains that overlap extensively with active and repressive epigenetic marks. Chromosomal contacts are hierarchically organized between domains. Global modeling of contact density and clustering of domains show that inactive domains are condensed and confined to their chromosomal territories, whereas active domains reach out of the territory to form remote intra-and interchromosomal contacts. Moreover, we systematically identify specific long-range intrachromosomal contacts between Polycomb-repressed domains. Together, these observations allow for quantitative prediction of the Drosophila chromosomal contact map, laying the foundation for detailed studies of chromosome structure and function in a genetically tractable system.", 
"Hybridization has many and varied impacts on the process of speciation. Hybridization may slow or reverse differentiation by allowing gene flow and recombination. It may accelerate speciation via adaptive introgression or cause near-instantaneous speciation by allopolyploidization. It may have multiple effects at different stages and in different spatial contexts within a single speciation event. We offer a perspective on the context and evolutionary significance of hybridization during speciation, highlighting issues of current interest and debate. In secondary contact zones, it is uncertain if barriers to gene flow will be strengthened or broken down due to recombination and gene flow. Theory and empirical evidence suggest the latter is more likely, except within and around strongly selected genomic regions. Hybridization may contribute to speciation through the formation of new hybrid taxa, whereas introgression of a few loci may promote adaptive divergence and so facilitate speciation. Gene regulatory networks, epigenetic effects and the evolution of selfish genetic material in the genome suggest that the DobzhanskyMuller model of hybrid incompatibilities requires a broader interpretation. Finally, although the incidence of reinforcement remains uncertain, this and other interactions in areas of sympatry may have knock-on effects on speciation both within and outside regions of hybridization.", 
"Glioblastoma (GBM) is a brain tumor that carries a dismal prognosis and displays considerable heterogeneity. We have recently identified recurrent H3F3A mutations affecting two critical amino acids (K27 and G34) of histone H3.3 in one-third of pediatric GBM. Here, we show that each H3F3A mutation defines an epigenetic subgroup of GBM with a distinct global methylation pattern, and that they are mutually exclusive with IDH1 mutations, which characterize a third mutation-defined subgroup. Three further epigenetic subgroups were enriched for hallmark genetic events of adult GBM and/or established transcriptomic signatures. We also demonstrate that the two H3F3A mutations give rise to GBMs in separate anatomic compartments, with differential regulation of transcription factors OLIG1, OLIG2, and FOXG1, possibly reflecting different cellular origins.", 
"Epigenetic phenomena in animals and plants are mediated by DNA methylation and stable chromatin modifications. There has been considerable interest in whether environmental factors modulate the establishment and maintenance of epigenetic modifications, and could thereby influence gene expression and phenotype. Chemical pollutants, dietary components, temperature changes and other external stresses can indeed have long-lasting effects on development, metabolism and health, sometimes even in subsequent generations. Although the underlying mechanisms remain largely unknown, particularly in humans, mechanistic insights are emerging from experimental model systems. These have implications for structuring future research and understanding disease and development.", 
"Methylation of cytosine bases in DNA provides a layer of epigenetic control in many eukaryotes that has important implications for normal biology and disease. Therefore, profiling DNA methylation across the genome is vital to understanding the influence of epigenetics. There has been a revolution in DNA methylation analysis technology over the past decade: analyses that previously were restricted to specific loci can now be performed on a genome-scale and entire methylomes can be characterized at single-base-pair resolution. However, there is such a diversity of DNA methylation profiling techniques that it can be challenging to select one. This Review discusses the different approaches and their relative merits and introduces considerations for data analysis.", 
"We have developed a new generation of genome-wide DNA methylation Bead Chip which allows high-throughput methylation profiling of the human genome. The new high density BeadChip can assay over 480K CpG sites and analyze twelve samples in parallel. The innovative content includes coverage of 99% of RefSeq genes with multiple probes per gene, 96% of CpG islands from the UCSC database, CpG island shores and additional content selected from whole-genome bisulfite sequencing data and input from DNA methylation experts. The well-characterized Infinium (R) Assay is used for analysis of CpG methylation using bisulfite-converted genomic DNA. We applied this technology to analyze DNA methylation in normal and tumor DNA samples and compared results with whole-genome bisulfite sequencing (WGBS) data obtained for the same samples. Highly comparable DNA methylation profiles were generated by the array and sequencing methods (average R-2 of 0.95). The ability to determine genome-wide methylation patterns will rapidly advance methylation research. (C) 2011 Elsevier Inc. All rights reserved.", 
"Expression of the INK4b/ARF/INK4a tumor suppressor locus in normal and cancerous cell growth is controlled by methylation of histone H3 at lysine 27 (H3K27me) as directed by the Polycomb group proteins. The antisense noncoding RNA ANRIL of the INK4b/ARF/INK4a locus is also important for expression of the protein-coding genes in cis, but its mechanism has remained elusive. Here we report that chromobox 7 (CBX7) within the polycomb repressive complex 1 binds to ANRIL, and both CBX7 and ANRIL are found at elevated levels in prostate cancer tissues. In concert with H3K27me recognition, binding to RNA contributes to CBX7 function, and disruption of either interaction impacts the ability of CBX7 to repress the INK4b/ARF/INK4a locus and control senescence. Structure-guided analysis reveals the molecular interplay between noncoding RNA and H3K27me as mediated by the conserved chromodomain. Our study suggests a mechanism by which noncoding RNA participates directly in epigenetic transcriptional repression.", 
"Determining how somatic copy number alterations (SCNAs) promote cancer is an important goal. We characterized SCNA patterns in 4,934 cancers from The Cancer Genome Atlas Pan-Cancer data set. Whole-genome doubling, observed in 37% of cancers, was associated with higher rates of every other type of SCNA, TP53 mutations, CCNE1 amplifications and alterations of the PPP2R complex. SCNAs that were internal to chromosomes tended to be shorter than telomere-bounded SCNAs, suggesting different mechanisms underlying their generation. Significantly recurrent focal SCNAs were observed in 140 regions, including 102 without known oncogene or tumor suppressor gene targets and 50 with significantly mutated genes. Amplified regions without known oncogenes were enriched for genes involved in epigenetic regulation. When levels of genomic disruption were accounted for, 7% of region pairs were anticorrelated, and these regions tended to encompass genes whose proteins physically interact, suggesting related functions. These results provide insights into mechanisms of generation and functional consequences of cancer-related SCNAs.", 
"High-throughput technologies are widely used, for example to assay genetic variants, gene and protein expression, and epigenetic modifications. One often overlooked complication with such studies is batch effects, which occur because measurements are affected by laboratory conditions, reagent lots and personnel differences. This becomes a major problem when batch effects are correlated with an outcome of interest and lead to incorrect conclusions. Using both published studies and our own analyses, we argue that batch effects (as well as other technical and biological artefacts) are widespread and critical to address. We review experimental and computational approaches for doing so.", 
"Cytosine methylation is the major covalent modification of mammalian genomic DNA and plays important roles in transcriptional regulation. The molecular mechanism underlying the enzymatic removal of this epigenetic mark, however, remains elusive. Here, we show that 5-methylcytosine (5mC) hydroxylase TET1, by converting 5mCs to 5-hydroxymethylcytosines (5hmCs), promotes DNA demethylation in mammalian cells through a process that requires the base excision repair pathway. Though expression of the 12 known human DNA glycosylases individually did not enhance removal of 5hmCs in mammalian cells, demethylation of both exogenously introduced and endogenous 5hmCs is promoted by the AID (activation-induced deaminase)/APOBEC (apolipoprotein B mRNA-editing enzyme complex) family of cytidine deaminases. Furthermore, Tet1 and Apobec1 are involved in neuronal activity-induced, region-specific, active DNA demethylation and subsequent gene expression in the dentate gyrus of the adult mouse brain in vivo. Our study suggests a TET1-induced oxidation-deamination mechanism for active DNA demethylation in mammals.", 
"Nuclear factor-erythroid 2 p45-related factor 2 (Nrf2, also called Nfe212) is a transcription factor that regulates the cellular redox status. Nrf2 is controlled through a complex transcriptional/epigenetic and post-translational network that ensures its activity increases during redox perturbation, inflammation, growth factor stimulation and nutrient/energy fluxes, thereby enabling the factor to orchestrate adaptive responses to diverse forms of stress. Besides mediating stress-stimulated induction of antioxidant and detoxification genes, Nr12 contributes to adaptation by upregulating the repair and degradation of damaged macromolecules, and by modulating intermediary metabolism. In the latter case, Nrf2 inhibits lipogenesis, supports 0-oxidation of fatty acids, facilitates flux through the pentose phosphate pathway, and increases NADPH regeneration and purine biosynthesis; these observations suggest Nrf2 directs metabolic reprogramming during stress.", 
"Genomes of complex organisms encode an abundance and diversity of long noncoding RNAs (IncRNAs) that are expressed throughout the cell and fulfill a wide variety of regulatory roles at almost every stage of gene expression. These roles, which encompass sensory, guiding, scaffolding and allosteric capacities, derive from folded modular domains in IncRNAs. In this diverse functional repertoire, we focus on the well-characterized ability for IncRNAs to function as epigenetic modulators. Many IncRNAs bind to chromatin-modifying proteins and recruit their catalytic activity to specific sites in the genome, thereby modulating chromatin states and impacting gene expression. Considering this regulatory potential in combination with the abundance of IncRNAs suggests that IncRNAs may be part of a broad epigenetic regulatory network.", 
"Bromodomains (BRDs) are protein interaction modules that specifically recognize epsilon-N-lysine acetylation motifs, a key event in the reading process of epigenetic marks. The 61 BRDs in the human genome cluster into eight families based on structure/sequence similarity. Here, we present 29 high-resolution crystal structures, covering all BRD families. Comprehensive crossfamily structural analysis identifies conserved and family-specific structural features that are necessary for specific acetylation-dependent substrate recognition. Screening of more than 30 representative BRDs against systematic histone-peptide arrays identifies new BRD substrates and reveals a strong influence of flanking posttranslational modifications, such as acetylation and phosphorylation, suggesting that BRDs recognize combinations of marks rather than singly acetylated sequences. We further uncovered a structural mechanism for the simultaneous binding and recognition of diverse diacetyl-containing peptides by BRD4. These data provide a foundation for structure-based drug design of specific inhibitors for this emerging target family.", 
"Epigenetics is one of the most promising and expanding fields in the current biomedical research landscape. Since the inception of epigenetics in the 1940s, the discoveries regarding its implications in normal and disease biology have not stopped, compiling a vast amount of knowledge in the past decade. The field has moved from just one recognized marker, DNA methylation, to a variety of others, including a wide spectrum of histone modifications. From the methodological standpoint, the successful initial single gene candidate approaches have been complemented by the current comprehensive epigenomic approaches that allow the interrogation of genomes to search for translational applications in an unbiased manner. Most important, the discovery of mutations in the epigenetic machinery and the approval of the first epigenetic drugs for the treatment of subtypes of leukemias and lymphomas has been an eye-opener for many biomedical scientists and clinicians. Herein, we will summarize the progress in the field of cancer epigenetics research that has reached mainstream oncology in the development of new biomarkers of the disease and new pharmacological strategies.", 
"Abnormalities of chromosome 7q are common in myeloid malignancies, but no specific target genes have yet been identified. Here, we describe the finding of homozygous EZH2 mutations in 9 of 12 individuals with 7q acquired uniparental disomy. Screening of a total of 614 individuals with myeloid disorders revealed 49 monoallelic or biallelic EZH2 mutations in 42 individuals; the mutations were found most commonly in those with myelodysplastic/myeloproliferative neoplasms (27 out of 219 individuals, or 12%) and in those with myelofibrosis (4 out of 30 individuals, or 13%). EZH2 encodes the catalytic subunit of the polycomb repressive complex 2 (PRC2), a highly conserved histone H3 lysine 27 (H3K27) methyltransferase that influences stem cell renewal by epigenetic repression of genes involved in cell fate decisions. EZH2 has oncogenic activity, and its overexpression has previously been causally linked to differentiation blocks in epithelial tumors. Notably, the mutations we identified resulted in premature chain termination or direct abrogation of histone methyltransferase activity, suggesting that EZH2 acts as a tumor suppressor for myeloid malignancies.", 
"With rare exceptions, spontaneous tumors originate from a single cell. Yet, at the time of clinical diagnosis, the majority of human tumors display startling heterogeneity in many morphological and physiological features, such as expression of cell surface receptors, proliferative and angiogenic potential. To a substantial extent, this heterogeneity might be attributed to morphological and epigenetic plasticity, but there is also strong evidence for the co-existence of genetically divergent tumor cell clones within tumors. In this perspective, we summarize the sources of intra-tumor phenotypic heterogeneity with emphasis on genetic heterogeneity. We review experimental evidence for the existence of both intra-tumor clonal heterogeneity as well as frequent evolutionary divergence between primary tumors and metastatic outgrowths. Furthermore, we discuss potential biological and clinical implications of intra-tumor clonal heterogeneity. (C) 2009 Elsevier B.V. All rights reserved.", 
"Since the human genome was sequenced, the term ``epigenetics'' is increasingly being associated with the hope that we are more than just the sum of our genes. Might what we eat, the air we breathe, or even the emotions we feel influence not only our genes but those of descendants? The environment can certainly influence gene expression and can lead to disease, but transgenerational consequences are another matter. Although the inheritance of epigenetic characters can certainly occur-particularly in plants-how much is due to the environment and the extent to which it happens in humans remain unclear.", 
"The incorporation of histone H3 variants has been implicated in the epigenetic memory of cellular state. Using genome editing with zinc-finger nucleases to tag endogenous H3.3, we report genome-wide profiles of H3 variants in mammalian embryonic stem cells and neuronal precursor cells. Genome-wide patterns of H3.3 are dependent on amino acid sequence and change with cellular differentiation at developmentally regulated loci. The H3.3 chaperone Hira is required for H3.3 enrichment at active and repressed genes. Strikingly, Hira is not essential for localization of H3.3 at telomeres and many transcription factor binding sites. Immunoaffinity purification and mass spectrometry reveal that the proteins Atrx and Daxx associate with H3.3 in a Hira-independent manner. Atrx is required for Hira-independent localization of H3.3 at telomeres and for the repression of telomeric RNA. Our data demonstrate that multiple and distinct factors are responsible for H3.3 localization at specific genomic locations in mammalian cells.", 
"Cyclin-dependent kinases (Cdks) are serine/threonine kinases and their catalytic activities are modulated by interactions with cyclins and Cdk inhibitors (CKIs). Close cooperation between this trio is necessary for ensuring orderly progression through the cell cycle. In addition to their well-established function in cell cycle control, it is becoming increasingly apparent that mammalian Cdks, cyclins and CKIs play indispensable roles in processes such as transcription, epigenetic regulation, metabolism, stem cell self-renewal, neuronal functions and spermatogenesis. Even more remarkably, they can accomplish some of these tasks individually, without the need for Cdk/cyclin complex formation or kinase activity. In this Review, we discuss the latest revelations about Cdks, cyclins and CKIs with the goal of showcasing their functional diversity beyond cell cycle regulation and their impact on development and disease in mammals.", 
"During the course of malignant cancer progression, neoplastic cells undergo dynamic and reversible transitions between multiple phenotypic states, the extremes of which are defined by the expression of epithelial and mesenchymal phenotypes. This plasticity is enabled by underlying shifts in epigenetic regulation. A small cohort of pleiotropically acting transcription factors is widely recognized to effect these shifts by controlling the expression of a constituency of key target genes. These master regulators depend on complex epigenetic regulatory mechanisms, notably the induction of changes in the modifications of chromatin-associated histones, in order to achieve the widespread changes in gene expression observed during epithelial-mesenchymal transitions (EMTs). These associations indicate that an understanding of the functional interactions between such EMT-inducing transcription factors and the modulators of chromatin configuration will provide crucial insights into the fundamental mechanisms underlying cancer progression and may, in the longer term, generate new diagnostic and therapeutic modalities for treating high-grade malignancies.", 
"Despite the success of genome-wide association studies (GWASs) in identifying loci associated with common diseases, a substantial proportion of the causality remains unexplained. Recent advances in genomic technologies have placed us in a position to initiate large-scale studies of human disease-associated epigenetic variation, specifically variation in DNA methylation. Such epigenome-wide association studies (EWASs) present novel opportunities but also create new challenges that are not encountered in GWASs. We discuss EWAS design, cohort and sample selections, statistical significance and power, confounding factors and follow-up studies. We also discuss how integration of EWASs with GWASs can help to dissect complex GWAS haplotypes for functional analysis.", 
"Discoveries over the past decade portend a paradigm shift in molecular biology. Evidence suggests that RNA is not only functional as a messenger between DNA and protein but also involved in the regulation of genome organization and gene expression, which is increasingly elaborate in complex organisms. Regulatory RNA seems to operate at many levels; in particular, it plays an important part in the epigenetic processes that control differentiation and development. These discoveries suggest a central role for RNA in human evolution and ontogeny. Here, we review the emergence of the previously unsuspected world of regulatory RNA from a historical perspective.", 
"DNA methylation, an important type of epigenetic modification in humans, participates in crucial cellular processes, such as embryonic development, X-inactivation, genomic imprinting and chromosome stability. Several platforms have been developed to study genome-wide DNA methylation. Many investigators in the field have chosen the Illumina Infinium HumanMethylation microarray for its ability to reliably assess DNA methylation following sodium bisulfite conversion. Here, we analyzed methylation profiles of 489 adult males and 357 adult females generated by the Infinium HumanMethylation450 microarray. Among the autosomal CpG sites that displayed significant methylation differences between the two sexes, we observed a significant enrichment of cross-reactive probes co-hybridizing to the sex chromosomes with more than 94% sequence identity. This could lead investigators to mistakenly infer the existence of significant autosomal sex-associated methylation. Using sequence identity cutoffs derived from the sex methylation analysis, we concluded that 6% of the array probes can potentially generate spurious signals because of co-hybridization to alternate genomic sequences highly homologous to the intended targets. Additionally, we discovered probes targeting polymorphic CpGs that overlapped SNPs. The methylation levels detected by these probes are simply the reflection of underlying genetic polymorphisms but could be misinterpreted as true signals. The existence of probes that are cross-reactive or of target polymorphic CpGs in the Illumina HumanMethylation microarrays can confound data obtained from such microarrays. Therefore, investigators should exercise caution when significant biological associations are found using these array platforms. A list of all cross-reactive probes and polymorphic CpGs identified by us are annotated in this paper.", 
"Cancer therapy is challenged by the diversity of molecular implementations of oncogenic processes and by the resulting variation in therapeutic responses. Projects such as The Cancer Genome Atlas (TCGA) provide molecular tumor maps in unprecedented detail. The interpretation of these maps remains a major challenge. Here we distilled thousands of genetic and epigenetic features altered in cancers to similar to 500 selected functional events (SFEs). Using this simplified description, we derived a hierarchical classification of 3,299 TCGA tumors from 12 cancer types. The top classes are dominated by either mutations (M class) or copy number changes (C class). This distinction is clearest at the extremes of genomic instability, indicating the presence of different oncogenic processes. The full hierarchy shows functional event patterns characteristic of multiple cross-tissue groups of tumors, termed oncogenic signature classes. Targetable functional events in a tumor class are suggestive of class-specific combination therapy. These results may assist in the definition of clinical trials to match actionable oncogenic signatures with personalized therapies.", 
"DNA methylation is one of the best-characterized epigenetic modifications and has been implicated in numerous biological processes, including transposable element silencing, genomic imprinting and X chromosome inactivation. Compared with other epigenetic modifications, DNA methylation is thought to be relatively stable. Despite its role in long-term silencing, DNA methylation is more dynamic than originally thought as active DNA demethylation has been observed during specific stages of development. In the past decade, many enzymes have been proposed to carry out active DNA demethylation and growing evidence suggests that, depending on the context, this process may be achieved by multiple mechanisms. Insight into how DNA methylation is dynamically regulated will broaden our understanding of epigenetic regulation and have great implications in somatic cell reprogramming and regenerative medicine.", 
"DNA methylation is one of the most intensely studied epigenetic modifications in mammals. In normal cells, it assures the proper regulation of gene expression and stable gene silencing. DNA methylation is associated with histone modifications and the interplay of these epigenetic modifications is crucial to regulate the functioning of the genome by changing chromatin architecture. The covalent addition of a methyl group occurs generally in cytosine within CpG dinucleotides which are concentrated in large clusters called CpG islands. DNA methyltransferases are responsible for establishing and maintenance of methylation pattern. It is commonly known that inactivation of certain tumor-suppressor genes occurs as a consequence of hypermethylation within the promoter regions and a numerous studies have demonstrated a broad range of genes silenced by DNA methylation in different cancer types. On the other hand, global hypomethylation, inducing genomic instability, also contributes to cell transformation. Apart from DNA methylation alterations in promoter regions and repetitive DNA sequences, this phenomenon is associated also with regulation of expression of noncoding RNAs such as microRNAs that may play role in tumor suppression. DNA methylation seems to be promising in putative translational use in patients and hypermethylated promoters may serve as biomarkers. Moreover, unlike genetic alterations, DNA methylation is reversible what makes it extremely interesting for therapy approaches. The importance of DNA methylation alterations in tumorigenesis encourages us to decode the human epigenome. Different DNA methylome mapping techniques are indispensable to realize this project in the future. (C) 2010, Elsevier Inc.", 
"Epigenetic information can be inherited through the mammalian germline and represents a plausible transgenerational carrier of environmental information. To test whether transgenerational inheritance of environmental information occurs in mammals, we carried out an expression profiling screen for genes in mice that responded to paternal diet. Offspring of males fed a low-protein diet exhibited elevated hepatic expression of many genes involved in lipid and cholesterol biosynthesis and decreased levels of cholesterol esters, relative to the offspring of males fed a control diet. Epigenomic profiling of offspring livers revealed numerous modest (similar to 20%) changes in cytosine methylation depending on paternal diet, including reproducible changes in methylation over a likely enhancer for the key lipid regulator Ppara. These results, in conjunction with recent human epidemiological data, indicate that parental diet can affect cholesterol and lipid metabolism in offspring and define a model system to study environmental reprogramming of the heritable epigenome.", 
"Induction of cell death and inhibition of cell survival are the main principles of cancer therapy. Resistance to chemotherapeutic agents is a major problem in oncology, which limits the effectiveness of anticancer drugs. A variety of factors contribute to drug resistance, including host factors, specific genetic or epigenetic alterations in the cancer cells and so on. Although various mechanisms by which cancer cells become resistant to anticancer drugs in the microenvironment have been well elucidated, how to circumvent this resistance to improve anticancer efficacy remains to be defined. Autophagy, an important homeostatic cellular recycling mechanism, is now emerging as a crucial player in response to metabolic and therapeutic stresses, which attempts to maintain/restore metabolic homeostasis through the catabolic lysis of excessive or unnecessary proteins and injured or aged organelles. Recently, several studies have shown that autophagy constitutes a potential target for cancer therapy and the induction of autophagy in response to therapeutics can be viewed as having a prodeath or a prosurvival role, which contributes to the anticancer efficacy of these drugs as well as drug resistance. Thus, understanding the novel function of autophagy may allow us to develop a promising therapeutic strategy to enhance the effects of chemotherapy and improve clinical outcomes in the treatment of cancer patients.", 
"DNA methylation is a critical epigenetic regulator in mammalian development. Here, we present a whole-genome comparative view of DNA methylation using bisulfite sequencing of three cultured cell types representing progressive stages of differentiation: human embryonic stem cells (hESCs), a fibroblastic differentiated derivative of the hESCs, and neonatal fibroblasts. As a reference, we compared our maps with a methylome map of a fully differentiated adult cell type, mature peripheral blood mononuclear cells (monocytes). We observed many notable common and cell-type-specific features among all cell types. Promoter hypomethylation (both CG and CA) and higher levels of gene body methylation were positively correlated with transcription in all cell types. Exons were more highly methylated than introns, and sharp transitions of methylation occurred at exon-intron boundaries, suggesting a role for differential methylation in transcript splicing. Developmental stage was reflected in both the level of global methylation and extent of non-CpG methylation, with hESC highest, fibroblasts intermediate, and monocytes lowest. Differentiation-associated differential methylation profiles were observed for developmentally regulated genes, including the HOX clusters, other homeobox transcription factors, and pluripotence-associated genes such as POU5F1, TCF3, and KLF4. Our results highlight the value of high-resolution methylation maps, in conjunction with other systems-level analyses, for investigation of previously undetectable developmental regulatory mechanisms.", 
"RNA-directed DNA methylation (RdDM) is the major small RNA-mediated epigenetic pathway in plants. RdDM requires a specialized transcriptional machinery that comprises two plant-specific RNA polymerases - Pot IV and Pot V - and a growing number of accessory proteins, the functions of which in the RdDM mechanism are only partially understood. Recent work has revealed variations in the canonical RdDM pathway and identified factors that recruit Pot IV and Pol V to specific target sequences. RdDM, which transcriptionally represses a subset of transposons and genes, is implicated in pathogen defence, stress responses and reproduction, as well as in interallelic and intercellular communication.", 
"Tumor heterogeneity is a major barrier to effective cancer diagnosis and treatment. We recently identified cancer-specific differentially DNA-methylated regions (cDMRs) in colon cancer, which also distinguish normal tissue types from each other, suggesting that these cDMRs might be generalized across cancer types. Here we show stochastic methylation variation of the same cDMRs, distinguishing cancer from normal tissue, in colon, lung, breast, thyroid and Wilms' tumors, with intermediate variation in adenomas. Whole-genome bisulfite sequencing shows these variable cDMRs are related to loss of sharply delimited methylation boundaries at CpG islands. Furthermore, we find hypomethylation of discrete blocks encompassing half the genome, with extreme gene expression variability. Genes associated with the cDMRs and large blocks are involved in mitosis and matrix remodeling, respectively. We suggest a model for cancer involving loss of epigenetic stability of well-defined genomic domains that underlies increased methylation variability in cancer that may contribute to tumor heterogeneity.", 
"Emergence of form and function during embryogenesis arises in large part through cell-type- and cell-state-specific variation in gene expression patterns, mediated by specialized cis-regulatory elements called enhancers. Recent large-scale epigenomic mapping revealed unexpected complexity and dynamics of enhancer utilization patterns, with 400,000 putative human enhancers annotated by the ENCODE project alone. These large-scale efforts were largely enabled through the understanding that enhancers share certain stereotypical chromatin features. However, an important question still lingers: what is the functional significance of enhancer chromatin modification? Here we give an overview of enhancer-associated modifications of histones and DNA and discuss enzymatic activities involved in their dynamic deposition and removal. We describe potential downstream effectors of these marks and propose models for exploring functions of chromatin modification in regulating enhancer activity during development.", 
"The generation of properly functioning gametes in vitro requires reconstitution of the multistepped pathway of germ cell development. We demonstrate here the generation of primordial germ cell-like cells (PGCLCs) in mice with robust capacity for spermatogenesis. PGCLCs were generated from embryonic stem cells (ESCs) and induced pluripotent stem cells (iPSCs) through epiblast-like cells (EpiLCs), a cellular state highly similar to pregastrulating epiblasts but distinct from epiblast stem cells (EpiSCs). Reflecting epiblast development, EpiLC induction from ESCs/iPSCs is a progressive process, and EpiLCs highly competent for the PGC fate are a transient entity. The global transcription profiles, epigenetic reprogramming, and cellular dynamics during PGCLC induction from EpiLCs meticulously capture those associated with PGC specification from the epiblasts. Furthermore, we identify Integrin-beta 3 and SSEA1 as markers that allow the isolation of PGCLCs with spermatogenic capacity from tumorigenic undifferentiated cells. Our findings provide a paradigm for the first step of in vitro gametogenesis.", 
"Observational epidemiological studies are prone to confounding, reverse causation and various biases and have generated findings that have proved to be unreliable indicators of the causal effects of modifiable exposures on disease outcomes. Mendelian randomization (MR) is a method that utilizes genetic variants that are robustly associated with such modifiable exposures to generate more reliable evidence regarding which interventions should produce health benefits. The approach is being widely applied, and various ways to strengthen inference given the known potential limitations of MR are now available. Developments of MR, including two-sample MR, bidirectional MR, network MR, two-step MR, factorial MR and multiphenotype MR, are outlined in this review. The integration of genetic information into population-based epidemiological studies presents translational opportunities, which capitalize on the investment in genomic discovery research.", 
"Successful reprogramming of differentiated human somatic cells into a pluripotent state would allow creation of patient- and disease-specific stem cells. We previously reported generation of induced pluripotent stem (iPS) cells, capable of germline transmission, from mouse somatic cells by transduction of four defined transcription factors. Here, we demonstrate the generation of iPS cells from adult human dermal fibroblasts with the same four factors: Oct3/4, Sox2, Klf4, and c-Myc. Human iPS cells were similar to human embryonic stem (ES) cells in morphology, proliferation, surface antigens, gene expression, epigenetic status of pluripotent cell-specific genes, and telomerase activity. Furthermore, these cells could differentiate into cell types of the three germ layers in vitro and in teratomas. These findings demonstrate that iPS cells can be generated from adult human fibroblasts.", 
"The most highly conserved noncoding elements (HCNEs) in mammalian genomes cluster within regions enriched for genes encoding developmentally important transcription factors (TFs). This suggests that HCNE-rich regions may contain key regulatory controls involved in development. We explored this by examining histone methylation in mouse embryonic stem (ES) cells across 56 large HCNE-rich loci. We identified a specific modification pattern, termed bivalent domains, consisting of large regions of H3 lysine 27 methylation harboring smaller regions of H3 lysine 4 methylation. Bivalent domains tend to coincide with TF genes expressed at low levels. We propose that bivalent domains silence developmental genes in ES cells while keeping them poised for activation. We also found striking correspondences between genome sequence and histone methylation in ES cells, which become notably weaker in differentiated cells. These results highlight the importance of DNA sequence in defining the initial epigenetic landscape and suggest a novel chromatin-based mechanism for maintaining pluripotency.", 
"RNA is not only a messenger operating between DNA and protein. Transcription of essentially the entire eukaryotic genome generates a myriad of non-protein-coding RNA species that show complex overlapping patterns of expression and regulation. Although long noncoding RNAs (lncRNAs) are among the least well-understood of these transcript species, they cannot all be dismissed as merely transcriptional noise.'' Here, we review the evolution of lncRNAs and their roles in transcriptional regulation, epigenetic gene regulation, and disease.", 
"Noncoding RNAs (ncRNA) participate in epigenetic regulation but are poorly understood. Here we characterize the transcriptional landscape of the four human HOX loci at five base pair resolution in 11 anatomic sites and identify 231 HOX ncRNAs that extend known transcribed regions by more than 30 kilobases. HOX ncRNAs are spatially expressed along developmental axes and possess unique sequence motifs, and their expression demarcates broad chromosomal domains of differential histone methylation and RNA polymerase accessibility. We identified a 2.2 kilobase ncRNA residing in the HOXC locus, termed HOTAIR, which represses transcription in trans across 40 kilobases of the HOXD locus. HOTAIR interacts with Polycomb Repressive Complex 2 (PRC2) and is required for PRC2 occupancy and histone H3 lysine-27 trimethylation of HOXD locus. Thus, transcription of ncRNA may demarcate chromosomal domains of gene silencing at a distance; these results have broad implications for gene regulation in development and disease states.", 
"Over the past several years it has become clear that alterations in the expression of microRNA (miRNA) genes contribute to the pathogenesis of most - if not all - human malignancies. These alterations can be caused by various mechanisms, including deletions, amplifications or mutations involving miRNA loci, epigenetic silencing or the dysregulation of transcription factors that target specific miRNAs. Because malignant cells show dependence on the dysregulated expression of miRNA genes, which in turn control or are controlled by the dysregulation of multiple protein-coding oncogenes or tumour suppressor genes, these small RNAs provide important opportunities for the development of future miRNA-based therapies.", 
"The genomes of many animals, plants and fungi are tagged by methylation of DNA cytosine. To understand the biological significance of this epigenetic mark it is essential to know where in the genome it is located. New techniques are making it easier to map DNA methylation patterns on a large scale and the results have already provided surprises. In particular, the conventional view that DNA methylation functions predominantly to irreversibly silence transcription is being challenged. Not only is promoter methylation often highly dynamic during development, but many organisms also seem to target DNA methylation specifically to the bodies of active genes.", 
"DNA methylation is a crucial epigenetic modification of the genome that is involved in regulating many cellular processes. These include embryonic development, transcription, chromatin structure, X chromosome inactivation, genomic imprinting and chromosome stability. Consistent with these important roles, a growing number of human diseases have been found to be associated with aberrant DNA methylation. The study of these diseases has provided new and fundamental insights into the roles that DNA methylation and other epigenetic modifications have in development and normal cellular homeostasis.", 
"Methylation of DNA at position five of the cytosine ring occurs at most CpG dinucleotides in the mammalian genome and is essential for embryonic viability. With several of the key proteins now known, it has become possible to approach the biological significance of this epigenetic system through both biochemistry and genetics. As a result, advances have been made in our understanding of the mechanisms by which DNA methylation is targeted to specific regions of the genome and interpreted by methyl-CpG-binding proteins. Recent studies have illuminated the role of DNA methylation in controlling gene expression and have strengthened its links with histone modification and chromatin remodelling.", 
"Epidemiological evidence increasingly suggests that environmental exposures early in development have a role in susceptibility to disease in later life. In addition, some of these environmental effects seem to be passed on through subsequent generations. Epigenetic modifications provide a plausible link between the environment and alterations in gene expression that might lead to disease phenotypes. An increasing body of evidence from animal studies supports the role of environmental epigenetics in disease susceptibility. Furthermore, recent studies have demonstrated for the first time that heritable environmentally induced epigenetic modifications underlie reversible transgenerational alterations in phenotype. Methods are now becoming available to investigate the relevance of these phenomena to human disease.", 
"Deciphering the multiple layers of epigenetic regulation that control transcription is critical to understanding how plants develop and respond to their environment. Using sequencing-by-synthesis technology we directly sequenced the cytosine methylome (methylC-seq), transcriptome (mRNA-seq), and small RNA transcriptome (smRNA-seq) to generate highly integrated epigenome maps for wild-type Arabidopsis thaliana and mutants defective in DNA methyltransferase or demethylase activity. At singlebase resolution we discovered extensive, previously undetected DNA methylation, identified the context and level of methylation at each site, and observed local sequence effects upon methylation state. Deep sequencing of smRNAs revealed a direct relationship between the location of smRNAs and DNA methylation, perturbation of smRNA biogenesis upon loss of CpG DNA methylation, and a tendency for smRNAs to direct strand-specific DNA methylation in regions of RNA-DNA homology. Finally, strandspecific mRNA-seq revealed altered transcript abundance of hundreds of genes, transposons, and unannotated intergenic transcripts upon modification of the DNA methylation state.", 
"Plants and animals use small RNAs (microRNAs [miRNAs] and siRNAs) as guides for posttranscriptional and epigenetic regulation. In plants, miRNAs and trans-acting (ta) siRNAs form through distinct biogenesis pathways, although they both interact with target transcripts and guide cleavage. An integrated approach to identify targets of Arabidopsis thaliana miRNAs and ta-siRNAs revealed several new classes of small RNA-regulated genes, including conventional genes such as Argonaute2 and an E2-ubiquitin conjugating enzyme. Surprisingly, five tasiRNA-generating transcripts were identified as targets of miR173 or miR390. Rather than functioning as negative regulators, miR173- and miR390-guided cleavage was shown to set the 21-nucleotide phase for ta-siRNA precursor processing. These data support a model in which miRNA-guided formation of a 5' or 3' terminus within pre-ta-siRNA transcripts, followed by RDR6-dependent formation of dsRNA and Dicer-like processing, yields phased ta-siRNAs that negatively regulate other genes.", 
"For the past 25 years, it has been known that alterations in DNA methylation (DNAm) occur in cancer, including hypomethylation of oncogenes and hypermethylation of tumor suppressor genes. However, most studies of cancer methylation have assumed that functionally important DNAm will occur in promoters, and that most DNAm changes in cancer occur in CpG islands. Here we show that most methylation alterations in colon cancer occur not in promoters, and also not in CpG islands, but in sequences up to 2 kb distant, which we term 'CpG island shores'. CpG island shore methylation was strongly related to gene expression, and it was highly conserved in mouse, discriminating tissue types regardless of species of origin. There was a notable overlap (45-65%) of the locations of colon cancer-related methylation changes with those that distinguished normal tissues, with hypermethylation enriched closer to the associated CpG islands, and hypomethylation enriched further from the associated CpG island and resembling that of noncolon normal tissues. Thus, methylation changes in cancer are at sites that vary normally in tissue differentiation, consistent with the epigenetic progenitor model of cancer, which proposes that epigenetic alterations affecting tissue-specific differentiation are the predominant mechanism by which epigenetic changes cause cancer.", 
"An altered pattern of epigenetic modifications is central to many common human diseases, including cancer. Many studies have explored the mosaic patterns of DNA methylation and histone modification in cancer cells on a gene-by-gene basis; among their results has been the seminal finding of transcriptional silencing of tumour-suppressor genes by CpG-island-promoter hypermethylation. However, recent technological advances are now allowing cancer epigenetics to be studied genome-wide-an approach that has already begun to provide both biological insight and new avenues for translational research. It is time to 'upgrade' cancer epigenetics research and put together an ambitious plan to tackle the many unanswered questions in this field using epigenomics approaches.", 
"The term non-coding RNA (ncRNA) is commonly employed for RNA that does not encode a protein, but this does not mean that such RNAs do not contain information nor have function. Although it has been generally assumed that most genetic information is transacted by proteins, recent evidence suggests that the majority of the genomes of mammals and other complex organisms is in fact transcribed into ncRNAs, many of which are alternatively spliced and/or processed into smaller products. These ncRNAs include microRNAs and snoRNAs (many if not most of which remain to be identified), as well as likely other classes of yet-to-be-discovered small regulatory RNAs, and tens of thousands of longer transcripts (including complex patterns of interlacing and overlapping sense and antisense transcripts), most of whose functions are unknown. These RNAs (including those derived from introns) appear to comprise a hidden layer of internal signals that control various levels of gene expression in physiology and development, including chromatin architecture/epigenetic memory, transcription, RNA splicing, editing, translation and turnover. RNA regulatory networks may determine most of our complex characteristics, play a significant role in disease and constitute an unexplored world of genetic variation both within and between species.", 
" ", 
"X-chromosome inactivation is the mammalian dosage compensation mechanism by which transcription of X-linked genes is equalized between females and males. In an N-ethyl-N-nitrosourea (ENU) mutagenesis screen on mice for modifiers of epigenetic reprogramming, we identified the MommeD1 (modifier of murine metastable epialleles) mutation as a semidominant suppressor of variegation. MommeD1 shows homozygous female-specific mid-gestation lethality and hypomethylation of the X-linked gene Hprt1, suggestive of a defect in X inactivation(1). Here we report that the causative point mutation lies in a previously uncharacterized gene, Smchd1 (structural maintenance of chromosomes hinge domain containing 1). We find that SmcHD1 is not required for correct Xist expression, but localizes to the inactive X and has a role in the maintenance of X inactivation and the hypermethylation of CpG islands associated with the inactive X. This finding links a group of proteins normally associated with structural aspects of chromosome biology with epigenetic gene silencing.", 
" ", 
"The packaging of chromosomal DNA by nucleosomes condenses and organizes the genome, but occludes many regulatory DNA elements. However, this constraint also allows nucleosomes and other chromatin components to actively participate in the regulation of transcription, chromosome segregation, DNA replication, and DNA repair. To enable dynamic access to packaged DNA and to tailor nucleosome composition in chromosomal regions, cells have evolved a set of specialized chromatin remodeling complexes (remodelers). Remodelers use the energy of ATP hydrolysis to move, destabilize, eject, or restructure nucleosomes. Here, we address many aspects of remodeler biology: their targeting, mechanism, regulation, shared and unique properties, and specialization for particular biological processes. We also address roles for remodelers in development, cancer, and human syndromes.", 
"Ectopic expression of the four transcription factors Oct4, Sox2, c-Myc, and Klf4 is sufficient to confer a pluripotent state upon the fibroblast genome, generating induced pluripotent stem (PS) cells. It remains unknown if nuclear reprogramming induced by these four factors globally resets epigenetic differences between differentiated and pluripotent cells. Here, using novel selection approaches, we have generated PS cells from fibroblasts to characterize their epigenetic state. Female PS cells showed reactivation of a somatically silenced X chromosome and underwent random X inactivation upon differentiation. Genome-wide analysis of two key histone modifications indicated that PS cells are highly similar to ES cells. Consistent with these observations, PS cells gave rise to viable high-degree chimeras with contribution to the germline. These data show that transcription factor-induced reprogramming leads to the global reversion of the somatic epigenome into an ES-like state. Our results provide a paradigm for studying the epigenetic modifications that accompany nuclear reprogramming and suggest that abnormal epigenetic reprogramming does not pose a problem for the potential therapeutic applications of PS cells.", 
"Cancer is widely perceived as a heterogeneous group of disorders with markedly different biological properties, which are caused by a series of clonally selected genetic changes in key tumour-suppressor genes and oncogenes. However, recent data suggest that cancer has a fundamentally common basis that is grounded in a polyclonal epigenetic disruption of stem/progenitor cells, mediated by 'tumour-progenitor genes'. Furthermore, tumour cell heterogeneity is due in part to epigenetic variation in progenitor cells, and epigenetic plasticity together with genetic lesions drives tumour progression. This crucial early role for epigenetic alterations in cancer is in addition to epigenetic alterations that can substitute for genetic variation later in tumour progression. Therefore, non-neoplastic but epigenetically disrupted stem/progenitor cells might be a crucial target for cancer risk assessment and chemoprevention.", 
"Polyploids - organisms that have multiple sets of chromosomes - are common in certain plant and animal taxa, and can be surprisingly stable. The evidence that has emerged from genome analyses also indicates that many other eukaryotic genomes have a polyploid ancestry, suggesting that both humans and most other eukaryotes have either benefited from or endured polyploidy. Studies of polyploids soon after their formation have revealed genetic and epigenetic interactions between redundant genes. These interactions can be related to the phenotypes and evolutionary fates of polyploids. Here, I consider the advantages and challenges of polyploidy, and its evolutionary potential.", 
"CpG island hypermethylation and global genomic hypomethylation are common epigenetic features of cancer cells. Less attention has been focused on histone modifications in cancer cells. We characterized post-translational modifications to histone H4 in a comprehensive panel of normal tissues, cancer cell lines and primary tumors. Using immunodetection, high-performance capillary electrophoresis and mass spectrometry, we found that cancer cells had a loss of monoacetylated and trimethylated forms of histone H4. These changes appeared early and accumulated during the tumorigenic process, as we showed in a mouse model of multistage skin carcinogenesis. The losses occurred predominantly at the acetylated Lys16 and trimethylated Lys20 residues of histone H4 and were associated with the hypomethylation of DNA repetitive sequences, a well-known characteristic of cancer cells. Our data suggest that the global loss of monoacetylation and trimethylation of histone H4 is a common hallmark of human tumor cells.", 
"Cytosine methylation is required for mammalian development and is often perturbed in human cancer. To determine how this epigenetic modification is distributed in the genomes of primary and transformed cells, we used an immunocapturing approach followed by DNA microarray analysis to generate methylation profiles of all human chromosomes at 80-kb resolution and for a large set of CpG islands. In primary cells we identified broad genomic regions of differential methylation with higher levels in gene-rich neighborhoods. Female and male cells had indistinguishable profiles for autosomes but differences on the X chromosome. The inactive X chromosome ( Xi) was hypermethylated at only a subset of gene-rich regions and, unexpectedly, overall hypomethylated relative to its active counterpart. The chromosomal methylation profile of transformed cells was similar to that of primary cells. Nevertheless, we detected large genomic segments with hypomethylation in the transformed cell residing in gene-poor areas. Furthermore, analysis of 6,000 CpG islands showed that only a small set of promoters was methylated differentially, suggesting that aberrant methylation of CpG island promoters in malignancy might be less frequent than previously hypothesized.", 
"Inheritance requires genome duplication, reproduction of chromatin and its epigenetic information, mechanisms to ensure genome integrity, and faithful transmission of the information to progeny. Proliferating cell nuclear antigen (PCNA)-a cofactor of DNA polymerases that encircles DNA-orchestrates several of these functions by recruiting crucial players to the replication fork. Remarkably, many factors that are involved in replication-linked processes interact with a particular face of PCNA and through the same interaction domain, indicating that these interactions do not occur simultaneously during replication. Switching of PCNA partners may be triggered by affinity-driven competition, phosphorylation, proteolysis, and modification of PCNA by ubiquitin and SUMO.", 
"Cytosine methylation is important for transposon silencing and epigenetic regulation of endogenous genes, although the extent to which this DNA modification functions to regulate the genome is still unknown. Here we report the first comprehensive DNA methylation map of an entire genome, at 35 base pair resolution, using the flowering plant Arabidopsis thaliana as a model. We find that pericentromeric heterochromatin, repetitive sequences, and regions producing small interfering RNAs are heavily methylated. Unexpectedly, over one-third of expressed genes contain methylation within transcribed regions, whereas only similar to 5% of genes show methylation within promoter regions. Interestingly, genes methylated in transcribed regions are highly expressed and constitutively active, whereas promoter-methylated genes show a greater degree of tissue-specific expression. Whole-genome tiling-array transcriptional profiling of DNA methyltransferase null mutants identified hundreds of genes and intergenic noncoding RNAs with altered expression levels, many of which may be epigenetically controlled by DNA methylation.", 
"Overlapping epigenetic mechanisms have evolved in eukaryotic cells to silence the expression and mobility of transposable elements (TEs). Owing to their ability to recruit the silencing machinery, TEs have served as building blocks for epigenetic phenomena, both at the level of single genes and across larger chromosomal regions. Important progress has been made recently in understanding these silencing mechanisms. In addition, new insights have been gained into how this silencing has been co-opted to serve essential functions in 'host' cells, highlighting the importance of TEs in the epigenetic regulation of the genome.", 
"Cellular senescence, a state of irreversible growth arrest, can be triggered by multiple mechanisms including telomere shortening, the epigenetic derepression of the INK4a/ARF locus, and DNA damage. Together these mechanisms limit excessive or aberrant cellular proliferation, and so the state of senescence protects against the development of cancer. Recent evidence suggests that cellular senescence also may be involved in aging.", 
"Histones comprise the major protein component of chromatin, the scaffold in which the eukaryotic genome is packaged, and are subject to many types of post-translational modifications (PTMs), especially on their flexible tails. These modifications may constitute a 'histone code' and could be used to manage epigenetic information that helps extend the genetic message beyond DNA sequences. This proposed code, read in part by histone PTM-binding 'effector' modules and their associated complexes, is predicted to define unique functional states of chromatin and/or regulate various chromatin-templated processes. A wealth of structural and functional data show how chromatin effector modules target their cognate covalent histone modifications. Here we summarize key features in molecular recognition of histone PTMs by a diverse family of 'reader pockets', highlighting specific readout mechanisms for individual marks, common themes and insights into the downstream functional consequences of the interactions. Changes in these interactions may have far-reaching implications for human biology and disease, notably cancer.", 
"Chromatin immunoprecipitation followed by sequencing (ChIP-seq) is a technique for genome-wide profiling of DNA-binding proteins, histone modifications or nucleosomes. Owing to the tremendous progress in next-generation sequencing technology, ChIP-seq offers higher resolution, less noise and greater coverage than its array-based predecessor ChIP-chip. With the decreasing cost of sequencing, ChIP-seq has become an indispensable tool for studying gene regulation and epigenetic mechanisms. In this Review, I describe the benefits and challenges in harnessing this technique with an emphasis on issues related to experimental design and data analysis. ChIP-seq experiments generate large quantities of data, and effective computational analysis will be crucial for uncovering biological mechanisms.", 
"Inflammation is linked clinically and epidemiologically to cancer, and NF-kappa B appears to play a causative role, but the mechanisms are poorly understood. We show that transient activation of Src oncoprotein can mediate an epigenetic switch from immortalized breast cells to a stably transformed line that forms self-renewing mammospheres that contain cancer stem cells. Src activation triggers an inflammatory response mediated by NF-kappa B that directly activates Lin28 transcription and rapidly reduces let-7 microRNA levels. Let-7 directly inhibits IL6 expression, resulting in higher levels of IL6 than achieved by NF-kappa B activation. IL6-mediated activation of the STAT3 transcription factor is necessary for transformation, and IL6 activates NF-kappa B, thereby completing a positive feedback loop. This regulatory circuit operates in other cancer cells lines, and its transcriptional signature is found in human cancer tissues. Thus, inflammation activates a positive feedback loop that maintains the epigenetic transformed state for many generations in the absence of the inducing signal.", 
"Expression profiling of T24 cells revealed that 17 out of 313 human miRNAs were upregulated more than 3-fold by simultaneous treatment with the chromatin-modifying drugs 5-aza-2'-deoxycytidine and 4-phenylbutyric acid. One of these, miR-127, is embedded in a CpG island and is highly induced from its own promoter after treatment. miR-127 is usually expressed as part of a miRNA cluster in normal cells but not in cancer cells, suggesting that it is subject to epigenetic silencing. In addition, the proto-oncogene BCL6, a potential target of miR-127, was translationally downregulated after treatment. These results suggest that DNA demethylation and histone deacetylase inhibition can activate expression of miRNAs that may act as tumor suppressors.", 
"Polycomb proteins form chromatin-modifying complexes that implement transcriptional silencing in higher eukaryotes. Hundreds of genes are silenced by Polycomb proteins, including dozens of genes that encode crucial developmental regulators in organisms ranging from plants to humans. Two main families of complexes, called Polycomb repressive complex 1 (PRC1) and PRC2, are targeted to repressed regions. Recent studies have advanced our understanding of these complexes, including their potential mechanisms of gene silencing, the roles of chromatin modifications, their means of delivery to target genes and the functional distinctions among variant complexes. Emerging concepts include the existence of a Polycomb barrier to transcription elongation and the involvement of non-coding RNAs in the targeting of Polycomb complexes. These findings have an impact on the epigenetic programming of gene expression in many biological systems.", 
"A recent meeting (December 2008) regarding chromatin-based epigenetics was hosted by the Banbury Conference Center and Cold Spring Harbor Laboratory. The intent was to discuss aspects of epigenetic control of genomic function, and to arrive at a consensus definition of epigenetics'' to be considered by the broader community. It was evident that multiple mechanistic steps lead to the stable heritance of the epigenetic phenotype. Below we provide our view and interpretation of the proceedings at the meeting.", 
"Epigenetic genome modifications are thought to be important for specifying the lineage and developmental stage of cells within a multicellular organism. Here, we show that the epigenetic profile of pluripotent embryonic stem cells (ES) is distinct from that of embryonic carcinoma cells, haematopoietic stem cells (HSC) and their differentiated progeny. Silent, lineage-specific genes replicated earlier in pluripotent cells than in tissue-specific stem cells or differentiated cells and had unexpectedly high levels of acetylated H3K9 and methylated H3K4. Unusually, in ES cells these markers of open chromatin were also combined with H3K27 trimethylation at some non-expressed genes. Thus, pluripotency of ES cells is characterized by a specific epigenetic profile where lineage-specific genes may be accessible but, if so, carry repressive H3K27 trimethylation modifications. H3K27 methylation is functionally important for preventing expression of these genes in ES cells as premature expression occurs in embryonic ectoderm development (Eed)-deficient ES cells. Our data suggest that lineage-specific genes are primed for expression in ES cells but are held in check by opposing chromatin modifications.", 
"DNA methylation is the most stable type of epigenetic modification modulating the transcriptional plasticity of mammalian genomes. Using bisulfite DNA sequencing, we report high-resolution methylation profiles of human chromosomes 6, 20 and 22, providing a resource of about 1.9 million CpG methylation values derived from 12 different tissues. Analysis of six annotation categories showed that evolutionarily conserved regions are the predominant sites for differential DNA methylation and that a core region surrounding the transcriptional start site is an informative surrogate for promoter methylation. We find that 17% of the 873 analyzed genes are differentially methylated in their 5 ' UTRs and that about one-third of the differentially methylated 5 ' UTRs are inversely correlated with transcription. Despite the fact that our study controlled for factors reported to affect DNA methylation such as sex and age, we did not find any significant attributable effects. Our data suggest DNA methylation to be ontogenetically more stable than previously thought.", 
"The Polycomb group (PcG) proteins form chromatin-modifying complexes that are essential for embryonic development and stem cell renewal and are commonly deregulated in cancer. Here, we identify their target genes using genome-wide location analysis in human embryonic fibroblasts. We find that Polycomb-Repressive Complex I (PRC1), PRC2, and tri-methylated histone H3K27 co-occupy > 1000 silenced genes with a strong functional bias for embryonic development and cell fate decisions. We functionally identify 40 genes derepressed in human embryonic fibroblasts depleted of the PRC2 components (EZH2, EED, SUZ12) and the PRC1 component, BMI-1. Interestingly, several markers of osteogenesis, adipogenesis, and chrondrogenesis are among these genes, consistent with the mesenchymal origin of fibroblasts. Using a neuronal model of differentiation, we delineate two different mechanisms for regulating PcG target genes. For genes activated during differentiation, PcGs are displaced. However, for genes repressed during differentiation, we paradoxically find that they are already bound by the PcGs in nondifferentiated cells despite being actively transcribed. Our results are consistent with the hypothesis that PcGs are part of a preprogrammed memory system established during embryogenesis marking certain key genes for repressive signals during subsequent developmental and differentiation processes.", 
"Epigenetic marking systems confer stability of gene expression during mammalian development. Genome-wide epigenetic reprogramming occurs at stages when developmental potency of cells changes. At fertilization, the paternal genome exchanges protamines for histones, undergoes DNA demethylation, and acquires histone modifications, whereas the maternal genome appears epigenetically more static. During preimplantation development, there is passive DNA demethylation and further reorganization of histone modifications. In blastocysts, embryonic and extraembryonic lineages first show different epigenetic marks. This epigenetic reprogramming is likely to be needed for totipotency, correct initiation of embryonic gene expression, and early lineage development in the embryo. Comparative work demonstrates reprogramming in all mammalian species analysed, but the extent and timing varies, consistent with notable differences between species during preimplantation development. Parental imprinting marks originate in sperm and oocytes and are generally protected from this genome-wide reprogramming. Early primordial germ cells possess imprinting marks similar to those of somatic cells. However, rapid DNA demethylation after midgestation erases these parental imprints, in preparation for sex-specific de novo methylation during gametogenesis. Aberrant reprogramming of somatic epigenetic marks after somatic cell nuclear transfer leads to epigenetic defects in cloned embryos and stem cells. Links between epigenetic marking systems appear to be developmentally regulated contributing to plasticity. A number of activities that confer epigenetic marks are firmly established, while for those that remove marks, particularly methylation, some interesting candidates have emerged recently which need thorough testing in vivo. A mechanistic understanding of reprogramming will be crucial for medical applications of stem cell technology.", 
"Background: In animal models, variations in early maternal care are associated with differences in hypothalamic-pituitary-adrenal (HPA) stress response in the offspring, mediated via changes in the epigenetic regulation of glucocorticoid receptor (GR) gene (Nr3c1) expression. Objective: To study this in humans, relationships between prenatal exposure to maternal mood and the methylation status of a CpG-rich region in the promoter and exon 1F of the human GR gene (NR3C1) in newborns and HPA stress reactivity at age three months were examined. Results: Prenatal exposure to increased third trimester maternal depressed/anxious mood was associated with increased methylation of NR3C1 at a predicted NGFI-A binding site. Increased NR3C1 methylation at this site was also associated with increased salivary cortisol stress responses at 3 months, controlling for prenatal SRI exposure, postnatal age and pre and postnatal maternal mood. Methods: The methylation status of a CpG-rich region of the NR3C1 gene, including exon 1F, in genomic DNA from cord blood mononuclear cells was quantified by bisulfite pyrosequencing in infants of depressed mothers treated with a serotonin reuptake inhibitor antidepressant (SRI) (n = 33), infants of depressed non treated mothers (n = 13) and infants of non depressed/non treated mothers (n = 36). To study the functional implications of the newborn methylation status of NR3C1 in newborns, HPA function was assessed at three months using salivary cortisol obtained before and following a non noxious stressor and at a late afternoon basal time. Conclusions: Methylation status of the human NR3C1 gene in newborns is sensitive to prenatal maternal mood and may offer a potential epigenetic process that links antenatal maternal mood and altered HPA stress reactivity during infancy.", 
"The formation of heterochromatin, which requires methylation of histone H3 at lysine 9 and the subsequent recruitment of chromodomain proteins such as heterochromatin protein HP1, serves as a model for the role of histone modifications and chromatin assembly in epigenetic control of the genome. Recent studies in Schizosaccharomyces pombe indicate that heterochromatin serves as a dynamic platform to recruit and spread a myriad of regulatory proteins across extended domains to control various chromosomal processes, including transcription, chromosome segregation and long-range chromatin interactions.", 
"Compelling evidence suggests that the transcription factor Foxp3 acts as a master switch governing the development and function of CD4(+) regulatory T cells (Tregs). However, whether transcriptional control of Foxp3 expression itself contributes to the development of a stable Treg lineage has thus far not been investigated. We here identified an evolutionarily conserved region within the foxp3 locus upstream of exon-1 possessing transcriptional activity. Bisulphite sequencing and chromatin immunoprecipitation revealed complete demethylation of CpG motifs as well as histone modifications within the conserved region in ex vivo isolated Foxp3(+)CD25(+)CD4(+) Tregs, but not in naive CD25(-) CD4(+) T cells. Partial DNA demethylation is already found within developing Foxp3(+) thymocytes; however, Tregs induced by TGF-beta in vitro display only incomplete demethylation despite high Foxp3 expression. In contrast to natural Tregs, these TGF-beta-induced Foxp3(+) Tregs lose both Foxp3 expression and suppressive activity upon restimulation in the absence of TGF-beta. Our data suggest that expression of Foxp3 must be stabilized by epigenetic modification to allow the development of a permanent suppressor cell lineage, a finding of significant importance for therapeutic applications involving induction or transfer of Tregs and for the understanding of long-term cell lineage decisions.", 
"Many genes associated with CpG islands undergo de novo methylation in cancer. Studies have suggested that the pattern of this modification may be partially determined by an instructive mechanism that recognizes specifically marked regions of the genome(1). Using chromatin immunoprecipitation analysis, here we show that genes methylated in cancer cells are specifically packaged with nucleosomes containing histone H3 trimethylated on Lys27. This chromatin mark is established on these unmethylated CpG island genes early in development and then maintained in differentiated cell types by the presence of an EZH2- containing Polycomb complex(2,3). In cancer cells(4), as opposed to normal cells, the presence of this complex brings about the recruitment of DNA methyl transferases, leading to de novo methylation. These results suggest that tumor-specific targeting of de novo methylation is pre-programmed by an established epigenetic system that normally has a role in marking embryonic genes for repression(2).", 
"Ageing results from complex genetically and epigenetically programmed processes that are elicited in part by noxious or stressful events that cause programmed cell death. Here, we report that administration of spermidine, a natural polyamine whose intracellular concentration declines during human ageing, markedly extended the lifespan of yeast, flies and worms, and human immune cells. In addition, spermidine administration potently inhibited oxidative stress in ageing mice. In ageing yeast, spermidine treatment triggered epigenetic deacetylation of histone H3 through inhibition of histone acetyltransferases (HAT), suppressing oxidative stress and necrosis. Conversely, depletion of endogenous polyamines led to hyperacetylation, generation of reactive oxygen species, early necrotic death and decreased lifespan. The altered acetylation status of the chromatin led to significant upregulation of various autophagy-related transcripts, triggering autophagy in yeast, flies, worms and human cells. Finally, we found that enhanced autophagy is crucial for polyamine-induced suppression of necrosis and enhanced longevity.", 
"Induced pluripotent stem (iPS) cells are derived by epigenetic reprogramming, but their DNA methylation patterns have not yet been analyzed on a genome-wide scale. Here, we find substantial hypermethylation and hypomethylation of cytosine-phosphate-guanine (CpG) island shores in nine human iPS cell lines as compared to their parental fibroblasts. The differentially methylated regions (DMRs) in the reprogrammed cells (denoted R-DMRs) were significantly enriched in tissue-specific (T-DMRs; 2.6-fold, P < 10(-4)) and cancer-specific DMRs (C-DMRs; 3.6-fold, P < 10(-4)). Notably, even though the iPS cells are derived from fibroblasts, their R-DMRs can distinguish between normal brain, liver and spleen cells and between colon cancer and normal colon cells. Thus, many DMRs are broadly involved in tissue differentiation, epigenetic reprogramming and cancer. We observed colocalization of hypomethylated R-DMRs with hypermethylated C-DMRs and bivalent chromatin marks, and colocalization of hypermethylated R-DMRs with hypomethylated C-DMRs and the absence of bivalent marks, suggesting two mechanisms for epigenetic reprogramming in iPS cells and cancer.", 
"Histone methylation has important roles in regulating gene expression and forms part of the epigenetic memory system that regulates cell fate and identity. Enzymes that directly remove methyl marks from histones have recently been identified, revealing a new level of plasticity within this epigenetic modification system. Here we analyse the evolutionary relationship between Jumonji C ( JmjC)-domain-containing proteins and discuss their cellular functions in relation to their potential enzymatic activities.", 
"Protein lysine deacetylases have a pivotal role in numerous biological processes and can be divided into the Rpd3/Hda1 and sirtuin families, each having members in diverse organisms including prokaryotes. In vertebrates, the Rpd3/Hda1 family contains 11 members, traditionally referred to as histone deacetylases (HDAC)1-11, which are further grouped into classes I, II and IV. Whereas most class I HDACs are subunits of multiprotein nuclear complexes that are crucial for transcriptional repression and epigenetic landscaping, class II members regulate cytoplasmic processes or function as signal transducers that shuttle between the cytoplasm and the nucleus. Little is known about class IV HDAC11, although its evolutionary conservation implies a fundamental role in various organisms.", 
"Recent investigations have implicated long antisense noncoding RNAs in the epigenetic regulation of chromosomal domains. Here we show that Kcnq1ot1 is an RNA polymerase II-encoded, 91 kb-long, moderately stable nuclear transcript and that its stability is important for bidirectional silencing of genes in the Kcnq1 domain. Kcnq1ot1 interacts with chromatin and with the H3K9- and H3K27-specific histone methyltransferases G9a and the PRC2 complex in a lineage-specific manner. This interaction correlates with the presence of extended regions of chromatin enriched with H3K9me3 and H3K27me3 in the Kcnq1 domain in placenta, whereas fetal liver lacks both chromatin interactions and heterochromatin structures. In addition, the Kcnq1 domain is more often found in contact with the nucleolar compartment in placenta than in liver. Taken together, our data describe a mechanism whereby Kcnq1ot1 establishes lineage-specific transcriptional silencing patterns through recruitment of chromatin remodeling complexes and maintenance of these patterns through subsequent cell divisions occurs via targeting the associated regions to the perinucleolar compartment.", 
"It is more evident now than ever that nucleosomes can transmit epigenetic information from one cell generation to the next. It has been demonstrated during the past decade that the post-translational modifications of histone proteins within the chromosome impact chromatin structure, gene transcription, and epigenetic information. Multiple modifications decorate each histone tail within the nucleosome, including some amino acids that can be modified in several different ways. Covalent modifications of histone tails known thus far include acetylation, phosphorylation, surnoylation, ubiquitination, and methylation. A large body of experimental evidence compiled during the past several years has demonstrated the impact of histone acetylation on transcriptional control. Although histone modification by methylation and ubiquitination was discovered long ago, it was only recently that functional roles for these modifications in transcriptional regulation began to surface. Highlighted in this review are the recent biochemical, molecular, cellular, and physiological functions of histone methylation and ubiquitination involved in the regulation of gene expression as determined by a combination of enzymological, structural, and genetic methodologies.", 
"Cells employ elaborate mechanisms to introduce structural and chemical variation into chromatin. Here, we focus on one such element of variation: methylation of lysine 4 in histone H3 (H3K4). We assess a growing body of literature, including treatment of how the mark is established, the patterns of methylation, and the functional consequences of this epigenetic signature. We discuss structural aspects of the H3K4 methyl recognition by the downstream effectors and propose a distinction between sequence-specific recruitment mechanisms and stabilization on chromatin through methyllysine recognition. Finally, we hypothesize how the unique properties of the polyvalent chromatin fiber and associated effectors may amplify small differences in methyl-lysine recognition, simultaneously allowing for a dynamic chromatin architecture.", 
"Embryonic stem cells rely on Polycomb group proteins to reversibly repress genes required for differentiation. We report that stem cell Polycomb group targets are up to 12-fold more likely to have cancer- specific promoter DNA hypermethylation than non-targets, supporting a stem cell origin of cancer in which reversible gene repression is replaced by permanent silencing, locking the cell into a perpetual state of self-renewal and thereby predisposing to subsequent malignant transformation.", 
"Adult cancers may derive from stem or early progenitor cells(1,2). Epigenetic modulation of gene expression is essential for normal function of these early cells but is highly abnormal in cancers, which often show aberrant promoter CpG island hypermethylation and transcriptional silencing of tumor suppressor genes and pro-differentiation factors(3-5). We find that for such genes, both normal and malignant embryonic cells generally lack the hypermethylation of DNA found in adult cancers. In embryonic stem cells, these genes are held in a 'transcription- ready' state mediated by a 'bivalent' promoter chromatin pattern consisting of the repressive mark, histone H3 methylated at Lys27 ( H3K27) by Polycomb group proteins, plus the active mark, methylated H3K4. However, embryonic carcinoma cells add two key repressive marks, dimethylated H3K9 and trimethylated H3K9, both associated with DNA hypermethylation in adult cancers(6-8). We hypothesize that cell chromatin patterns and transient silencing of these important regulatory genes in stem or progenitor cells may leave these genes vulnerable to aberrant DNA hypermethylation and heritable gene silencing during tumor initiation and progression.", 
" ", 
"Patterns of DNA methylation and chromatin structure are profoundly altered in neoplasia and include genome-wide losses of, and regional gains in, DNA methylation. The recent explosion in our knowledge of how chromatin organization modulates gene transcription has further highlighted the importance of epigenetic mechanisms in the initiation and progression of human cancer. These epigenetic changes - in particular, aberrant promoter hypermethylation that is associated with inappropriate gene silencing - affect virtually every step in tumour progression. In this review, we discuss these epigenetic events and the molecular alterations that might cause them and/or underlie altered gene expression in cancer.", 
"Cells of a multicellular organism are genetically homogeneous but structurally and functionally heterogeneous owing to the differential expression of genes. Many of these differences in gene expression arise during development and are subsequently retained through mitosis. Stable alterations of this kind are said to be 'epigenetic', because they are heritable in the short term but do not involve mutations of the DNA itself. Research over the past few years has focused on two molecular mechanisms that mediate epigenetic phenomena: DNA methylation and histone modifications. Here, we review advances in the understanding of the mechanism and role of DNA methylation in biological processes. Epigenetic effects by means of DNA methylation have an important role in development but can also arise stochastically as animals age. Identification of proteins that mediate these effects has provided insight into this complex process and diseases that occur when it is perturbed. External influences on epigenetic processes are seen in the effects of diet on long-term diseases such as cancer. Thus, epigenetic mechanisms seem to allow an organism to respond to the environment through changes in gene expression. The extent to which environmental effects can provoke epigenetic responses represents an exciting area of future research.", 
"Epithelial and hematopoietic cells have a high turnover and their progenitor cells divide continuously, making them prime targets for genetic and epigenetic changes that lead to cell transformation and tumorigenesis. The consequent changes in cell behavior and responsiveness result not only from genetic alterations such as activation of oncogenes or inactivation of tumor suppressor genes, but also from altered production of, or responsiveness to, stimulatory or inhibitory growth and differentiation factors. Among these, transforming growth factor beta (TGF-beta) and its signaling effectors act as key determinants of carcinoma cell behavior. The autocrine and paracrine effects of TGF-beta on tumor cells and the tumor micro-environment exert both positive and negative influences on cancer development. Accordingly, the TGF-beta signaling pathway has been considered as both a tumor suppressor pathway and a promoter of tumor progression and invasion. Here we evaluate the role of TGF-beta in tumor development and attempt to reconcile the positive and negative effects of TGF-beta in carcinogenesis.", 
"Genomic imprinting affects several dozen mammalian genes and results in the expression of those genes from only one of the two parental chromosomes. This is brought about by epigenetic instructions - imprints - that are laid down in the parental germ cells. Imprinting is a particularly important genetic mechanism in mammals, and is thought to influence the transfer of nutrients to the fetus and the newborn from the mother. Consistent with this view is the fact that imprinted genes tend to affect growth in the womb and behaviour after birth. Aberrant imprinting disturbs development and is the cause of various disease syndromes. The study of imprinting also provides new insights into epigenetic gene modification during development.", 
"The developmental programme of embryogenesis is controlled by both genetic and epigenetic mechanisms. An emerging theme from recent studies is that the regulation of higher-order chromatin structures by DNA methylation and histone modification is crucial for genome reprogramming during early embryogenesis and gametogenesis, and for tissue-specific gene expression and global gene silencing. Disruptions to chromatin modification can lead to the dysregulation of developmental processes, such as X-chromosome inactivation and genomic imprinting, and to various diseases. Understanding the process of epigenetic reprogramming in development is important for studies of cloning and the clinical application of stem-cell therapy.", 
"Early nutrition affects adult metabolism in humans and other mammals, potentially via persistent alterations in DNA methylation. With viable yellow agouti (A(vy)) mice, which harbor a transposable element in the agouti gene, we tested the hypothesis that the metastable methylation status of specific transposable element insertion sites renders them epigenetically labile to early methyl donor nutrition. Our results show that dietary methyl supplementation of ala dams with extra folic acid, vitamin B-12, choline, and betaine alter the phenotype of their A(vy)/a offspring via increased CpG methylation at the A(vy) locus and that the epigenetic metastability which confers this lability is due to the A(vy) transposable element. These findings suggest that dietary supplementation, long presumed to be purely beneficial, may have unintended deleterious influences on the establishment of epigenetic gene regulation in humans.", 
"Recently, the concept that epigenetic, as well as genetic, events might be central to the evolution of human cancer is re-emerging. Cancers often exhibit an aberrant methylation of gene promoter regions that is associated with loss of gene function. This DNA change constitutes a heritable state, not mediated by altered nucleotide sequence, that appears to be tightly linked to the formation of transcriptionally repressive chromatin. This epigenetic process acts as an alternative to mutations to disrupt tumor-suppressor gene function and can predispose to genetic alterations through inactivating DNA-repair genes. Dissecting the molecular processes that mediate these methylation changes will enhance our understanding of chromatin modeling and gene regulation and might present novel possibilities for cancer therapy. Methylation changes constitute potentially sensitive molecular markers to define risk states, monitor prevention strategies, achieve early diagnosis, and track the prognosis of cancer.", 
"Polyploidy is a prominent process in plants and has been significant in the evolutionary history of vertebrates and other eukaryotes. In plants, interdisciplinary approaches combining phylogenetic and molecular genetic perspectives have enhanced our awareness of the myriad genetic interactions made possible by polyploidy. Here, processes and mechanisms of gene and genome evolution in polyploids are reviewed. Genes duplicated by polyploidy may retain their original or similar function, undergo diversification in protein function or regulation, or one copy may become silenced through mutational or epigenetic means. Duplicated genes also may interact through inter-locus recombination, gene conversion, or concerted evolution. Recent experiments have illuminated important processes in polyploids that operate above the organizational level of duplicated genes. These include inter-genomic chromosomal exchanges, saltational, non-Mendelian genomic evolution in nascent polyploids, inter-genomic invasion, and cytonuclear stabilization. Notwithstanding many recent insights, much remains to be learned about many aspects of polyploid evolution, including: the role of transposable elements in structural and regulatory gene evolution; processes and significance of epigenetic silencing; underlying controls of chromosome pairing; mechanisms and functional significance of rapid genome changes; cytonuclear accommodation; and coordination of regulatory factors contributed by two, sometimes divergent progenitor genomes. Continued application of molecular genetic approaches to questions of polyploid genome evolution holds promise for producing lasting insight into processes by which novel genotypes are generated and ultimately into how polyploidy facilitates evolution and adaptation.", 
"Histone H3 lysine 9 methylation has been proposed to provide a major switch for the functional organization of chromosomal subdomains. Here, we show that the murine Suv39h histone methyltransferases (HMTases) govern H3-K9 methylation at pericentric heterochromatin and induce a specialized histone methylation pattern that differs from the broad H3-K9 methylation present at other chromosomal regions. Suv39h-deficient mice display severely impaired viability and chromosomal instabilities that are associated with an increased tumor risk and perturbed chromosome interactions during male meiosis. These in vivo data assign a crucial role for pericentric H3-K9 methylation in protecting genome stability, and define the: Suv39h HMTases as important epigenetic regulators for mammalian development.", 
"Transcriptional activation of a gene involves an orchestrated recruitment of components of the basal transcription machinery and intermediate factors, concomitant with an alteration in local chromatin structure generated by posttranslational modifications of histone tails and nucleosome remodeling. We provide here a comprehensive picture of events resulting in transcriptional activation of a gene, through evaluating the estrogen receptor-alpha (NR3A1) target pS2 gene promoter in MCF-7 cells. This description integrates chromatin remodeling with a kinetic evaluation of cyclical networks of association of 46 transcription factors with the promoter, as determined by chromatin immunoprecipitation assays. We define the concept of a transcriptional clock that directs and achieves the sequential and combinatorial assembly of a transcriptionally productive complex on a promoter. Furthermore, the unanticipated findings of key roles for histone deacetylases and nucleosome-remodeling complexes in limiting transcription implies that transcriptional activation is a cyclical process that requires both activating and repressive epigenetic processes.", 
"Hepatocarcinogenesis is a slow process during which genomic changes progressively alter the hepatocellular phenotype to produce cellular intermediates that evolve into hepatocellular carcinoma. During the long pre-neoplastic stage, in which the liver is often the site of chronic hepatitis, cirrhosis, or both, hepatocyte cycling is accelerated by upregulation of mitogenic pathways, in part through epigenetic mechanisms. This leads to the production of monoclonal populations of aberrant and dysplastic hepatocytes that have telomere erosion and telomerase re-expression, sometimes microsatellite instability, and occasionally structural aberrations in genes and chromosomes. Development of dysplastic hepatocytes in foci and nodules and emergence of hepatocellular carcinoma are associated with the accumulation of irreversible structural alterations in genes and chromosomes, but the genomic basis of the malignant phenotype is heterogeneous. The malignant hepatocyte phenotype may be produced by the disruption of a number of genes that function in different regulatory pathways, producing several molecular variants of hepatocellular carcinoma. New strategies should enable these variants to be characterized.", 
"Enhancer of Zeste is a Polycomb Group protein essential for the establishment and maintenance of repression of homeotic and other genes. In the early embryo it is found in a complex that includes ESC and is recruited to Polycomb Response Elements. We show that this complex contains a methyltransferase activity that methylates lysine 9 and lysine 27 of histone H3, but the activity is lost when the E(Z) SET domain is mutated. The lysine 9 position is trimethylated and this mark is closely associated with Polycomb binding sites on polytene chromosomes but is also found in centric heterochromatin, chromosome 4, and telomeric sites. Histone H3 methylated in vitro by the E(Z)/ ESC complex binds specifically to Polycomb protein.", 
"Cytosine-5 DNA methylation occurs in the context of CpG dinucleotides in vertebrates. Aberrant methylation of CpG islands in human tumors has been shown to cause transcriptional silencing of tumor-suppressor genes. Most methods used to analyze cytosine-5 methylation patterns require cumbersome manual techniques that employ gel electrophoresis, restriction enzyme digestion, radiolabeled dNTPs or hybridization probes. The development of high-throughput technology for the analysis of DNA methylation would significantly expand our ability to derive molecular information from clinical specimens. This study describes a high-throughput quantitative methylation assay that utilizes fluorescence-based real-time PCR (TaqMan (R)) technology that requires no further manipulations after the PCR step. Methy-Light is a highly sensitive assay, capable of detecting methylated alleles in the presence of a 10 000-fold excess of unmethylated alleles. The assay is also highly quantitative and can very accurately determine the relative prevalence of a particular pattern of DNA methylation. We show that MethyLight can distinguish between mono-allelic and bi-allelic methylation of the MLH1 mismatch repair gene in human colorectal tumor specimens. The development of this technique should considerably enhance our ability to rapidly and accurately generate epigenetic profiles of tumor samples.", 
"Cancer-associated DNA hypomethylation is as prevalent as cancer-linked hypermethylation, but these two types of epigenetic abnormalities usually seem to affect different DNA sequences. Much more of the genome is generally subject to undermethylation rather than overmethylation. Genomic hypermethylation in cancer has been observed most often in CpG islands in gene regions. In contrast, very frequent hypomethylation is seen in both highly and moderately repeated DNA sequences in cancer, including heterochromatic DNA repeats, dispersed retrotransposons, and endogenous retroviral elements. Also, unique sequences, including transcription control sequences, are often subject to cancer-associated undermethylation. The high frequency of cancer-linked DNA hypomethylation, the nature of the affected sequences, and the absence of associations with DNA hypermethylation are consistent with an independent role for DNA undermethylation in cancer formation or tumor progression. Increased karyotypic instability and activation of tumor-promoting genes by cis or trans effects, that might include altered heterochromatin-euchromatin interactions, may be important consequences of DNA hypomethylation which favor oncogenesis. The relationship of DNA hypomethylation to tumorigenesis is important to be considered in the light of cancer therapies involving decreasing DNA methylation. Inducing DNA hypomethylation may have short-term anticancer effects, but might also help speed tumor progression from cancer cells surviving the DNA demethylation chemotherapy.", 
"Sirtuins (Sir2-like proteins) are present in prokaryotes and eukaryotes, Here, two new human sirtuins (SIRT6 and SIRT7) are found to be similar to a particular subset of insect, nematode, plant, and protozoan sirtuins. Molecular phylogenetic analysis of 60 sirtuin conserved core domain sequences from a diverse array of organisms (including archaeans, bacteria, yeasts, plants, protozoans, and metazoans) shows that eukaryotic Sir2-like proteins group into four main branches designated here as classes I-IV. Prokaryotic sirtuins include members of classes II and III. A fifth class of sirtuin is present in gram positive bacteria and Thermotoga maritima. Saccharomyces cerevisiae has five class I sirtuins. Caenorhabditis elegans and Drosophila melanogaster have sirtuin genes from classes I, II, and TV. The seven human sirtuin genes include all four classes: SIRT1, SIRT2, and SIRT3 are class I, SIRT4 is class II, SIRT5 is class III, and SIRT6 and SIRT7 are class IV. (C) 2000 Academic Press.", 
"Chromatin is the physiologically relevant substrate for all genetic processes inside the nuclei of eukaryotic cells. Dynamic changes in the local and global organization of chromatin are emerging as key regulators of genomic function. Indeed, a multitude of signals from outside and inside the cell converges on this gigantic signaling platform. Numerous post-translational modifications of histones, the main protein components of chromatin, have been documented and analyzed in detail. These 'marks' appear to crucially mediate the functional activity of the genome in response to upstream signaling pathways. Different layers of cross-talk between several components of this complex regulatory system are emerging, and these epigenetic circuits are the focus of this review.", 
"Allelic loss at the short arm of chromosome 3 is one of the most common and earliest events in the pathogenesis of lung cancer, and is observed in more than 90% of small-cell lung cancers (SCLCs) and in 50-80% of non-small-cell lung cancers(1,2) (NSCLCs). Frequent and early loss of heterozygosity and the presence of homozygous deletions suggested a critical role of the region 3p21.3 in tumorigenesis(2-4) and a region of common homozygous deletion in 3p21.3 was narrowed to 120 kb (ref. 5). Several putative tumour-suppressor genes located at 3p21 have been characterized, but none of these genes appear to be altered in lung cancer. Here we describe the cloning and characterization of a human RAS effector homologue (RASSF1) located in the 120-kb region of minimal homozygous deletion. We identified three transcripts, A, B and C, derived from alternative splicing and promoter usage. The major transcripts A and C were expressed in all normal tissues. Transcript A was missing in all SCLC cell lines analysed and in several other cancer cell lines. loss of expression was correlated with methylation of the CpG-island promoter sequence of RASSF1A. The promoter was highly methylated in 24 of 60 (40%) primary lung tumours, and 4 of 41 tumours analysed carried missense mutations. Re-expression of transcript A in lung carcinoma cells reduced colony formation, suppressed anchorage-independent growth and inhibited tumour formation in nude mice. These characteristics indicate a potential role for RASSF1A asa lung tumour suppresor gene.", 
"Deposition of the major histone H3 (H3.1) is coupled to DNA synthesis during DNA replication and possibly DNA repair, whereas histone variant H3.3 serves as the replacement variant for the DNA-synthesis-independent deposition pathway. To address how histories H3.1 and H3.3 are deposited into chromatin through distinct pathways, we have purified deposition machineries for these histories. The H3.1 and H3.3 complexes contain distinct histone chaperones, CAF-1 and HIRA, that we show are necessary to mediate DNA-synthesis-dependent and -independent nucleosome assembly, respectively. Notably, these complexes possess one molecule each of H3.1/H3.3 and H4, suggesting that histories H3 and H4 exist as dimeric units that are important intermediates in nucleosome formation. This finding provides new insights into possible mechanisms for maintenance of epigenetic information after chromatin duplication.", 
"Dynamic epigenetic modification of the genome occurs during early development of the mouse. Active demethylation of the paternal genome occurs in the zygote, followed by passive demethylation during cleavage stages, and de novo methylation, which is thought to happen after implantation. We have investigated these processes by using indirect immunofluoresence with an antibody to 5-methyl cytosine. In contrast to previous work, we show that demethylation of the male pronucleus is completed within 4 h of fertilisation. This activity is intricately linked with and not separable from pronucleus formation. In conditions permissive for polyspermy, up to five male pronuclei underwent demethylation in the same oocyte. Paternal demethylation in fertilised oocytes deficient for MBD2, the only candidate demethylase, occurred normally. Passive loss of methylation occurred in a stepwise fashion up to the morulae stage without any evidence of spatial compartmentalisation. De novo methylation was observed specifically in the inner cell mass (ICM) but not in the trophectoderm of the blastocyst and hence may have an important role in early lineage specification. This is the first complete and detailed analysis of the epigenetic reprogramming cycle during preimplantation development. The three phases of methylation reprogramming may have roles in imprinting, the control of gene expression, and the establishment of nuclear totipotency. (C) 2001 Elsevier Science.", 
"5-Azacytidine was first synthesized almost 40 years ago. It was demonstrated to have a wide range of anti-metabolic activities when tested against cultured cancer cells and to be an effective chemotherapeutic agent for acute myelogenous leukemia. However, because of 5-azacytidine's general toxicity, other nucleoside analogs were favored as therapeutics. The finding that 5-azacytidine was incorporated into DNA and that, when present in DNA, it inhibited DNA methylation, led to widespread use of 5-azacytidine and 5-aza-2'-deoxycytidine (Decitabine) to demonstrate the correlation between loss of methylation in specific gene regions and activation of the associated genes. There is now a revived interest in the use of Decitabine as a therapeutic agent for cancers in which epigenetic silencing of critical regulatory genes has occurred. Here, the current status of our understanding of the mechanism(s) by which 5-azacytosine residues in DNA inhibit DNA methylation is reviewed with an emphasis on the interactions of these residues with bacterial and mammalian DNA (cytosine-C5) methyltransferases. The implications of these mechanistic studies for development of less toxic inhibitors of DNA methylation are discussed.", 
"The enzyme-catalyzed acetylation of the N-terminal tail domains of core histones provides a rich potential source of epigenetic information. This may be used both to mediate transient changes in transcription, through modification of promoter-proximal nucleosomes, and for the longer-term maintenance and modulation of patterns of gene expression. The latter may be achieved by setting specific patterns of histone acetylation, perhaps involving acetylation of particular lysine residues, across relatively large chromatin domains. The histone acetylating and deacetylating enzymes (HATs and HDACs, respectively) can be targeted to specific regions of the genome and show varying degrees of substrate specificity, properties that are consistent with a role in maintaining a dynamic, acetylation-based epigenetic code. The code may be read (ie. exert a functional effect) either through non-histone proteins that bind in an acetylation-dependent manner, or through direct effects on chromatin structure. Recent evidence raises the interesting possibility that an acetylation-based code may operate through both mitosis and meiosis, providing a possible mechanism for germ-line transmission of epigenetic changes. BioEssays 22:836-845, 2000. (C) 2000 John Wiley & Sons, Inc.", 
"We have come a long way since the first reports of the existence of aberrant DNA methylation in human cancer. Hypermethylation of CpG islands located in the promoter regions of tumor suppressor genes is now firmly established as an important mechanism for gene inactivation. CpG island hypermethylation has been described in almost every tumor type. Many cellular pathways are inactivated by this type of epigenetic lesion: DNA repair (hMLH1, MGMT), cell cycle (p16(INK4a), p15(INK4b), p14(ARF)), apoptosis (DAPK), cell adherence (CDH1, CDH13), detoxification (GSTP1), etc... However, we still know little of the mechanisms of aberrant methylation and why certain genes are selected over others. Hypermethylation is not an isolated layer of epigenetic control, but is linked to the other pieces of the puzzle such as methyl-binding proteins, DNA methyltransferases and histone deacetylase, but our understanding of the degree of specificity of these epigenetic layers in the silencing of specific tumor suppressor genes remains incomplete. The explosion of user-friendly technologies has given rise to a rapidly increasing list of hypermethylated genes. Careful functional and genetic studies are necessary to determine which hypermethylation events are truly relevant for human tumorigenesis. The development of CpG island hypermethylation profiles for every form of human tumors has yielded valuable pilot clinical data in monitoring and treating cancer patients based in our knowledge of DNA methylation. Basic and translational will both be needed in the near future to fully understand the mechanisms, roles and uses of CpG island hypermethylation in human cancer. The expectations are high.", 
"Genome-wide epigenetic reprogramming in mammalian germ cells, zygote and early embryos, plays a crucial role in regulating genome functions at critical stages of development. We show here that mouse primordial germ cells (PGCs) exhibit dynamic changes in epigenetic modifications between days 10.5 and 12.5 post coitum (dpc). First, contrary to previous suggestions, we show that PGCs do indeed acquire genome-wide de novo methylation during early development and migration into the genital ridge. However, following their entry into the genital ridge, there is rapid erasure of DNA methylation of regions within imprinted and non-imprinted loci. For most genes, the erasure commences simultaneously in PGCs in both male and female embryos, which is completed within I day of development. Based on the kinetics of this process, we suggest that this is an active demethylation process initiated upon the entry of PGCs into the gonadal anlagen. The timing of reprogramming in PGCs is crucial since it ensures that germ cells of both sexes acquire an equivalent epigenetic state prior to the differentiation of the definitive male and female germ cells in which new parental imprints are established subsequently. Some repetitive elements, however, show incomplete erasure, which may be essential for chromosome stability and for preventing activation of transposons to reduce the risk of germline mutations. Aberrant epigenetic reprogramming in the germ line would cause the inheritance of epimutations that may have consequences for human diseases as suggested by studies on mouse models. (C) 2002 Elsevier Science Ireland Ltd. All rights reserved.", 
"Aberrant WNT pathway signaling is an early progression event in 90% of colorectal cancers(1). It occurs through mutations mainly of APC and less often of CTNNB1 (encoding beta-catenin)(1-3) or AXIN2 (encoding axin-2, also known as conductin)(4). These mutations allow ligand-independent WNT signaling that culminates in abnormal accumulation of free beta-catenin in the nucleus(1-3). We previously identified frequent promoter hypermethylation and gene silencing of the genes encoding secreted frizzled-related proteins (SFRPs) in colorectal cancer(5). SFRPs possess a domain similar to one in the WNT-receptor frizzled proteins and can inhibit WNT receptor binding to downregulate pathway signaling during development(6-10). Here we show that restoration of SFRP function in colorectal cancer cells attenuates WNT signaling even in the presence of downstream mutations. We also show that the epigenetic loss of SFRP function occurs early in colorectal cancer progression and may thus provide constitutive WNT signaling that is required to complement downstream mutations in the evolution of colorectal cancer.", 
"Methylation of position-specific lysine residues in histone N termini is a central modification for regulating epigenetic transitions in chromatin. Each methylatable lysine residue can exist in a mono-, di-, or trimethylated state, thereby extending the indexing potential of this particular modification. Here, we examine all possible methylation states for histone H3 lysine 9 (H3-K9) and lysine 27 (H3-K27) in mammalian chromatin. Using highly specific antibodies together with quantitative mass spectrometry, we demonstrate that pericentric heterochromatin is selectively enriched for H3-K27 monomethylation and H3-K9 trimethylation. This heterochromatic methylation profile is dependent on the Suv39h histone methyltransferases (HMTases) but independent of the euchromatic G9a HMTase. In Suv39h double null cells, pericentric heterochromatin is converted to alternative methylation imprints and accumulates H3-K27 trimethylation and H3-K9 monomethylation. Our data underscore the selective presence of distinct histone lysine methylation states in partitioning chromosomal subdomains but also reveal a surprising plasticity in propagating methylation patterns in eukaryotic chromatin.", 
"During the development of multicellular organisms, cells become different from one another by changing their genetic program in response to transient stimuli. Long after the stimulus is gone, cellular memory mechanisms enable cells to remember their chosen fate over many cell divisions. The Polycomb and Trithorax groups of proteins, respectively, work to maintain repressed or active transcription states of developmentally important genes through many rounds of cell division. Here we review current ideas on the protein and DNA components of this transcriptional memory system and how they interact dynamically with each other to orchestrate cellular memory for several hundred genes.", 
"DNA methylation has recently moved to centre stage in the aetiology of human neurodevelopmental syndromes such as the fragile X, ICF and Rett syndromes. These diseases result from the misregulation of genes that occurs with the toss of appropriate epigenetic controls during neuronal development. Recent advances have connected DNA methylation to chromatin-remodeling enzymes, and understanding this link will be central to the design of new therapeutic tools.", 
"The centromere is a chromosomal locus that ensures delivery of one copy of each chromosome to each daughter at cell division. Efforts to understand the nature and specification of the centromere have demonstrated that this central element for ensuring inheritance is itself epigenetically determined. The kinetochore, the protein complex assembled at each centromere, serves as the attachment site for spindle microtubules and the site at which motors generate forces to power chromosome movement. Unattached kinetochores are also the signal generators for the mitotic checkpoint, which arrests mitosis until all kinetochores have correctly attached to spindle microtubules, thereby representing the major cell cycle control mechanism protecting against loss of a chromosome (aneuploidy).", 
"On the histone H3 tail, Lys 9 and Lys 27 are both methylation sites associated with epigenetic repression, and reside within a highly related sequence motif ARKS. Here we show that the chromodomain proteins Polycomb (Pc) and HP1 (heterochromatin protein 1) are highly discriminatory for binding to these sites in vivo and in vitro. In Drosophila S2 cells, and on polytene chromosomes, methyl-Lys 27 and Pc are both excluded from areas that are enriched in methyl-Lys 9 and HP1. Swapping of the chromodomain regions of Pc and HP1 is sufficient for switching the nuclear localization patterns of these factors, indicating a role for their chromodomains in both target site binding and discrimination. To better understand the molecular basis for the selection of methyl-lysine binding sites, we solved the 1.8 Angstrom structure of the Pc chromodomain in complex with a H3 peptide bearing trimethyl-Lys 27, and compared it with our previously determined structure of the HP1 chromodomain in complex with a H3 peptide bearing trimethyl-Lys 9. The Pc chromodomain distinguishes its methylation target on the H3 tail via an extended recognition groove that binds five additional residues preceding the ARKS motif.", 
"DNA methylation is essential for the control of a number of biological mechanisms in mammals [1]. Mammalian development is accompanied by two major waves of genome-wide demethylation and remethylation: one during germ-cell development and the other after fertilisation [2-7]. Most previous studies have suggested that the genome-wide demethylation observed after fertilisation occurs passively, that is, by the lack of maintenance methylation following DNA replication and cell division [6,7], although one other study has reported that replication-independent demethylation may also occur during early embryogenesis [8]. Here, we report that genes that are highly methylated in sperm are rapidly demethylated in the zygote only hours after fertilisation, before the first round of DNA replication commences. By contrast, the oocyte-derived maternal alleles are unaffected by this reprogramming. They either remain methylated after fertilisation or become further methylated de novo. These results provide the first direct evidence for active demethylation of single-copy genes in the mammalian zygote and, moreover, reveal a striking asymmetry in epigenetic methylation reprogramming. Whereas paternally (sperm)-derived sequences are exposed to putative active demethylases in the oocyte cytoplasm, maternally (oocyte)-derived sequences are protected from this reaction. These results, whose generality is supported by findings of Mayer et al. [9], have important implications for the establishment of biparental genetic totipotency after fertilisation, the establishment and maintenance of genomic imprinting, and the reprogramming of somatic cells during cloning.", 
"Aberrant hypermethylation of gene promoters is a major mechanism associated with inactivation of tumor-suppressor genes in cancer. We previously showed this transcriptional silencing to be mediated by both methylation and histone deacetylase activity, with methylation being dominant. Here, we have used cDNA microarray analysis to screen for genes that are epigenetically silenced in human colorectal cancer. By screening over 10,000 genes, we show that our approach can identify a substantial number of genes with promoter hypermethylation in a given cancer; these are distinct from genes with unmethylated promoters, for which increased expression is produced by histone deacetylase inhibition alone. Many of the hypermethylated genes we identified have high potential for roles in tumorigenesis by virtue of their predicted function and chromosome position. We also identified a group of genes that are preferentially hypermethylated in colorectal cancer and gastric cancer. One of these genes, SFRP1, belongs to a gene family; we show that hypermethylation of four genes in this family occurs very frequently in colorectal cancer, providing for (i) a unique potential mechanism for loss of tumor-suppressor gene function and (ii) construction of a molecular marker panel that could detect virtually all colorectal cancer.", 
"Gene function in cancer can be disrupted either through genetic alterations, which directly mutate or delete genes, or epigenetic alterations, which alter the heritable state of gene expression. The latter events are mediated by formation of transcriptionally repressive chromatin states around gene transcription start sites and an associated gain of methylation in normally unmethylated CpG islands in these regions. The genes affected include over half of the tumor suppressor genes that cause familial cancers when mutated in the germline; the selective advantage for genetic and epigenetic dysfunction in these genes is very similar. The aberrant methylation can begin very early in tumor progression and mediate most of the important pathway abnormalities in cancer including loss of cell cycle control, altered function of transcription factors, altered receptor function, disruption of normal cell-cell and cell-substratum interaction, inactivation of signal transduction pathways, loss of apoptotic signals and genetic instability. The active role of the aberrant methylation in transcriptional silencing of genes is becoming increasingly understood and involves a synergy between the methylation and histone deacetylase (HDAC) activity, This synergy can be mediated directly by HDAC interaction with DNA methylating enzymes and by recruitment through complexes involving methyl-cytosine binding proteins. In the translational arena, the promoter hypermethylation changes hold great promise as DNA tumor markers and their potentially reversible state creates a target for cancer therapeutic strategies involving gene reactivation.", 
"DNA methylation plays an important role in mammalian development and correlates with chromatin-associated gene silencing. The recruitment of MeCP2 to methylated CpG dinucleotides represents a major mechanism by which DNA methylation can repress transcription. MeCP2 silences gene expression partly by recruiting histone deacetylase (HDAC) activity, resulting in chromatin remodeling. Here, we show that MeCP2 associates with histone methyltransferase activity in vivo and that this activity is directed against Lys(9) of histone H3. Two characterized repression domains of MeCP2 are involved in tethering the histone methyltransferase to MeCP2. We asked if MeCP2 can deliver Lys(9)H3 methylation to the H19 gene, whose activity it represses. We show that the presence of MeCP2 on nucleosomes within the repressor region of the H19 gene (the differentially methylated domain) coincides with an increase in H3 Lys(9) methylation. Our data provide evidence that MeCP2 reinforces a repressive chromatin state by acting as a bridge between two global epigenetic modifications, DNA methylation and histone methylation.", 
"A large set of endogenous small RNAs of predominantly 21 to 24 nucleotides was identified in Arabidopsis. These small RNAs resembled micro-RNAs from animals and were similar in size to small interfering RNAs; that accumulated during RNA silencing triggered by multiple types of inducers. Among the 125 sequences identified, the vast majority (90%) arose from intergenic regions, although small RNAs corresponding to predicted protein-coding genes, transposon-like sequences, and a structural RNA gene also were identified. Evidence consistent with the derivation of small RNAs of both polarities, and from highly base-paired precursors, was obtained through the identification and analysis of clusters of small RNA loci. The accumulation of specific small RNAs was regulated developmentally. We propose that Arabidopsis small RNAs; participate in a wide range of post-transcriptional and epigenetic events.", 
"Recent results from diverse organisms point to a self-reinforcing network of interactions among the three best-characterized covalent modifications that mark heterochromatin: histone hypoacetylation, histone H3-Lys9 methylation, and cytosine methylation. These modification systems suggest a mechanistic basis for spreading of heterochromatin over large domains and for stable epigenetic inheritance of the silent state. All three modifications used in packaging heterochromatin are also used in stable silencing of euchromatic genes.", 
"Recent data in humans and animals suggest that assisted reproductive technology (ART) might affect the epigenetics of early embryogenesis and might cause birth defects. We report the first evidence, to our knowledge, that ART is associated with a human overgrowth syndrome-namely, Beckwith-Wiedemann syndrome (BWS). In a prospective study, the prevalence of ART was 4.6% (3 of 65), versus the background rate of 0.8% in the United States. A total of seven children with BWS were born after ART-five of whom were conceived after intracytoplasmic sperm injection. Molecular studies of six of the children indicate that five of the six have specific epigenetic alterations associated with BWS-four at LIT1 and one at both LIT1 and H19. We discuss the implications of our finding that ART is associated with human overgrowth, similar to the large offspring syndrome reported in ruminants.", 
"Cadmium is a heavy metal of considerable environmental and occupational concern. Cadmium compounds are classified as human carcinogens by several regulatory agencies. The most convincing data that cadmium is carcinogenic in humans comes from studies indicating occupational cadmium exposure is associated with lung cancer. Cadmium exposure has also been linked to human prostate and renal cancer, although this linkage is weaker than for lung cancer. Other target sites of cadmium carcinogenesis in humans, such as liver, pancreas and stomach, are considered equivocal. In animals, cadmium effectively induces cancers at multiple sites and by various routes. Cadmium inhalation in rats induces pulmonary adenocarcinomas, in accord with its role in human lung cancer. Cadmium can induce tumors and/or preneoplastic lesions within the rat prostate after ingestion or injection. At relatively high doses, cadmium induces benign testicular tumors in rats, but these appear to be due to early toxic lesions and loss of testicular function, rather than from a specific carcinogenic effect of cadmium. Like many other metals, cadmium salts will induce mesenchymal tumors at the site of subcutaneous (s.c.) or intramuscular (i.m.) injections, but the human relevance of these is dubious. Other targets of cadmium in rodents include the liver, adrenal, pancreas, pituitary, and hematopoietic system. With the exception of testicular tumors in rodents, the mechanisms of cadmium carcinogenesis are poorly defined. Cadmium can cause any number of molecular lesions that would be relevant to oncogenesis in various cellular model systems. Most studies indicate cadmium is poorly mutagenic and probably acts through indirect or epigenetic mechanisms, potentially including aberrant activation of oncogenes and suppression of apoptosis. (C) 2003 Elsevier B.V. All rights reserved.", 
"Diverse post-translational modifications of histone amino termini represent an important epigenetic mechanism for the organisation of chromatin structure and the regulation of gene activity. Within the past two years, great progress has been made in understanding the functional implications of histone methylation; in particular through the characterisation of histone methyltransferases that direct the site-specific methylation of, for example, lysine 9 and lysine 4 positions in the histone H3 amino terminus. All known histone methyltransferases of this type contain the evolutionarily conserved SET domain and appear to be able to stimulate either gene repression or gene activation. Methylation of H3 Lys9 and Lys4 has been visualised in native chromatin, indicating opposite roles in structuring repressive or accessible chromatin domains. For example, at the mating-type loci in Schizosaccharomyces pombe, at pericentric heterochromatin and at the inactive X chromosome in mammals, striking differences between these distinct marks have been observed. H3 Lys9 methylation is also important to direct additional epigenetic signals such as DNA methylation - for example, in Neurospora crassa and in Arabidopsis thaliana. Together, the available data strongly establish histone lysine methylation as a central modification for the epigenetic organisation of eukaryotic genomes.", 
"Polyploidy has long been recognized as a prominent force shaping the evolution of eukaryotes, especially flowering plants. New phenotypes often arise with polyploid formation and can contribute to the success of polyploids in nature or their selection for use in agriculture. Although the causes of novel variation in polyploids are not well understood, they could involve changes in gene expression through increased variation in dosage-regulated gene expression, altered regulatory interactions, and rapid genetic and epigenetic changes. New research approaches are being used to study these mechanisms and the results should provide a more complete understanding of polyploidy.", 
"Adult germline stem cells are capable of self-renewal, tissue regeneration and production of large numbers of differentiated progeny. We show here that the classical mouse mutant luxoid affects adult germline stem cell self-renewal. Young homozygous luxoid mutant mice produce limited numbers of normal spermatozoa and then progressively lose their germ line after birth. Transplantation studies showed that germ cells from mutant mice did not colonize recipient testes, suggesting that the defect is intrinsic to the stem cells. We determined that the luxoid mutant contains a nonsense mutation in the gene encoding Plzf, a transcriptional repressor that regulates the epigenetic state of undifferentiated cells, and showed that Plzf is coexpressed with Oct4 in undifferentiated spermatogonia. This is the first gene shown to be required in germ cells for stem cell self-renewal in mammals.", 
"The covalent modification of nucleosomal histones has emerged as a major determinant of chromatin structure and gene activity. To understand the interplay between various histone modifications, including acetylation and methylation, we performed a genome-wide chromatin structure analysis in a higher eukaryote. We found a binary pattern of histone modifications among euchromatic genes, with active genes being hyperacetylated for H3 and H4 and hypermethylated at Lys 4 and Lys 79 of 113, and inactive genes being hypomethylated and deacetylated at the same residues. Furthermore, the degree of modification correlates with the level of transcription, and modifications are largely restricted to transcribed regions, suggesting that their regulation is tightly linked to polymerase activity.", 
"Manipulation or non-physiological embryo culture environments can lead to defective fetal programming in livestock. Our demonstration of reduced fetal methylation and expression of ovine IGF2R suggests pre-implantation embryo procedures may be vulnerable to epigenetic alterations in imprinted genes. This highlights the potential benefits of epigenetic diagnostic screening in developing embryo procedures.", 
"Proteins from the Polycomb group (PcG) are epigenetic chromatin modifiers involved in cancer development and also in the maintenance of embryonic and adult stem cells, The therapeutic potential of stem cells and the growing conviction that tumors contain stem cells highlights the importance of understanding the extrinsic and intrinsic circuitry controlling stem cell fate and their connections to cancer.", 
"Genomes are organized into active regions known as euchromatin and inactive regions known as heterochromatin, or silenced chromatin. This review describes contemporary knowledge and models for how silenced chromatin in Saccharomyces cerevisiae forms, functions, and is inherited. In S. cerevisiae, Sir proteins are the key structural components of silenced chromatin. Sir proteins interact first with silencers, which dictate which regions are silenced, and then with histone tails in nucleosomes as the Sir proteins spread from silencers along chromosomes. Importantly, the spreading of silenced chromatin requires the histone deacetylase activity of Sir2p. This requirement leads to a general model for the spreading and inheritance of silenced chromatin or other special chromatin states Such chromatin domains are marked by modifications of the nucleosomes or DNA, and this mark is able to recruit an enzyme that makes further marks. Thus, among different organisms, multiple forms of repressive chromatin can be formed using similar strategies but completely different proteins. We also describe emerging evidence that mutations that cause global changes in the modification of histones can alter the balance between euchromatin and silenced chromatin within a cell.", 
"T lymphocytes differentiate in discrete stages within the thymus. Immature thymocytes lacking CD4 and CD8 coreceptors differentiate into double-positive cells (CD4(+)CD8(+)), which are selected to become either CD4(+)CD8(-)helper cells or CD4(-)CD8(+) cytotoxic cells. A stage-specific transcriptional silencer regulates expression of CD4 in both immature and CD4(-)CD8(+) thymocytes. We show here that binding sites for Runt domain transcription factors are essential for CD4 silencer function at both stages, and that different Runx family members are required to fulfill unique functions at each stage. Runxi is required for active repression in CD4(-)CD8(-) thymocytes whereas Runx3 is required for establishing epigenetic silencing in cytotoxic lineage thymocytes. Runx3-deficient cytotoxic T cells, but not helper cells, have defective responses to antigen, suggesting that Runx proteins have critical functions in lineage specification and homeostasis of CD8lineage T lymphocytes.", 
"The Cyclin D-Cdk4,6/lNK4/Rb/E2F pathway plays a key role in controlling cell growth by integrating multiple mitogenic and antimitogenic stimuli. The components of this pathway are gene families with a high level of structural and functional redundancy and are expressed in an overlapping fashion in most tissues and cell types. Using classical transgenic technology as well as gene-targeting in ES cells, a series of mouse models have been developed to study the in vivo function of individual components of this pathway in both normal homeostasis and tumor development. These models have proven to be useful to define specific as well as redundant roles among members of these cell cycle regulatory gene families. This pathway is deregulated in the vast majority of human tumors by genetic and epigenetic alterations that target at least some of its key members such as Cyclin D1, Cdk4, INK4a and INK4b, pRb etc. As a consequence, some of these molecules are currently being considered as targets for cancer therapy, and several novel molecules, such as Cdk inhibitors, are under development as potential anti-cancer drugs. (C) 2002 Elsevier Science B.V. All rights reserved.", 
"Twin studies have been vital for establishing an important genetic contribution to the etiology of schizophrenia. The five newest studies since 1995 from Europe and Japan have confirmed earlier findings. They yielded probandwise concordance rates of 41-65% in monozygotic (MZ) pairs and 0-28% in dizygotic (DZ) pairs, and heritability estimates of approximately 80-85%. Twin studies are also Valuable for investigating the etiological relationships between schizophrenia and other disorders, and the genetic basis of clinical heterogeneity within schizophrenia. Studies of discordant MZ pairs provide further insights into noninherited factors that contribute to the multifactorial etiology of this disorder. More recently, twin studies have begun to be used to directly investigate molecular genetic and epigenetic processes underlying schizophrenia. (C) 2000 Wiley-Liss, Inc.", 
"Interspecific or intergeneric hybridization, followed by chromosome doubling, can lead to the formation of new allopolyploid species. Recent studies indicate that allopolyploid formation is associated with genetic and epigenetic changes, although little is known about the type of changes that occur, how rapidly they occur, and the type of sequences involved. To address these matters, we have surveyed F1 hybrids between diploid species from the wheat (Aegilops and Triticum) group and their derived allotetraploids by screening a large number of loci using amplified fragment length polymorphism and DNA gel blot analysis and by assaying the extent of cytosine methylation. We found that sequence elimination is one of the major and immediate responses of the wheat genome to wide hybridization or allopolyploidy, that it affects a large fraction of the genome, and that it is reproducible. In one cross between Ae. sharonensis x Ae. umbellulata, 14% of the loci from Ae. sharonensis were eliminated compared with only 0.5% from Ae. umbellulata, with most changes occurring in the F1 hybrid. In contrast, crosses between Ae. longissima x T. urartu showed that sequence elimination was more frequent after chromosome doubling. Alterations in cytosine methylation occurred in similar to 13% of the loci, either in the F1 hybrid or in the allopolyploid. For eight of nine bands that were isolated, the sequences that underwent elimination corresponded to low-copy DNA, whereas alterations in methylation patterns affected both repetitive DNA sequences, such as retrotransposons, and low-copy DNA in approximately equal proportions.", 
"Cytosine methylation in CpG dinucleotides is believed to be important in gene regulation, and is generally associated with reduced levels of transcription. Methylation-mediated gene silencing involves a series of DNA-protein and protein-protein interactions that begins with the binding of methyl-CpG binding proteins (MBPs) followed by the recruitment of histone-modifying enzymes that together promote chromatin condensation and inactivation. It is widely known that alterations in methylation patterns, and associated gene activities, are often found in human tumors. However, the mechanisms by which methylation patterns are altered are not currently understood. In this paper, we investigate the impact of oxidative damage to a methyl-CpG site on MBP binding by the selective placement of 8-oxoguanine (8-oxoG) and 5-hydroxymethylcytosine (HmC) in a MBP recognition sequence. Duplexes containing these specific modifications were assayed for binding to the methyl-CpG binding domain (MBD) of one member of the MBP family, methyl-CpG binding protein 2 (MeCP2). Our results reveal that oxidation of either a single guanine to 8-oxoG or of a single 5mC to HmC, significantly inhibits binding of the MBD to the oligonucleotide duplex, reducing the binding affinity by at least an order of magnitude. Oxidative damage to DNA could therefore result in heritable, epigenetic changes in chromatin organization.", 
"Rett syndrome(1) (RTT, MIM 312750) is a progressive neurodevelopmental disorder and one of the most common causes of mental retardation in females, with an incidence of 1 in 10,000-15,000 (ref. 2). Patients with classic RTT appear to develop normally until 6-18 months of age, then gradually lose speech and purposeful hand use, and develop microcephaly, seizures, autism, ataxia, intermittent hyperventilation and stereotypic hand movements(3). After initial regression, the condition stabilizes and patients usually survive into adulthood. As RTT occurs almost exclusively in females, it has been proposed that RTT is caused by an X-linked dominant mutation with lethality in hemizygous males(3-8). Previous exclusion mapping studies using RTT families mapped the locus to Xq28 (refs 6,7,9-11). Using a systematic gene screening approach, we have identified mutations in the gene (MECP2) encoding X-linked methyl-CpG-binding protein 2 (MeCP2) as the cause of some Eases of RTT. MeCP2 selectively binds CpG dinucleotides in the mammalian genome and mediates transcriptional repression through interaction with histone deacetylase and the corepressor SIN3A (refs 12,13). In 5 of 21 sporadic patients, we found 3 de novo missense mutations in the region encoding the highly conserved methyl-binding domain (MBD) as well as a de novo frameshift and a de novo nonsense mutation, both of which disrupt the transcription repression domain (TRD). In two affected half-sisters of a RTT family, we found segregation of an additional missense mutation not detected in their obligate carrier mother. This suggests that the mother is a germline mosaic for this mutation. Our study reports the first disease-causing mutations in RTT and points to abnormal epigenetic regulation as the mechanism underlying the pathogenesis of RTT.", 
"The discovery of numerous hypermethylated promoters of tumour-suppressor genes, along with a better understanding of gene-silencing mechanisms, has moved DNA methylation from obscurity to recognition as an alternative mechanism of tumour-suppressor inactivation in cancer. Epigenetic events can also facilitate genetic damage, as illustrated by the increased mutagenicity of 5-methylcytosine and the silencing of the MLH1 mismatch repair gene by DNA methylation in colorectal tumours. We review here current mechanistic understanding of the role of DNA methylation in malignant transformation, and suggest Knudson's two-hit hypothesis should now be expanded to include epigenetic mechanisms of gene inactivation.", 
"Densely methylated DNA associates with transcriptionally repressive chromatin characterized by the presence of underacetylated histones(1,2). Recently, these two epigenetic processes have been dynamically linked. The methyl-CpG-binding protein MeCP2 appears to reside in a complex with histone deacetylase activity(3,4). MeCP2 can mediate formation of transcriptionally repressive chromatin on methylated promoter templates in vitro, and this process can be reversed by trichostatin A (TSA), a specific inhibitor of histone deacetylase(3-5). Little is known, however, about the relative roles of methylation and histone deacetylase activity in the stable inhibition of transcription on densely methylated endogenous promoters, such as those for silenced alleles of imprinted genes(6), genes on the female inactive X chromosome(7) and tumour-suppressor genes inactivated in cancer cells(8,9). We show here that the hypermethylated genes MLH1, TIMP3 (TIMP3), CDKN2B (INK4B, p15) and CDKN2A (INK4, p16) cannot be transcriptionally reactivated with TSA alone in tumour cells in which we have shown that TSA alone can upregulate the expression of non-methylated genes. Following minimal demethylation and slight gene reactivation in the presence of low dose 5-aza-2'deoxycytidine (5Aza-dC), however, TSA treatment results in robust re-expression of each gene. TSA does not contribute to demethylation of the genes, and none of the treatments alter the chromatin structure associated with the hypermethylated promoters. Thus, although DNA methylation and histone deacetylation appear to act as synergistic layers for the silencing of genes in cancer, dense CpG island methylation is dominant for the stable maintenance of a silent state at these loci.", 
" ", 
"'Viable yellow' (A(vy)/alpha) mice are larger, obese, hyperinsulinemic, more susceptible to cancer, and, on average, shorter lived than their non-yellow siblings. They are epigenetic mosaics ranging from a yellow phenotype with maximum ectopic agouti overexpression, through a continuum of mottled agouti/yellow phenotypes with partial agouti overexpression, to a pseudoagouti phenotype with minimal ectopic expression. Pseudoagouti A(vy)/a mice are lean, healthy, and longer lived than their yellow siblings. Here we report that feeding pregnant black a/a dams methyl-supplemented diets alters epigenetic regulation of agouti expression in their offspring, as indicated by increased agouti/black mottling in the direction of the pseudoagouti phenotype, We also present confirmatory evidence that epigenetic phenotypes are maternally heritable. Thus A(vy) expression, already known to be modulated by imprinting, strain-specific modification, and maternal epigenetic inheritance, is also modulated by maternal diet. These observations suggest, at least in this special case, that maternal dietary supplementation may positively affect health and longevity of the offspring, Therefore, this experimental system should be useful for identifying maternal factors that modulate epigenetic mechanisms, especially DNA methylation, in developing embryos.", 
"The yeast Sir2 protein regulates epigenetic gene silencing and as a possible antiaging effect it suppresses recombination of rDNA. Studies involving cobB, a bacterial SIR2-like gene, have suggested it could encode a pyridine nucleotide transferase. Here five human sirtuin cDNAs are characterized. The SIRT1 sequence has the closest homology to the S. cerevisiae Sir2p. The SIRT4 and SIRT5 sirtuins more closely resemble prokaryotic sirtuin sequences. The five human sirtuins are widely expressed in fetal and adult tissues. Recombinant E. coli cobT and cobB proteins each showed a weak NAD-dependent mono-ADP-ribosyltransferase activity using 5,6-dimethylbenzimidazole as a substrate. Recombinant E. coli cobB and human SIRT2 sirtuin proteins were able to cause radioactivity to be transfered from [P-32]NAD to bovine serum albumin (BSA). When a conserved histidine within the human SIRT2 sirtuin was converted to a tyrosine, the mutant recombinant protein was unable to transfer radioactivity from [P-32]NAD to BSA. These results suggest that the sirtuins may function via mono-ADP-ribosylation Of proteins. (C) 1999 Academic Press.", 
"Gene activation in eukaryotes requires chromatin remodeling complexes like Swi/Snf and histone acetylases like SAGA. How these factors are recruited to promoters is not yet understood. Using CHIP, we measured recruitment of Swi/Snf, SAGA, the repressor Ash1p, and transcription factors Swi5p and SBF to the HO endonuclease promoter as cells progress through the yeast cell cycle. Swi5p's entry into nuclei at the end of anaphase recruits Swi/Snf, which then recruits SAGA. These two factors then facilitate SBF's binding. Ash1p, which only accumulates in daughter cell nuclei, binds to HO soon after Swi5p and aborts recruitment of Swi/Snf, SAGA, and SBF. Swi5p remains at HO for only 5 min. Swi/Snf's and SAGA's subsequent persistence at HO is self sustaining and constitutes an epigenetic memory of HO's transient interaction with Swi5p.", 
"Altering cytosine methylation by genetic means leads to a variety of developmental defects in mice(1), plants(2-5) and fungi(6,7) Deregulation of cytosine methylation also has a role in human carcinogenesis(8). In some cases, these defects have been tied to the inheritance of epigenetic alterations (such as chromatin imprints and DNA methylation patterns) that do not involve changes in DNA sequence(3,8-10). Using a forward genetic screen, we identified a gene (DDM1, decrease in DNA methylation) from the flowering plant Arabidopsis thaliana required to maintain normal cytosine methylation patterns(11). Additional ddm1 alleles (som4, 5, 6, 7, 8) were isolated in a selection for mutations that relieved transgene silencing(12) (E.J.R., unpublished data). Loss of DDM1 function causes a 70% reduction of genomic cytosine methylation, with most of the immediate hypomethylation occurring in repeated sequences(11). In contrast, many low-copy sequences initially retain their methylation in ddm1 homozygotes, but lose methylation over time as the mutants are propagated through multiple generations by self-pollination(3,13). The progressive effect of ddm1 mutations on low-copy sequence methylation suggests that ddm1 mutations compromise the efficiency of methylation of newly incorporated cytosines after DNA replication. In parallel with the slow decay of methylation during inbreeding, ddm1 mutants accumulate heritable alterations (mutations or stable epialleles) at dispersed sites in the genome that lead to morphological abnormalities(3,5,14). Here we report that DDM1 encodes a SWI2/SNF2-like protein, implicating chromatin remodelling as an important process for maintenance of DNA methylation and genome integrity.", 
"Chromatin assembly factor 1 (CAF-1) is required for inheritance of epigenetically determined chromosomal states in vivo and promotes assembly of chromatin during DNA replication in vitro. Herein, we demonstrate that after DNA replication, replicated, but not unreplicated, DNA is also competent for CAF-1-dependent chromatin assembly. The proliferating cell nuclear antigen (PCNA), a DNA polymerase clamp, is a component of the replication-dependent marking of DNA for chromatin assembly. The clamp loader, replication factor C (RFC), can reverse this mark by unloading PCNA from the replicated DNA. PCNA binds directly to p150, the largest subunit of CAF-1, and the two proteins colocalize at sites of DNA replication in cells. We suggest that PCNA and CAF-1 connect DNA replication to chromatin assembly and the inheritance of epigenetic chromosome states.", 
"A handful of autosomal genes in the mammalian genome are inherited in a silent state from one of the two parents, and in a fully active form from the other, thereby rendering the organism functionally hemizygous for imprinted genes. To date 19 imprinted genes have been identified; 5 are expressed from the maternal chromosome while the rest are expressed from the paternal chromosome. Allele-specific methylation of CpG residues, established in one of the germlines and maintained throughout embryogenesis, has been clearly implicated in the maintenance of imprinting in somatic cells. Although the function of imprinting remains a subject of some debate, the process is thought to have an important role in regulating the rate of fetal growth.", 
"Members of the Plasmodium falciparum var gene family encode clonally variant adhesins, which play an important role in the pathogenicity of tropical malaria. Here we employ a selective panning protocol to generate isogenic P falciparum populations with defined adhesive phenotypes for CD36, ICAM-1 and CSA, expressing single and distinct var gene variants. This technique has established the framework for examining var gene expression, its regulation and switching. It was found that var gene switching occurs irt situ. Ubiquitous transcription of all var gene variants appears to occur in early ring stages. However, var gene expression is tightly regulated in trophozoites and is exerted through a silencing mechanism. Transcriptional control is mutually exclusive in parasites that express defined adhesive phenotypes. lit situ var gene switching is apparently mediated at the level of transcriptional initiation, as demonstrated by nuclear run-on analyses. Our results suggest that an epigenetic mechanism(s) is involved in var gene regulation.", 
"Nuclear receptors (NRs) are ligand-inducible transcription factors that mediate complex effects on development, differentiation and homeostasis. They regulate the transcription of their target genes through binding to cognate DNA sequences as homodimers or heterodimers, The molecular mechanisms underlying transcriptional activation by NRs are still poorly understood, although intermediary factors (mediators) appear to be involved in mediating the transactivation functions of NRs, TIF1 has been identified previously as a protein that interacts specifically with the ligand binding domain of several nuclear receptors, both in yeast and in vitro. The characteristics of these interactions have led us to suggest that TIF1 might be a mediator of the NR ligand-inducible activation function AF-2, Using a two-hybrid screening in yeast, we have now identified two TLFP-binding proteins, mHP1 alpha and mMOD1, that are mouse homologues of the Drosophila heterochromatinic protein 1. Using mHP1 alpha as a bait in a second two-hybrid screening, we have isolated cDNAs encoding proteins that are also very likely to be involved in chromatin structure and function, as well as a protein structurally and functionally related to TIF1 (renamed TLF1 alpha), which was named TIF1 beta. Here we discuss how the function of members of the TIF1 family in the control of transcription could be exerted at the level of the structure of the chromatin template.", 
"The past few years have seen a wider acceptance of a role for DNA methylation in cancer. This can be attributed to three developments. First, the documentation of the over-representation of mutations at CpG dinucleotides has convincingly implicated DNA methylation in the generation of oncogenic point mutations. The second important advance has been the demonstration of epigenetic silencing of tumor suppressor genes by DNA methylation. The third development has been the utilization of experimental methods to manipulate DNA methylation levels. These studies demonstrate that DNA methylation changes in cancer cells are not mere by-products of malignant transformation, but can play an instrumental role in the cancer process. It seems clear that DNA methylation plays a variety of roles in different cancer types and probably at different stages of oncogenesis. DNA methylation is intricately involved in a wide diversity of cellular processes. Likewise, it appears to exert its influence on the cancer process through a diverse array of mechanisms. It is our task not only to identify these mechanisms, but to determine their relative importance for each stage and type of cancer. Our hope then will be to translate that knowledge into clinical applications.", 
"The inactive X chromosome differs from the active X in a number of ways; some of these, such as allocyclic replication and altered histone acetylation, are associated with all types of epigenetic silencing, whereas others, such as DNA methylation, are of more restricted use. These features are acquired progressively by the inactive X after onset of initiation. Initiation of X-inactivation is controlled by the X-inactivation center (Xic) and influenced by the X chromosome controlling element (Xce), which causes primary nonrandom X-inactivation. Other examples of nonrandom X-inactivation are also presented in this review. The definition of a major role for Xist, a noncoding RNA, in X-inactivation has enabled investigation of the mechanism leading to establishment of the heterochromatinized X-chromosome and also of the interactions between X-inactivation and imprinting as well as between X-inactivation and developmental processes in the early embryo.", 
"The centromere is required to ensure the equal distribution of replicated chromosomes to daughter nuclei. Centromeres are frequently associated with heterochromatin, an enigmatic nuclear component that causes the epigenetic transcriptional repression of nearby marker genes (position-effect variegation of silencing). The process of chromosome segregation by movement along microtubules to spindle poles is highly conserved, yet the putative cis-acting centromeric DNA sequences bear little or no similarity across species. Recently, studies in several systems have revealed that the centromere itself might be epigenetically regulated and that the higher-order structure of the underlying heterochromatin contributes to centromere function and kinetochore assembly.", 
"The chrome and SET domains are conserved sequence moths present in chromosomal proteins that function in epigenetic control of gene expression, presumably by modulating higher order chromatin, Based on sequence information from the SET domain, we have isolated human (SUV39H1) and mouse (Suv39h1) homologues of the dominant Drosophila modifier of position-effect-variegation (PEV) Su(var)3-9. Mammalian homologues contain, in addition to the SET domain, the characteristic chrome domain, a combination that is also preserved in the Schizosaccharyomyces pombe silencing factor clr4. Chromatin-dependent gene regulation is demonstrated by the potential of human SUV39H1 to increase repression of the pericentromeric white marker gene in transgenic flies. Immunodetection of endogenous Suv39h1/SUV39H1 proteins in a variety of mammalian cell lines reveals enriched distribution at heterochromatic foci during interphase and centromere-specific localization during metaphase, In addition, Suv39h1/SUV39H1 proteins associate with M31, currently the only other characterized mammalian SU(VAR) homologue, These data indicate the existence of a mammalian SU(VAR) complex and define Suv39h1/SUV39H1 as novel components of mammalian higher order chromatin.", 
"post-transcriptional gene silencing (PTGS) is a homology-dependent process that reduces cytoplasmic RNA levels. In several experimental systems, there is also an association of PTGS with methylation of DNA. To investigate this association, we used plants carrying a transgene encoding the green fluorescent protein (GFP), Gene silencing was induced using potato virus X RNA vectors carrying parts of the coding sequence or the promoter of the GFP transgene. In each instance, homology-based, RNA-directed methylation was associated with silencing. When the GFP-transcribed region was targeted, PTGS affected both transgene and viral RNA revels. When methylation was targeted to a promoter region, transgene RNA levels were reduced; however, viral RNA revels were unaffected. For comparison, we induced PTGS of the gene encoding the endogenous ribulose-1,5-bisphosphate carboxylase oxygenase (Rubisco) small subunit (rbcS) by inoculation with potato virus X-rbcS, In this example, no methylation of the rbcS DNA was associated with the reduction in rbcS transcript levels, and viral RNA levels were unaffected. Finally, we investigated DNA methylation by using GFP-transformed plants in which PTGS was induced by localized introduction of a T-DNA carrying GFP sequences. In these plants, there was methylation of a GFP transgene associated with systemic spread of a gene-silencing signal from the infiltrated part of the plant. This transgene methylation was not affected when systemic PTGS was blocked by suppressors of silencing encoded by potato virus Y and cucumber mosaic virus. Combined, these data support an epigenetic model of PTGS in which transgene methylation is associated with an RNA-DNA interaction that ensures that PTGS is maintained.", 
"Although the link between the BRCA1 tumour-suppressor gene and hereditary breast and ovarian cancer is established(1-5), the role, if any, of BRCA1 in non-familial cancers is unclear. BRCA1 mutations are rare in sporadic cancers(6-8), but loss of BRCA1 resulting from reduced expression or incorrect subcellular localization(9,10) is postulated to be important in non-familial breast and ovarian cancers. Epigenetic loss, however, has not received general acceptance due to controversy regarding the subcellular localization of BRCA1 proteins, reports of which have ranged from exclusively nuclear(11-15), to conditionally nuclear(10), to the ER/golgi(16), to cytoplasmic invaginations into the nucleus(17). In an attempt to resolve this issue, we have comprehensively characterized 19 anti-BRCA1. antibodies. These reagents detect a 220-kD protein localized in discrete nuclear foci in all epithelial cell lines, including those derived from breast malignancies. Immunohistochemical staining of human breast specimens also revealed BRCA1 nuclear foci in benign breast, invasive lobular cancers and low-grade ductal carcinomas. Conversely, BRCA1 expression was reduced or undetectable in the majority of high-grade, ductal carcinomas, suggesting that absence of BRCA1 may contribute to the pathogenesis of a significant percentage of sporadic breast cancers.", 
"CpG island hypermethylation is known to be associated with gene silencing in cancer. This epigenetic event is generally accepted as a stochastic process in tumor cells resulting from aberrant DNA methyltransferase (DNA-MTase) activities. Specific patterns of CPG island methylation could result from clonal selection of cells having growth advantages due to silencing of associated tumor suppressor genes. Alternatively, methylation patterns may be determined by other, as yet unidentified factors, To explore further the underlying mechanisms, we developed a novel array-based method, called differential methylation hybridization (DMH), which allows a genome-wide screening of hypermethylated CpG islands in tumor cells. DMH was used to determine the methylation status of >276 CpG island loci in a group of breast cancer cell lines. Between 5 and 14% of these loci were hypermethylated extensively in these cells relative to a normal control. Pattern analysis of 30 positive loci by Southern hybridization indicated that CpG islands might differ in their susceptibility to hypermethylation, Loci exhibiting preexisting methylation in normal controls were more susceptible to de novo methylation in these cancer cells than loci without this condition, In addition, these cell lines exhibited different intrinsic abilities to methylate CpG islands not directly associated with methyltransferase activities. Our study provides evidence that, aside from random DNA-MTase action, additional cellular factors exist that govern aberrant methylation in breast cancer cells.", 
"Genomic reprogramming of primordial germ cells (PGCs), which includes genome-wide demethylation, prevents aberrant epigenetic modifications from being transmitted to subsequent generations, This process also ensures that homologous chromosomes first acquire an identical epigenetic status before an appropriate switch in the imprintable loci in the female and male germ lines. Embryonic germ (EG) cells have a similar epigenotype to PGCs from which they are derived, We used EG cells to investigate the mechanism of epigenetic modifications in the germ line by analysing the effects on a somatic nucleus in the EG-thymic lymphocyte hybrid cells. There were striking changes in methylation of the somatic nucleus, resulting in demethylation of several imprinted and non-imprinted genes, These epigenetic modifications were heritable and affected gene expression as judged by re-activation of the silent maternal allele of Peg1/Mest imprinted gene in the somatic nucleus. This remarkable change in the epigenotype of the somatic nucleus is consistent with the observed pluripotency of the EG-somatic hybrid cells as they differentiated into a variety of tissues in chimeric embryos. The epigenetic modifications observed in EG-somatic cell hybrids in vitro are comparable to the reprogramming events that occur during germ cell development.", 
"To determine which CD3 components are required for early T cell development, we generated mice with a targeted mutation of the CD3-epsilon gene and characterized their T cell populations relative to those found in CD3-zeta/eta- and recombinase activating gene (RAG)-deficient mice, In the absence of intact CD3-epsilon subunit, thymocytes do not progress beyond the CD44(-/low)CD25(+) triple-negative stage and appear to be arrested at the very same developmental control point as RAG-deficient thymocytes, In contrast, the disruption of the CD3-zeta/eta gene does not totally abrogate the progression through this control point, CD3-epsilon-deficient thymocytes do rearrange their T cell receptor (TCR) beta gene segments and produce low levels of full-length TCR beta transcripts. Taken together, these results establish an essential role for the CD3-epsilon gene products during T cell development and further suggest that the CD3-epsilon polypeptides start to exert their function as part of a pre-TCR through which CD44(-/low)CD25(+) triple-negative cells monitor the occurrence of productive TCR beta gene rearrangements. Finally, the absence of intact CD3-epsilon polypeptides had no discernible effect on the completion of TCR gamma and TCR delta gene rearrangements, emphasizing that they are probably not subjected to the same epigenetic controls as those operating on the expression of TCR alpha and beta genes.", 
"The constitutive DNase I hypersensitive site at the 5' end of the chicken beta-globin locus marks the boundary of the active chromatin domain in erythroid cells. The DNA sequence containing this site has the properties of an insulator, as shown by its ability in stable transformation experiments to block enhancer-promoter interaction when it lies between the two, but not when it lies outside, and to protect against position effects in Drosophila. We now show that the chicken insulator can protect a stably integrated gene, which is otherwise subject to great variability of expression, from chromatin-mediated repression in cell culture. When the integrated reporter gene is surrounded by insulator elements, stably transformed cell lines display consistent enhancer-dependent expression levels, in accord with the strength of the enhancer. In the absence of insulators, long-term nonselective propagation of cells carrying the integrated reporter gene results in gradual extinction of the reporter's expression, with expression patterns from tandemly repeated inserted genes suggesting that the extinction of adjacent genes is coupled. We show that the uninsulated reporter genes, in addition to becoming transcriptionally inactive, lose several epigenetic hallmarks of active chromatin, including nuclease accessibility, DNA hypomethylation, and histone hyperacetylation during time in culture. Treatment with inhibitors of histone deacetylase or DNA methylation reverses the extinction of the uninsulated genes. Extinction is completely prevented by flanking the reporter construct with insulators. Furthermore, in contrast to the uninsulated reporter genes, chromatin over the insulated genes retains nuclease accessibility and histone hyperacetylation. However, there is no clear correlation between the presence of the insulators and the level of DNA methylation. This leads us to propose a model for the insulator's ability to protect against extinction in the transformed cell lines and to function as a chromatin boundary for the chicken beta-globin locus in normal erythroid cells.", 
"Kruppel-associated box (KRAB) domains are present in approximately one-third of all human zinc finger proteins (ZFPs) and are potent transcriptional repression modules. We have previously cloned a corepressor for the KRAB domain, KAP-1, which is required for KRAB-mediated repression in vivo. To characterize the repression mechanism utilized by KAP-1, we have analyzed the ability of KAP-1 to interact with murine (M31 and M32) and human (HP1 alpha and HP1 gamma) homologues of the HPI protein family, a class of nonhistone heterochromatin-associated proteins with a well-established epigenetic gene silencing function in Drosophila, In vitro studies confirmed that KAP-1 is capable of directly interacting with M31 and hHP1 alpha, which are normally found in centromeric heterochromatin, as well as M32 and hHP1 gamma, both of which are found in euchromatin, Mapping of the region in KAP-1 required for HPI interaction showed that amino acid substitutions which abolish HP1 binding in vitro reduce KAP-1 mediated repression in vivo. We observed colocalization of KAP-1 with M31 and M32 in interphase nuclei, lending support to the biochemical evidence that M31 and M32 directly interact with KAP-1, The colocalization of KAP-1 with M31 is sometimes found in subnuclear territories of potential pericentromeric heterochromatin, whereas colocalization of KAP-1 and M32 occurs in punctate euchromatic domains throughout the nucleus. This work suggests a mechanism for the recruitment of HP1-like gene products by the KRAB-ZFP-KAP-1 complex to specific loci within the genome through formation of heterochromatin-like complexes that silence gene activity. We speculate that gene-specific repression may be a consequence of the formation of such complexes, ultimately leading to silenced genes in newly formed heterochromatic chromosomal environments.", 
"Flower pigmentation patterns were scored in 185 sense Chalcone synthase (Chs) transgenotes and 85 antisense Chs transgenotes; upon first flowering, 139 (75%) of sense transgenotes were found to be phenotypically altered, as were 70 (82%) of the antisense transgenotes. The observed patterns document the range of phenotypic variations that occur, as well as confirm and extend the finding that sense Chs constructs produce several types of morphology-based flower pigmentation patterns that antisense Chs constructs do not. Long-term monitoring for epigenetic variations in one population of 44 sense Chs transgenotes showed that 43 (98%) were capable of producing a cosuppression phenotype. The primary determinant of sense-specific patterns of cosuppression of Chs was found to be the repetitiveness and organization pattern of the transgene, not 'position effects' by, or 'readthrough' from, flanking plant DNA sequences. The degree of cosuppression observed in progeny of transgenotes carrying multiple, dispersed copies as compared to that observed with a single copy of the transgene suggests that sense cosuppression of Chs is subject to a transgene dosage effect.", 
"A transgenic gpt(+) Chinese hamster cell line (G12) was found to be susceptible to carcinogenic nickel-induced inactivation of gpt expression without mutagenesis or deletion of the transgene. Many nickel-induced 6-thioguanine-resistant variants spontaneously reverted to actively express gpt, as indicated by both reversion assays and direct enzyme measurements. Since reversion was enhanced in many of the nickel-induced variant cell lines following 24-h treatment with the demethylating agent 5-azacytidine, the involvement of DNA methylation in silencing gpt expression was suspected. This was confirmed by demonstrations of increased DNA methylation, as well as by evidence indicating condensed chromatin and heterochromatinization of the gpt integration site in 6 thioguanine-resistant cells. Upon reversion to active gpt expression, DNA methylation and condensation are lost. We propose that DNA condensation and methylation result in heterochromatinization of the gpt sequence with subsequent inheritance of the now silenced gene. This mechanism is supported by direct evidence showing that acute nickel treatment of cultured cells, and of isolated nuclei in vitro, can indeed facilitate gpt sequence-specific chromatin condensation. Epigenetic mechanisms have been implicated in the actions of some nonmutagenic carcinogens, and DNA methylation changes are now known to be important in carcinogenesis. This paper further supports the emerging theory that nickel is a human carcinogen that can alter gene expression by enhanced DNA methylation and compaction, rather than by mutagenic mechanisms.", 
"In vitro manipulation of preimplantation mammalian embryos can influence differentiation and growth at later stages of development, In the mouse, culture of embryonic stem (ES) cells affects their totipotency and may give rise to fetal abnormalities, To investigate whether this is associated with epigenetic alterations in imprinted genes, we analysed two maternally expressed genes (Igf2r, H19) and two paternally expressed genes (Igf2, U2af1-rs1) in ES cells and in completely ES cell-derived fetuses, Altered allelic methylation patterns were detected in all four genes, and these were consistently associated with allelic changes in gene expression, All the methylation changes that had arisen in the ES cells persisted on in vivo differentiation to fetal stages, Alterations included loss of methylation with biallelic expression of U2af1-rs1, maternal methylation and predominantly maternal expression of Igf2, and biallelic methylation and expression of Igf2r In many of the ES fetuses, the levels of H19 expression were strongly reduced, and this biallelic repression was associated with biallellic methylation of the H19 upstream region, Surprisingly, biallelic H19 repression was not associated with equal levels of Igf2 expression from both parental chromosomes, but rather with a strong activation of the maternal Igf2 allele, ES fetuses derived from two of the four ES lines appeared developmentally compromised, with polyhydramnios, poor mandible development and interstitial bleeding and, in chimeric fetuses, the degree of chimerism correlated with increased fetal mass, Our study establishes a model for how early embryonic epigenetic alterations in imprinted genes persist to later developmental stages, and are associated with aberrant phenotypes.", 
"Polycomb group (PcG) and trithorax group (trxG) gene products are responsible for the maintenance of repressed and active expression patterns of many developmentally important regulatory genes including the homeotic genes. In Drosophila embryos, Polycomb protein and the trxG protein GAGA factor colocalize at the Fab-7 DNA element of the bithorax complex. In transgenic lines, the Fab-7 element induces extensive silencing on a flanking GAL4-driven lacZ reporter and mini-white genes. However, a short single pulse of GAL4 during embryogenesis is sufficient to release PcG-dependent silencing from the transgene. Such an activated state of Fab-7 is mitotically inheritable through development and can be transmitted in a GAL4-independent manner to the subsequent generations through female meiosis. Thus, Fab-7 is a switchable chromosomal element, which can convey memory of epigenetically determined active and repressed chromatin states.", 
"Methylation of cytosine residues in DNA provides a mechanism of gene control. There are two classes of methyltransferase in Arabidopsis; one has a carboxy-terminal methyltransferase domain fused to an amino-terminal regulatory domain and is similar to mammalian methyltransferases. The Second class apparently lacks an amino-terminal domain and is less well conserved. Methylcytosine can occur at any cytosine residue, but it is likely that clonal transmission of methylation patterns only occurs for cytosines in strand-symmetrical sequences CpG and CpNpG. In plants, as in mammals, DNA methylation has dual roles in defense against invading DNA and transposable elements and in gene regulation. Although originally reported as having no phenotypic consequence, reduced DNA methylation disrupts normal plant development.", 
"The trilaminar kinetochore directs the segregation of chromosomes in mitosis and meiosis. Despite its importance, the molecular architecture of this structure remains poorly understood [1]. The best known component of the kinetochore plates is CENP-C, a protein that is required for kinetochore assembly [2], but whose molecular role in kinetochore structure and function is unknown. Here we have raised for the first time monospecific antisera to CENP-A [3], a 17 kD centromere-specific histone variant that is 62% identical to the carboxy-terminal domain of histone H3 [4,5] and that resembles the yeast centromeric component CSE4 [6]. We have found by simultaneous immunofluorescence with centromere antigens of known ultrastructural location that CENP-A is concentrated in the region of the inner kinetochore plate at active centromeres. Because CENP A was previously shown to co-purify with nucleosomes [7], our data suggest a specific nucleosomal substructure for the kinetochore. In human cells, these kinetochore-specific nucleosomes are enriched in alpha-satellite DNA [8]. However, the association of CENP-A with neocentromeres lacking detectable alpha-satellite DNA, and the lack of CENP-A association with alpha-satellite-rich inactive centromeres of dicentric chromosomes together suggest that CENP-A association with kinetochores is unlikely to be determined solely by DNA sequence recognition. We speculate that CENP-A binding could be a consequence of epigenetic tagging of mammalian centromeres.", 
"Nucleolar dominance is an epigenetic phenomenon that describes nucleolus formation around rRNA genes inherited from only one progenitor of an interspecific hy;brid or allopolyploid. The phenomenon is widespread, occurring in plants, insects, amphibians, and mammals, yet its molecular basis remains unclear. We have demonstrated nucleolar dominance in three allotetraploids of the plant genus Brassica. In Brassica napus, accurately initiated pre-rRNA transcripts from one progenitor, Brassica rapa are detected readily, whereas transcripts from the similar to 3000 rRNA genes inherited from the other progenitor, Brassica oleracea, are undetectable. Nuclear run-on confirmed that dominance is controlled at the level of transcription. Growth of B. napus seedlings on 5-aza-2'-deoxycytidine to inhibit cytosine methylation caused the normally silent, under-dominant B. oleracea rRNA genes to become expressed to high levels. The histone deacetylase inhibitors sodium butyrate and trichostatin A also de-epressed silent rRNA genes. These results reveal an enforcement mechanism for nucleolar dominance in which DNA methylation and histone modifications combine to regulate rRNA gene loci spanning tens of megabase pairs of DNA.", 
"Genomic imprinting is an epigenetic chromosomal modification in the gamete or zygote causing preferential expression of a specific parental allele in somatic cells of the offspring. We and others have identified three imprinted human genes on 11p15.5, IGF2 (refs 1-4), H19 (refs 1,5), and p57(KIP2) (ref. 6), although the latter gene is separated by 700 kb from the other two, and it is unclear whether there are other imprinted genes within this large interval. We previously mapped an embryonal tumour suppressor gene to this region(7), as well as five balanced germline chromosomal rearrangement breakpoints from patients with Beckwith-Wiedemann syndrome (BWS)(8), a condition characterized by prenatal overgrowth and cancer. We isolated the upstream exons of the previously identified gene KVLQT1, which causes the familial cardiac defect long-QT (LQT) syndrome. We found that KVLQT1 spans much of the interval between p57(KIP2) and IGF2, and that it is also imprinted. We demonstrated that the gene is disrupted by chromosomal chromosomal translocation in an embryonal rhabdoid tumour. Furthermore, the lack of parent-of-origin effect in LQT syndrome appears to be due to relative lack of imprinting in the affected tissue, cardiac muscle, representing a novel mechanism for variable penetrance of a human disease gene.", 
"A multisubunit cohesin complex holds sister chromatids together after DNA replication. Using chromatin immunoprecipitation, we detected cohesin association with centromeres and with discrete sites along chromosome arms from S phase until metaphase in S. cerevisiae, Short DNA sequences (130-280 bp) are sufficient to confer cohesin association. Cohesin association with a centromere depends on Mif2p, the centromere binding factor CBF3, and a centromere-specific histone variant, Cse4p. Because only active centromeres confer cohesin association with centromeric DNA, we suggest that cohesin is recruited by the same chromatin structure that confers the attachment of microtubules. Propagation of this structure might be partly epigenetic. Finally, cohesion associated with minimal centromeres is insufficient to resist the splitting force exerted by microtubules and appears to be reinforced by cohesion provided by their flanking DNA sequences.", 
"Loss of imprinting (LOI) is an epigenetic alteration of some cancers involving loss of parental origin-specific expression of imprinted genes. We observed LOI of the insulin-like growth factor-ii gene in twelve of twenty-seven informative colorectal cancer patients (44%), as well as in the matched normal colonic mucosa of the patients with LOI in their cancers, and in peripheral blood samples of four patients. Ten of eleven cancers (91%) with microsatellite instability showed LOI, compared with only two of sixteen tumors (12%) without microsatellite instability (P< 0.001). Control patients without cancer showed LOI in colonic mucosa of only two of sixteen cases (12%, P < 0.001) and two of fifteen blood samples (13%, P < 0.001). These data suggest that LOI in tumor and normal tissue identifies most colorectal cancer patients with microsatellite instability in their tumors, and that LOI may identify an important subset of the population with cancer or at risk of developing cancer.", 
"Silencing is an epigenetic form of transcriptional regulation whereby genes are heritably, but not necessarily permanently, inactivated. We have identified the Saccharomyces cerevisiae genes SAS2 and SAS3 through a screen for enhancers of sir1 epigenetic silencing defects. SAS2, SAS3 and a Schizosaccharomyces pombe homologue are closely related to several human genes, including one associated with acute myeloid leukaemia arising from the recurrent translocation t(8;16)(p11;p13) and one implicated in HIV-1 Tat interactions. All of these genes encode proteins with an atypical zinc finger and well-conserved similarities to acetyltransferases. Sequence similarities and yeast mutant phenotypes suggest that SAS-like genes function in transcriptional regulation and cell-cycle exit and reveal novel connections between transcriptional silencing and human disease.", 
"The multisubunit MSL dosage compensation complex binds to hundreds of sites along the Drosophila single male X chromosome, mediating its hypertranscription, The male X chromosome is also coated with noncoding roX RNAs. When either msl3, mle, or mof is mutant, a partial MSL complex is bound at only similar to 35 unusual sites distributed along the X, We show that two of these sites are the roX1 and roX2 genes and postulate that one of their functions is to provide entry sites for the MSL complex to recognize the X chromosome. The roX1 gene provides a nucleation site for extensive spreading of the MSL complex into flanking chromatin even when moved to an autosome. The spreading can occur in cis or in trans between paired homologs. We present a model for how the dosage compensation complex recognizes X chromatin.", 
"Defective DNA mismatch repair in human turners leads to genome-wide instability of microsatellite repeats and a molecular phenotype referred to as microsatellite instability (MSI), MSI has been reported in a variety of cancers and is a consistent feature of tumors from patients with hereditary non-polyposis colorectal cancer. Approximately 20% of cancers of the uterine endometrium, the fifth most common cancer of women world-wide, exhibit MSI. Although the frequency of MSI is higher in endometrial cancers than in any other common malignancy, the genetic basis of MSI in these tumors has remained elusive, We investigated the role that methylation of the MLH1 DNA mismatch repair gene plays in the genesis of MSI in a large series of sporadic endometrial cancers. The MLH1 promoter was methylated in 41 of 53 (77%) MSI-positive cancers investigated. In MSI-negative tumors on the other hand, there was evidence for limited methylation in only one of 11 tumors studied. Immunohistochemical investigation of a subset of the tumors revealed that methylation of the MLH1 promoter in MSI-positive tumors was associated with loss of MLH1 expression. Immunohistochemistry proved that two MSI-positive tumors lacking MLH1 methylation failed to express the MSH2 mismatch repair gene. Both of these cancers came from women who had family and medical histories suggestive of inherited cancer susceptibility. These observations suggest that epigenetic changes in the MLH1 locus account for MSI in most cases of sporadic endometrial cancers and provide additional evidence that the MSH2 gene may contribute substantially to inherited forms of endometrial cancer.", 
"Several lines of evidence strongly suggest that DNA methylation is involved in embryo development, Perhaps the most direct evidence comes from experiments with methyltransferase deficient mice. Embryos that express low levels of the maintenance methyltransferase do not develop to term and die at the 5 to 20 somite stage corresponding to the level of the enzyme. In the mouse, dramatic methylation changes have been observed during the early steps of embryo development. Most genes are subject to a process of active demethylation starting promptly after fertilization. Except for a small number of methylated CpG sites in imprinted genes the vast majority of the sites are unmethylated by the stage of cavitation (16 cell). Such genome-wide demethylation may signify an erasure of epigenetic information originating in the highly differentiated gametes. This erasure may be essential for the establishment of a pluripotent state, before specific cell lineages can be determined. The process of laying down a new developmental program involves, initially, global de novo methylation at the stage of pregastrulation followed by gene specific demethylations associated with the onset of activity of these genes. CpG islands characteristic of housekeeping genes, appear to be protected from the global de novo methylation. An exception to this rule is the hypermethylation of CpG islands in X-linked housekeeping genes on the inactive X chromosome and of specific differentially methylated CpG sites in imprinted genes. Primordial germ cells escape the global de novo methylation which takes place at the pregastrula stage and undergo a very similar de novo methylation process in the differentiated gonads (15.5-18.5 days post coitum), forming the methylation patterns which are specific to the gametes. Specific demethylations then form a terminal methylation pattern which is then clonaly inherited in the soma and erased after fertilization.", 
"During the development of a multicellular organism from a zygote, a large number of epigenetic interactions take place on every level of suborganismal organization. This raises the possibility that the system of epigenetic interactions may compensate or ''buffer'' some of the changes that occur as mutations on its lowest levels, and thus stabilize the phenotype with respect to mutations. This hypothetical phenomenon will be called ''epigenetic stability.'' Its potential importance stems from the fact that phenotypic variation with a genetic basis is an essential prerequisite for evolution. Thus, variation in epigenetic stability might profoundly affect attainable rates of evolution. While representing a systemic property of a developmental system, epigenetic stability might itself be genetically determined and thus be subject to evolutionary change, Whether or not this is the case should ideally be answered directly, that is, by experimentation. The time seals involved and our insufficient quantitative understanding of developmental pathways will probably preclude such an approach in the foreseeable future. Preliminary answers are sought here by using a biochemically motivated model of a small bur central part of a developmental pathway. Modeled are sets of transcriptional regulators that mutually regulate each other's expression and thereby form stable gene expression patterns. Such gene-expression patterns, crucially involved in determining developmental pattern formation events, are most likely subject to strong stabilizing natural selection, After long periods of stabilizing selection. the fraction of mutations causing changes in gene-expression pattern is substantially reduced in the model. Epigenetic stability has increased. This phenomenon is found for widely varying regulatory scenarios among transcription factor genes. Its discussed that only epistatic (nonlinear) gene interactions can cause such change in epigenetic stability. Evidence from paleontology, molecular evolution, development, and genetics, consistent with the existence of variation in epigenetic stability, is discussed, The relation of epigenetic stability to developmental canalization is outlined. Experimental scenarios at suggested thar may provide further evidence.", 
"X-chromosome inactivation is the process by which a cell recognizes the presence of two copies of an X chromosome early in the development of XX embryos and chooses one to be active and one to be inactive(1) Although it is commonly believed that the initiation of X inactivation is random, with an equal probability (50:50) that either X chromosome will be the inactive X in a given cell, significant variation in the proportion of cells with either X inactive is observed both in mice heterozygous for alleles at the Xce locus(2) and among normal human females in the population(3-5). Families in which multiple females demonstrate extremely skewed inactivation patterns that are otherwise quite rare in the general population are thought to reflect possible genetic influences on the X-inactivation process(5-7). Here we report a rare cytosine to guanine mutation in the XIST minimal promoter that underlies both epigenetic and functional differences between the two X chromosomes in nine females from two unrelated families. All females demonstrate preferential inactivation of the X chromosome carrying the mutation, suggesting that there is an association between alterations in the regulation of XIST expression and X-chromosome inactivation.", 
"Genomic imprinting is an epigenetic process that results in the preferential silencing of one of the two parental copies of a gene. Although the precise mechanisms by which genomic imprinting occurs are unknown, the tendency of imprinted genes to exist in chromosomal clusters suggests long-range regulation through shared regulatory elements. We characterize a 800-kb region on the distal end of mouse chromosome 7 that contains a cluster of four maternally expressed genes, H19, Mash2, Kvlqt1, and p57(Kip2), as well as two paternally expressed genes, Igf2 and Ins2, and assess the expression and imprinting of Mash2, Kvlqt1, and p57(Kip2) during development in embryonic and extraembryonic tissues. Unlike Igf2 and Ins2, which depend on H19 for their imprinting, Mash2, p57(Kip2), and Kvlqt1 are unaffected by a deletion of the H19 gene region, suggesting that these more telomeric genes are not regulated by the mechanism that controls H19, Igf2, and Ins2. Mutations in human p57(Kip2) have been implicated in Beckwith-Wiedemann syndrome, a disease that has also been associated with loss of imprinting of IGF2. We find, however, that a deletion of the gene has no effect on imprinting within the cluster. Surprisingly, the three maternally expressed genes are regulated very differently by DNA methylation; p57(Kip2) activated, Kvlqt1 is silenced, and Mash2 is unaffected in mice lacking DNA methyltransferase. We conclude that H19 is not a global regulator of imprinting on distal chromosome 7 and that the telomeric genes are imprinted by a separate mechanism(s).", 
"Skeletal myoblasts are derived from paraxial mesoderm, but how myoblasts acquire their identity is still a matter of speculation. The characterization of molecular markers and, in some cases, the analysis of mutations in the corresponding genes, has now made it possible to ask specific questions about this process. Specification of somite cell fate depends on epigenetic factors. Adjacent tissues, such as the neural tube, notochord, dorsal ectoderm and lateral mesoderm, act either positively or negatively on the different myogenic precursor populations in the somite. Candidate molecules for this complex signalling activity include sonic hedgehog and the Wnt proteins as positive signals, and BMP4 as a possible inhibitor. Although it is generally assumed that induction is required, some observations suggest that embryonic cells might have a tendency to undergo myogenesis as a 'default' pathway. By analogy with Drosophila, where the neurogenic genes affect myogenesis, the vertebrate homologues of notch and its ligands could be candidate molecules for a repression or derepression mechanism. Similar studies with cultured muscle cells also implicate other HLH factors as potential inhibitors of the MyoD family and, hence, of inappropriate myogenesis.", 
"Overexpression of chimeric transgenes in plants can trigger post-transcriptional gene silencing that is dependent on epigenetic information and physiological conditions. The current view is that unproductive RNA serves as a crucial signal for gene silencing, although direct evidence is lacking for this theory. A signalling cascade then leads to strongly enhanced turnover of all RNAs that share a critical degree of sequence similarity. The molecular details of the mechanism are, however, insufficiently understood to explain the phenomenon completely and to comprehend its biological significance.", 
" ", 
"Photocarcinogenesis represents the sum of a complex of simultaneous and sequential biochemical events that ultimately lead to the occurrence of skin cancer. These events, initiated by UV radiation of appropriate wavelength, include the formation of DNA photoproducts: DNA repair; mutation of proto-oncogenes and tumor suppressor genes; UV-production of radical species with subsequent effects on mutation and extra-nuclear function; and other epigenetic events that influence the course of carcinogenesis. The epigenetic influences may include immunological responses, antioxidant defenses, and dietary factors, This review represents an effort to provide current research results in the aforementioned areas and an attempt to meld these events into a comprehensive overview of photocarcinogenesis. If effective prevention and intervention strategies for skin cancer are to be developed, a more thorough understanding of the disease process is imperative. (C) 1997 Elsevier Science S.A.", 
" ", 
"Several proteins that contribute to epigenetic mechanisms of gene regulation contain a characteristic motif of unknown function called the SET (Suvar3-9, Enhancer-of-zeste, Trithorax) domain. We have demonstrated that SET domains mediate highly conserved interactions with a specific family of proteins that display similarity with dual-specificity phosphatases (dsPTPases). These include myotubularin, the gene of which is mutated in a subset of patients with X-linked myotubular myopathy, and Sbf1, a newly isolated homologue of myotubularin. In contrast with myotubularin, Sbf1 lacks a functional catalytic domain which dephosphorylates phospho-tyrosine and serine-containing peptides in vitro. Competitive interference of endogenous SET domain-dsPTPase interactions by forced expression of Sbf1 induced oncogenic transformation of NIH 3T3 fibroblasts and impaired the in vitro differentiation of C2 myoblast cells. We conclude that myotubularin-type phosphatases link SET-domain containing components of the epigenetic regulatory machinery with signalling pathways involved in growth and differentiation.", 
"O-6-Methylguanine DNA methyltransferase (MGMT) repairs the mutagenic and cytotoxic O-6-alkylguanine lesions produced bg environmental carcinogens and the chemotherapeutic nitrosoureas, As such, MGMT-mediated repair of O-6-alkylguanine lesions constitutes a major form of resistance to nitrosourea chemotherapy and makes control of MGMT expression of clinical interest, The variability of expression in cell lines and tissues, along with the ease with which the MGMT phenotype reverts under various conditions, suggests that MGMT is under epigenetic control, One such epigenetic mechanism, 5-methylation of cytosines, has been linked to MGMT expression, We have used an isogenic human multiple myeloma tumor cell line model composed of an MGMT-positive parent cell line, RPMI 8226/S, and its MGMT-negative variant, termed 8226/V, to study the control of MGMT expression. The loss of MGMT activity in 8226/V was found to he due to the loss of detectable MGMT gene expression, Bisulfite sequencing of the MGMT CpG island promoter revealed large increases in the levels of CpG methylation within discrete regions of the 8226/V MGMT CpG island compared to those in 8226/S, These changes in CpG methylation are associated with local heterochromatinization of the 8226/V MGMT transcription start site and provide a likely mechanism for the loss of MGMT transcription in 8226/V.", 
"Inheritance of the active and inactive states of gene expression by individual cells is crucial for development. In fission yeast, mating-type region consists of three loci called mat1, mat2, and mat3. Transcriptionally silent mat2 and mat3 loci are separated by a 15 kb interval, designated the K-region, and serve as donors of information for transcriptionally active mat1 interconversion. In a strain carrying replacement of 7.5 kb of the K-region with the ura4 gene, we discovered that ura4 silencing and efficiency of mating-type switching were covariegated and were regulated by an epigenetic mechanism. Genetic analyses demonstrated that epigenetic states were remarkably stable not only in mitosis but also in meiosis and were linked to the mating-type region. This study indicates that different epigenetic states are heritable forms of chromatin organization at the mat region.", 
"Microdeletions of a region termed the imprinting center (IC) in chromosome 15q11-q13 have been identified in several families with Prader-Willi syndrome (PWS) or Angelman syndrome who show epigenetic inheritance for this region that is consistent with a mutation in the imprinting process. The IC controls resetting of parental imprints in 15q11-q13 during gametogenesis. We have identified a larger series of cases of familial PWS, including one case with a deletion of only 7.5 kb, that narrows the PWS critical region to <4.3 kb spanning the SNRPN gene CpG island and exon 1. Identification of a strong DNase I hypersensitive site, specific for the paternal allele, and six evolutionarily conserved (human-mouse) sequences that are potential transcription-factor binding sites is consistent with this region defining the SNRPN gene promoter. These findings suggest that promoter elements at SNRPN play a key role in the initiation of imprint switching during spermatogenesis. We also identified three patients with sporadic PWS who have an imprinting mutation (IM) and no detectable mutation in the IC. An inherited 15q11-q13 mutation or a trans-factor gene mutation are unlikely; thus, the disease in these patients may arise from a developmental or stochastic failure to switch the maternal-to-paternal imprint during parental spermatogenesis. These studies allow a better understanding of a novel mechanism of human disease, since the epigenetic effect of an IM in the parental germ line determines the phenotypic effect in the patient.", 
"The Wassilewskija strain of Arabidopsis has four genes encoding the tryptophan enzyme phosphoribosylanthranilate isomerase (PAI) located at three unlinked sites, These four PAI genes are methylated over their regions of DNA homology, When PAI copy number is reduced by deletion of two tandemly arrayed genes (MePAI1-PAI4), a mutant with fluorescent, tryptophan-deficient phenotypes results, because the two remaining methylated PAI genes (MePAI2 and MePAI3) supply insufficient PAI activity, These two methylated genes can be inherited through meiosis, even when they are segregated away from each other in crosses to a strain with unmethylated PAI genes, However, the mutant phenotypes conferred by the methylated PAI genes are unstable, and mutant plants yield occasional revertant somatic sectors and progeny. Revertant lines display coordinately reduced methylation of both PAI2 and PAI3, implying that this hypomethylation acts in a concerted manner across the genome rather than at individual sites.", 
"We report that CpG island methylation, an epigenetic modification of DNA known to correlate closely with silencing of gene transcription, appears in the oestrogen receptor (ER) gene in a subpopulation of cells which increases as a direct function of age in human colonic mucosa. This same methylation change characterizes virtually all cells in art 45 colorectal tumours examined, including the earliest stages of tumour formation. ER gene expression is diminished or absent in colorectal tumours, and introduction of an exogenous ER gene in cultured colon carcinoma cells resulted in marked growth suppression. Our data suggest that methylation associated inactivation of the ER gene in ageing cotorectal mucosa could be one of the earliest events that predispose to sporadic cotorectal tumorigenesis.", 
"A very large oxidative damage rate to DNA occurs as part of normal metabolism. In each rat cell the steady-state level is estimated to be about 10(6) oxidative adducts and about 10(5) new adducts are formed daily. It is argued that this endogenous DNA damage is a major contributor to aging and the degenerative diseases of aging, such as cancer. The oxidative damage rate in mammalian species with a high metabolic rate, short life span, and high age-specific cancer rate is much higher than the rate in humans, a long-lived creature with a lower metabolic rate and a lower age-specific cancer rate. It is argued that deficiency of micronutrients, such as dietary antioxidants or folate, is a major contributor to human cancer and degenerative diseases. Understanding the role of mitogenesis in mutagenesis is critical for clarifying the mechanisms of carcinogenesis and interpreting high-dose animal cancer tests. High-dose animal cancer tests have been done mainly on synthetic industrial chemicals, yet almost all of the chemicals humans are exposed to are natural. About half of natural chemicals tested in high-dose animal cancer tests are rodent carcinogens, a finding that is consistent with the view that high-dose tests frequently increase mitogenesis rates. Animals have numerous defenses against toxins that make them very well buffered against low doses of almost all toxins, whether synthetic or natural.", 
"The expression of the H19 gene is governed by parental imprinting in mammals. H19, an unusual gene encoding an RNA with no known function, is exclusively expressed from the maternal chromosome. In mouse, it lies 90 kb downstream from the Igf2 gene, which encodes a fetal-specific growth factor, insulin-like growth factor II, and is expressed primarily from the paternally inherited chromosome. In this report we have utilized interspecific hybrid mice to identify male-specific DNA methylation of a 7- to 9-kb domain surrounding the H19 gene and its promoter. This allele-specific methylation could function as a mark to suppress transcription of the H19 paternal allele. Consistent with this proposal, the H19 promoter displayed an open chromatin conformation only on the relatively unmethylated active maternal allele. In contrast, a cell type-specific enhancer that lies outside the methylation domain is hypersensitive to restriction enzyme digestion in nuclei on both maternal and paternal chromosomes. That the allele-specific methylation domain, coupled to the two H19 enhancers, contains all the information necessary for its imprinting was tested by examining two transgenic lines containing an internally deleted H19 transgene. Both displayed paternal-specific methylation of the transgene and maternal-specific expression. Although neither line has been tested in an inbred genetic background, and therefore the action of complex modifiers cannot be formally excluded, the result suggests that the sequences necessary for the imprinting of H19 have been identified.", 
"The insulin-like growth factor-II (IGF2) and H19 genes are imprinted in mouse and human, with expression of the paternal IGF2 and maternal H19 alleles. IGF2 undergoes loss of imprinting (LOI) in most Wilms' tumours (WT). We now show that: (i) LOI of IGF2 is associated with a 80-fold down regulation of H19 expression; (ii) these changes are associated with alterations in parental-origin-specific, tissue-independent sites of DNA methylation in the H19 promoter; and (iii) loss of heterozygosity is also associated with loss of H19 expression. Thus, imprinting of a large domain of the maternal chromosome results in a reversal to a paternal epigenotype. These data also suggest an epigenetic mechanism for inactivation of H19 as a tumour suppressor gene.", 
"The recent discovery of widely distributed targeting determinants, which govern the polarized cell-surface distribution of plasma membrane proteins in epithelial cells, has significantly changed our view of how polarized cells generate functionally distinct membrane domains. Together with the surprising finding that the same determinants are recognized on both the biosynthetic and the endocytic pathways, it now appears likely that a common epigenetic code may exist that controls molecular sorting of membrane proteins in a wide variety of polarized, and perhaps even non-polarized, cell types.", 
"Inactivation of the retinoblastoma gene appears to have a fundamental role in the genesis of retinoblastoma, osteosarcoma, and other malignant tumors. The gene is generally inactivated because of loss-of-function mutations, although epigenetic phenomena, such as hypermethylation of the promoter region, could possibly have the same effect. We investigated the methylation pattern at the 5' end of the retinoblastoma gene, including its promoter region and exon 1, in DNA purified from 56 primary retinoblastomas. We found five tumors with evidence for hypermethylation, all from unilateral, simplex patients. No methylation abnormalities were detected in DNA purified from the leukocytes from these patients. It is interesting that in one of these tumors the hypermethylation was confined to one allele. There were no mutations in a 1,306-bp sequence including the hypermethylated region that might account for the allele-specific hypermethylation. We believe that the hypermethylation of the retinoblastoma gene that we found in these tumors corresponds to the allelic inactivation of the gene, and we speculate that erroneous hypermethylation without alteration of nucleotide sequence occasionally plays a role in the genesis of this cancer. If this is true, then retinoblastomas with hypermethylation might be treatable with chemotherapeutic agents that interfere with methylation of DNA.", 
"Becker muscular dystrophy (BMD) often results from in-frame mutations of the dystrophin gene that allow production of an altered but partially functional protein. To address potential structure-function relationships for the various domains of dystrophin, we examined both the dystrophin gene and protein in 68 patients with abnormal dystrophin. Eighty-six percent of BMD patients with dystrophin of altered size have deletions or duplications, and the observed sizes of dystrophin fit well with predictions based on DNA data. Deletions within the amino-terminal domain I tended to result in low levels of dystrophin and a more severe phenotype. The phenotypes of patients with deletions or duplications in the central rod domain were more variable. This region can be divided into three portions based on differences in clinical presentations of patients. Deletions around exons 4553 were most common and generally caused typical BMD; however, phenotypic variability among patients with similar mutations suggests that epigenetic and/or environmental factors play an important role in determining the clinical progression. In contrast, deletions or duplications in the proximal portion of this domain tended to cause severe cramps and myalgia. Finally, loss of the middle of this region probably causes a very mild phenotype, as only one such patient was found and his only symptom was elevated serum creatine phosphokinase levels.", 
"Massive cell migration, proliferation, phenotypic differentiation, and enhanced biosynthetic activities characterize the sites of wound healing and fibrosis. Regulation of cellular functions by extracellular matrix, which consists of a dynamic assemblage of a variety of interacting molecules capable of reorganization in response to endogenous and exogenous stimuli, represents a fundamental epigenetic mechanism regulating cellular behavior and phenotype. Interactions of the individual components of extracellular matrix with specific cell surface molecules, integrin receptors, and proteoglycans initiate a cascade of signal transduction leading to varied short-term or persistent cellular responses. Extracellular matrix also serves as an important reservoir of cytokines and growth factors, thus modulating the action of a host of potent biological response modifiers by their selective, local accumulation and release. Currently known mechanisms by which extracellular matrix modulates different facets of the process of tissue remodeling after injury, which culminate either in normal wound repair or fibrosis, are discussed.", 
"To test the potential role of H19 as a tumour suppressor gene we have examined its expression and DNA methylation in Wilms' tumours (WTs). In most WTs (18/25), H19 RNA was reduced at least 20-fold from fetal kidney levels. Of the expression-negative tumours ten retained 11p15.5 heterozygosity: in nine of these, H19 DNA was biallelically hypermethylated and in two cases hypermethylation locally restricted to H19 sequences was also present in the non-neoplastic kidney parenchyma. IGF2 mRNA was expressed in most but not all WTs and expression patterns were consistent with IGF2/H19 enhancer competition without obligate inverse coupling. These observations implicate genetic and epigenetic inactivation of H19 in Wilms' tumorigenesis.", 
"Segmentation genes provide the signals for the activation and regulation of homeotic genes in Drosophila but cannot maintain the resulting pattern of expression because their activity ceases halfway through embryogenesis. Maintenance of the pattern is due to the Polycomb group of genes (Pc-G) and the trithorax group of genes (trx-G), responsible for the persistence of the active or repressed state of homeotic genes. We have identified a regulatory element in the Ubx gene that responds to Pc-G and trx-G genes. Transposons carrying this element create new binding sites for Pc-G products in the polytene chromosomes. This Pc-G maintenance element (PRE), establishes a repressive complex that keeps enhancers repressed in cells in which they were originally repressed and maintains this state through many cell divisions. The trx-G products stimulate the expression of enhancers in cells in which they were originally active. This mechanism is responsible for the correct regulation of imaginal disc enhancers, which lack themselves antero-posterior positional information. The PRE also causes severe variegation of the mini-white gene present in the transposon, a phenomenon very similar to heterochromatic position-effect variegation. The significance of this mechanism for homeotic gene regulation is discussed.", 
"Chromatin structure at Schizosaccharomyces pombe centromeres is unusual. The insertion of the ura4 gene within these centromeres resulted in genetically identical cells mosaic for its expression. Placement of the ade6 gene within cen1 or cen3 resulted in red-white sectored colonies, demonstrating the instability of gene expression. The occurrence of pink colonies implied that intermediate levels of repression were established. Repression of both genes within centromeres was temperature sensitive. The chromatin structure of the ura4 gene at centromeres was altered, suggesting that the unusual chromatin encroaches into the gene and inhibits normal expression. These repressive effects at S. pombe centromeres resemble the classical phenomenon of position effect variegation imposed by Drosophila heterochromatin on nearby genes. However, since the epigenetic states can be set at intermediate levels of expression, a purely euchromatin-heterochromatin dichotomy does not apply. A model for the epigenetic regulation of genes placed within S. pombe centromeres is presented.", 
"The mouse insulin-like growth factor II (Igf2) gene, which is located on distal chromosome 7 (Chr7), has been shown previously to undergo tissue-specific parental imprinting. This imprinting results in expression of Igf2 from the paternally inherited chromosome and repression of the maternally inherited allele in most tissues of the developing embryo. We are using embryos with the maternal duplication and paternal deficiency of distal Chr7 to characterize the mechanism that underlies repression of the maternal allele. We show that the chromatin of the 5' region of the repressed Igf2 allele is potentially active for transcription rather than heterochromatic. In particular, a CpG island that comprises one of the two strong promoters is unmethylated at both parental alleles, and DNase I hypersensitive sites in and around the strong promoters are consistently present on both parental chromosomes. In agreement with the chromatin state, primary transcripts from the maternal Igf2 allele have been detected at low but significant levels. These findings differ from observations in other instances of imprinting, namely, X-chromosome inactivation and transgene imprinting in mice. Although no parent-specific differences were detected in either DNA methylation or sensitivity to nucleases at these promoters, we have observed parental methylation differences in a region several kilobases upstream of the first exon. The differential activity of the parental Igf2 alleles could be achieved through epigenetic modifications situated outside the promoters or by subtle and yet unidentified modifications at the promoters.", 
"The Insulin-like growth factor 2 (Igf2) and H19 genes are reciprocally imprinted and closely linked. Igf2 encodes a fetal growth-factor and is predominantly expressed from the paternal allele, while H19 is expressed from the maternal allele and encodes a transcript which may downregulate cellular proliferation. One of the epigenetic modifications thought to be involved in parental imprinting is DNA methylation. Here we analyse methylation in two regions of the Igf2 gene, one approx. 3 kb upstream of the gene and one in the 3' part of the gene. Both regions are more methylated on the expressed paternal chromosome. Genomic sequencing of individual chromosomes in the first region shows this parent-specific methylation to be highly mosaic; interestingly, individual sperm chromosomes carry different methylation patterns into the egg. In the more 3' region, which is fully methylated in sperm, the level of methylation on the paternal allele is highly tissue-specific and is correlated with expression of the gene in fetal tissues. Hence, the paternal allele is highly methylated in fetal liver (high expression) but is undermethylated in fetal brain (virtually no expression). Adult choroid plexus, a brain tissue in which Igf2 is expressed from both alleles and H19 is not expressed, represents an apparent loss of imprinting. Here, both Igf2 and H19 adopt a paternal type methylation pattern on both parental chromosomes. Analysis of early-passage androgenetic and parthenogenetic embryonic stem (ES) cells shows that the methylation patterns of Igf2 and H19 on maternal and paternal chromosomes are very similar. Androgenetic and parthenogenetic teratomas derived from these ES cells show the appropriate paternal and maternal patterns, respectively, of allelic methylation in both genes. Our results suggest that allelic methylation patterns in Igf2 and H19 arise early in embryogenesis and change progressively during development. Some of these developmental changes are apparently under tissue-specific control.", 
"In several plant systems expression of structurally intact genes may be silenced epigenetically when a transgenic construct increases the copy number of DNA sequences. Here we report epigenetic silencing in Arabidopsis lines containing transgenic inserts of defined genetic structure, all at the same genomic locus. These comprise an allelic series that includes a single copy of the primary insert, which carries repeated drug resistance transgenes, and a set of its derivatives, which as a result of recombination within the insert carry different numbers and alleles of resistance genes. Although the drug resistance genes remained intact, both the primary and some recombinant lines nevertheless segregated many progeny that were partly or fully drug-sensitive because of silencing. As in other systems silencing was reversible, and correlated with decreased steady-state mRNA and increased DNA methylation. Each different number and combination of genes, on the same or different (i.e., homologous) chromosomes, conditioned its own idiosyncratic segregation pattern. Strikingly, lines with a single gene segregated only a few slightly drug-sensitive progeny whereas multi-gene lines segregated many highly sensitive progeny, indicating dependence of silencing at this locus on repeated sequences. This argues strongly against explanations based on antisense RNA, but is consistent with explanations based on ectopic DNA pairing. One possibility is that silencing reflects the interaction of paired homologous DNA with flanking heterologous DNA, which induces condensation of chromatin into a non-transcribable state.", 
"Modifiers of position-effect-variegation in Drosophila encode proteins that are thought to modify chromatin, rendering it heritably changed in its expressibility. In an attempt to identify similar modifier genes in other species we have utilized a known sequence homology, termed chromo box, between a suppressor of position-effect-variegation, Heterochromatin protein 1 (HP1), and a repressor of homeotic genes, Polycomb (Pc). A PCR generated probe encompassing the HP1 chromo box was used to clone full-length murine cDNAs that contain conserved chromo box motifs. Sequence comparisons, in situ hybridization experiments, and RNA Northern blot analysis suggest that the murine and human sequences presented in this report are homologues of the Drosophila HP1 gene. Chromo box sequences can also be detected in other animal species, and in plants, predicting a strongly conserved structural role for the peptide encoded by this sequence. We propose that epigenetic (yet heritable) changes in gene expressibility, characteristic of chromosomal imprinting phenomena, can largely be explained by the action of such modifier genes. The evolutionary conservation of the chromo box motif now enables the isolation and study of putative modifier genes in those animal and plant species where chromosomal imprinting has been described.", 
" ", 
" ", 
"The molecular mechanisms introducing epigenetic modifications that lead to differential silencing of some autosomal alleles depending on their parental legacy are still largely unknown, but recent results from studies of endogenously imprinted genes and particular transgenes make DNA methylation a strong candidate. At the same time, these results have raised new questions about the details of the imprinting process.", 
"Previous work has shown that two homologous, unlinked transgene loci can interact in plant nuclei, leading to non-reciprocal trans-inactivation and methylation of genes at one locus. Here, we report the structure and methylation of different transgene loci that contain the same construct but are variably able to inactivate and methylate a partially homologous, unlinked target locus. Silencing loci comprised multiple, methylated copies of the transgene construct, whereas a non-silencing locus contained a single, unmethylated copy. The correspondence between strength of silencing activity and copy number/degree of methylation was further demonstrated by producing novel alleles of a strong silencing locus: reducing the transgene copy number and methylation within this silencing locus decreased its ability to inactivate the target locus. The strong silencing locus, which was located close to a telomere, trans-inactivated various structural variants of the original target construct, regardless of their location in the genome. This suggests that the silencing locus can scan the entire genome for homologous regions, a process possibly aided by its telomeric location. Our data support the idea that epistatic trans-inactivation of unlinked, homologous transgenes in plants results from a pre-existing epigenetic difference between transgene loci, which is subsequently equalized by ''epigene conversion'' involving DNA-DNA pairing.", 
" ", 
"Major epigenetic modifications apparently occur during early development in the mouse. The factors that induce such modifications are complex and may involve the various components of a zygote. We have started to explore whether changes in the nucleocytoplasmic composition brought about by micromanipulation can induce phenotypic effects through epigenetic modifications. Nucleocytoplasmic hybrids were therefore prepared by transplanting a female pronucleus into a recipient egg from a different genotype. As a result, the maternal genome was of a different genetic background as compared with the egg cytoplasm. Specifically, experimental zygotes had cytoplasm from the inbred strain C57BL/6, a maternal genome from DBA/2, and a paternal genome from C57BL/6 (termed BDB hybrids). The mirror-image combination, termed DBD, was also made. The reconstituted zygotes were transferred to recipients and allowed to develop to term. Mice born from manipulated zygotes showed transcriptional repression and DNA methylation of major urinary protein genes in their liver, as well as growth deficiency resulting in reduced adult body weight. No altered phenotype was observed in controls in which the maternal pronucleus was simply transplanted back into another zygote of the same genetic background. These results clearly demonstrate phenotypic as well as molecular effects on DNA methylation and expression of at least one gene. Phenotype was therefore no longer predicted by genotype as a result of epigenetic modifications in experimental embryos. What precisely triggers the phenotypic and epigenetic changes is unknown, but presumably, nucleocytoplasmic interactions in hybrid zygotes may be partly responsible.", 
"The DNA of eukaryotic cells is organized in a complex with proteins, either as interphase chromatin or mitotic chromosomes. Nucleosomes, the structural subunits of chromatin, have long been considered as static structures, incompatible with processes occurring in chromatin. During the past few years it has become evident that the histone part of the nucleosome has important regulatory functions. Some of these functions are mediated by the N-terminal core histone domains which contain sites for posttranslational modifications, among them lysine residues for reversible acetylation. Recent results indicate that acetylation and deacetylation of N-terminal lysines of nucleosomal core histones represent a means of molecular communication between chromatin and the cellular signal transduction network, resulting in heritable epigenetic information. Data on enzymes involved in acetylation and the pattern of acetylated lysine sites on chromosomes, as well as genetic data on yeast transcriptional repression, suggest that acetylation may lead to structural transitions as well as specific signalling within distinct chromatin domains.", 
" ", 
"Messenger RNA and methylation levels of four imprinted genes, H19, Igf2r, Igf-2 and Snrpn were examined by northern and Southern blotting in mouse parthenogenetic, androgenetic and normal or wild-type embryonic stem cell lines during their differentiation in vitro as embryoid bodies. Zn most instances, mRNA levels in parthenogenetic and androgenetic embryoid bodies differed from wild type as expected from previously determined patterns of monoallelic expression in midgestation embryos and at later stages of development. These findings implicate aberrant mRNA levels of these genes in the abnormal development of parthenogenetic and androgenetic embryos and chimeras. Whereas complete silence of one of the parental alleles has previously been observed in vivo, we detected some mRNA in the corresponding embryonic stem cell line. This 'leakage' phenomenon could be explained by partial erasure, bypass or override of imprints, or could represent the actual activity status at very early stages of development. The mRNA levels of H19, Igf2r and Igf-2 and the degree of methylation at specific associated sequences were correlated according to previous studies in embryos, and thereby are consistent with suggestions that the methylation might play a role in controlling transcription of these genes. Paternal-specific methylation of the H19 promoter region is absent in sperm, yet we observed its presence in undifferentiated androgenetic embryonic stem cells, or before the potential expression phase of this gene in embryoid bodies. As such methylation is likely to invoke a repressive effect, this finding raises the possibility that it is part of the imprinting mechanism of H19, taking the form of a secondary imprint or postfertilization epigenetic modification necessary for repression of the paternal allele.", 
"The SNRPN gene encodes a small nuclear ribonucleoprotein subunit, SmN, thought to be involved in splicing of pre-mRNA. A closely related protein, SmB/B', is constitutively expressed in all tissues except the brain, where SmN is predominantly expressed. The mouse homolog of the SNRPN gene has been shown to be functionally imprinted in mouse brain, being expressed only from the paternally derived chromosome. SNRPN has been mapped to human chromosome 15q11-q13 within the shortest region of deletion overlap for the Prader-Willi syndrome. We have now demonstrated functional imprinting of the human SNRPN gene using reverse transcription followed by the polymerase chain reaction (RT-PCR). No expression was observed in cultured skin fibroblasts of Prader-Willi patients, but was found in all Angelman patients and normal controls examined. We have also demonstrated a parent-specific DNA methylation imprint within intron 5 of the SNRPN gene, which suggests an epigenetic mechanism by which parent-specific expression of this gene might be inherited. Our findings indicate that SNRPN is expressed only from the paternally derived chromosome 15 in humans and therefore may fullfill one major criterion for being involved in the pathogenesis of the Prader-Willi syndrome.", 
"Development in mammals is influenced by genome imprinting which results in differences in the expression of some homologous maternal and paternal alleles. This process, initiated in the germline, can continue following fertilization with interactions between oocyte cytoplasmic factors and the parental genomes involving modifier genes. Further epigenetic modifications may follow to render the 'imprints' heritable through subsequent cell divisions during development. Imprinting of genes can be critical for their dosage affecting embryonic growth, cell proliferation and differentiation. The cumulative effects of all the imprinted genes are observed in androgenones (AG) and parthenogenones (PG), which reveal complementary phenotypes with respect to embryonic and extraembryonic tissues. The presence of PG cells in chimeras causes growth retardation, while that of AG cells enhanced growth. AG cells apparently have a higher cell proliferation rate and, unlike PG cells, are less prone to selective elimination. However, the PG germ cells are exempt from cell selection. In chimeras, PG cells are more likely to be found in ectodermal derivatives such as epidermis and brain in contrast to AG cells which make pronounced contributions to many mesodermal derivatives such as muscle, kidney, dermis and skeleton. The presence of androgenetic cells in chimeras also results in the disproportionate elongation of the anterior-posterior axis and sometimes in the abnormal development of skeletal elements along the axis. Genetic studies highlight the influence of subsets of imprinted genes, and identify those that are critical for development.", 
"Zea mays transformants produced by particle bombardment of embryogenic suspension culture cells of the genotype A188 x B73 and selected on kanamycin or bialaphos were characterized with respect to transgene integration, expression, and inheritance. Selection on bialaphos, mediated by the bar or par genes, was more efficient than selection on kanamycin, mediated by the nptII gene. Most transformants contained multicopy, single locus, transgene insertion events. A transgene expression cassette was more likely to be rearranged if expression of that gene was not selected for during callus growth. Not all plants regenerated from calli representing single transformation events expressed the transgenes, and a nonselectable gene (uidA) was expressed in fewer plants than was the selectable transgene. Mendelian inheritance of transgenes consistent with transgene insertion at a single locus was observed for approximately two thirds of the transformants assessed. Transgene expression was typically, but not always, predictable in progeny plants - transgene silencing, as well as poor transgene transmission to progeny, was observed in some plant lines in which the parent plants had expressed the transgene.", 
"Although Angelman (AS) and Prader-Willi (PWS) syndromes are human genetic disorders with distinctly different developmental and neurobehavioural phenotypes, they both have abnormalities in inheritance of chromosome 15q11-q13. Whether AS or PWS arises depends on the parental origin of a deletion or uniparental disomy (the inheritance of 2 copies of a genetic locus from only one parent) for 15q11-q13. Normal development requires a genetic contribution for this genetic region from both a male and a female parent. The dependence on parental origin implies that genes in human 15q11-q13 have distinct functions depending upon epigenetic, parent-of-origin differences, known as genomic imprinting. Here, I review the role of uniparental disomy and genomic imprinting in the pathogenesis of AS and PWS, and briefly discuss phenotype-genotype correlations using candidate genes and mouse models, in particular for hypopigmentation.", 
"Because development is epigenetic, diverse aspects of morphology are integrated during ontogeny. Using the method of thin-plate splines, and the decomposition of these splines by their principal warps, we examine the ontogeny of integrated features of skull growth of the cotton rat, Sigmodon fulviventer as observed in landmark locations in the ventral view. Postnatal growth of the skull in Sigmodon is not adequately described by the familiar contrast between relatively rapid facial elongation and slow, precocial growth of the cranial base. No developmental units corresponding to facial skull and cranial base emerge from analysis of geometric shape change. Rather, skull growth is both more integrated and more complex, exhibiting both skull-wide integration and locally individualized regions. Like skull shape, integration has an ontogeny; different regions of the skull can be partitioned into developmentally individualized parts in different ways at different ages. The effective count of individualized parts decreases substantially before weaning occurs, suggesting that the integration required by the functionally demanding activity of chewing gradually develops before the functional transition occurs. Our description of skull growth and integration does not depend upon arbitrary a priori choices about what to measure; rather, we base our decomposition of the whole into parts upon results of the data analysis. Our approach complicates the study of heterochrony, but, because it expresses the spatiotemporal organization of ontogeny, it enables the study of heterotopy.", 
"A novel epigenetic mechanism that can affect minichromosome centromere function in vivo has been identified in S. pombe. This epigenetic system can result in the conversion of a nonfunctional centromere to a functional one without changes in the content, structural arrangement, or chemical modification state of the minichromosomal DNA. The conversion from a centromere-inactive to an active state, which is evident with minichromosomes carrying abbreviated centromeric DNA constructions, occurs with a relatively high frequency during mitotic cell divisions and readily provides an in vivo essay for proper centromere formation. The centromere-targeted epigenetic system supports a model for centromere function that involves specific de novo folding of centromeric components into a higher order chromatin structure.", 
"During development, patterns of differential gene expression, defining determined states of cells, need to be maintained over many cell generations. In Drosophila, genetic and molecular analyses led to the discovery of a set of proteins which seem to exert such a memory function by using epigenetic mechanisms. Recent experiments demonstrate that, in particular, the heritable inactivation of regulatory genes relies on stable changes in the higher-order constitution of chromatin.", 
"The ability of chemicals with tumor-promoting or tumor-inhibiting activity to modulate gap junctional intercellular communication is reviewed. The two most extensively used types of assays for screening tests are (1) metabolic cooperation assays involving exchange between cells of precursors of nucleic acid synthesis and (2) dye-transfer assays that measure exchange of fluorescent dye from loaded cells to adjacent cells. About 300 substances of different biological activities have been studied using various assays. For tumor promoters/epigenetic carcinogens, metabolic cooperation assays have a sensitivity of 62% and dye-transfer assays 60%. Thirty percent of DNA-reactive carcinogens also possess the ability to uncouple cells. The complete estimation of the predictive power of these assays could not be made because the majority of the substances studied for intercellular communication effects in vitro have not yet been studied for promoting activity in vivo. Both metabolic cooperation assays and dye transfer assays respond well to the following classes of substances: phorbol esters, organochlorine pesticides, polybrominated biphenyls, promoters for urinary bladder, some biological toxins, peroxisome proliferators, and some complex mixtures. Results of in vitro assays for such tumor promoters/nongenotoxic carcinogens, such as some bile acids, some peroxides, alkanes, some hormones, mineral dusts, ascorbic acid, okadaic acid, and benz(e)pyrene, do not correlate with the data of in vivo two-stage or complete carcinogenesis. Enhancement of intercellular communication was found for 18 chemicals. Among these, cAMP, retinoids, and carotenoids have demonstrated inhibition of carcinogenesis. We examine a number of factors that are important for routine screening, including the requirement for biotransformation for some agents to exert effects on gap junctions. We also discuss the mechanisms of tumor promoter and tumor inhibitor effects on gap junctional permeability, including influences of protein kinase activation, changes in proton and Ca2+ intracellular concentrations, and effects of oxy radical production.", 
"Ascobolus immersus artificial gene repeats were shown previously to be subject premeiotically to both cytosine methylation and inactivation. We studied sexual progenies of strains harbouring two wild type copies of the endogenous met2 gene lying either in tandem array or at ectopic unlinked positions, by (i) investigating the methylation status, (ii) searching for mutations and (iii) analysing the inheritance of inactivation both in mitotic and sexual offspring. 100% of the 'tandem' progeny and 64% of the 'ectopic' progeny had methylated repeats and displayed gene inactivation. Similar methylation patterns involving all or most of the cytosine residues within the repeats were observed in both arrangements. The inactivated met2 copies were totally devoid of mutation, as deduced from: (i) extensive restriction site analysis and DNA sequencing; (ii) the finding that all the Met- derivatives tested reverted to prototrophy in selective conditions; and (iii) the finding that an inactivated copy of met2 stripped of its methylation through amplification in Escherichia coli regained activity when reintroduced in A. immersus. In the absence of selection, gene silencing and methylation were faithfully maintained through mitotic divisions and through five successive sexual cycles. Altogether, these data show the epimutational nature of this methylation induced premeiotically (MIP) process.", 
"Transposable elements from the Robertson's Mutator family are highly active insertional mutagens in maize. However, mutations caused by the insertion of responder (non-autonomous) elements frequently depend on the presence of active regulator (autonomous) elements for their phenotypic effects. The hcf106::Mu1 mutation has been previously shown to depend on Mu activity in this way. The dominant Lesion-mimic 28 mutation also requires Mu activity for its phenotypic effects. We have used double mutants to show that the loss of Mu activity results in the coordinate suppression of both mutant phenotypes. This loss can occur somatically resulting in large clones of cells that have a wild-type phenotype. Autonomous and non-autonomous Mutator elements within these clones are insensitive to digestion with methylation-sensitive enzymes, suggesting extensive methylation of CG and non-CC cytosine residues. Our data are consistent with the sectors being caused by the cycling of MuDR regulatory elements between active and inactive phases. The pattern of sectors suggests that they are clonal and that they are derived from the apical cells of the vegetative shoot meristem. We propose that these cells are more likely to undergo epigenetic loss of Mu activity because of their longer cell division cycle during shoot growth. Coordinate suppression of unlinked mutations can be used to perform mosaic analysis in maize.", 
"The neural crest is a fascinating structure of the vertebrate embryo; its ontogeny includes a transient period during which its component cells undergo an epithelio-mesenchymal transition and become migratory. This phase was shown recently to be controlled by the 'Slug' gene which belongs to the 'Snail' family of Drosophila transcription factors. After homing to specific sites in the embryo, the crest-derived cells produce a large variety of phenotypes. Recent advances have shown that during migration most crest cells exhibit various degrees of pluripotentiality, some being already committed to a single and definite fate. Moreover, several lines of evidence point to the existence of totipotent stem cells in the neural crest, the progeny of which become progressively diversified through a combination of intrinsic and extrinsic influences. The latter have been documented by the disruption of several neurotrophin genes, which results in severe deficiencies of selected subsets of neural crest derivatives. The neural crest has also been shown to play an important role in the development of the vertebrate head and hypobranchial region. The genetic control of this process depends on the. activity of developmental genes, among which the vertebrate Hox genes are essential, particularly at the rhombencephalic level.", 
"The ability of simian virus 40-encoded large T antigen to disrupt the growth control of a variety of cell types is related to its ability to interfere with certain cellular proteins, such as p53 and the retinoblastoma susceptibility gene product (pRB). We have used wild-type and mutant forms of T antigen in transgenic mice to dissect the roles of pRB, p53, and other cellular proteins in tumorigenesis of different cell types. In this study, using a cell-specific promoter to target expression specifically to brain epithelium (the choroid plexus) and to B and T lymphoid cells, we characterize the tumorigenic capacity of a T-antigen fragment that comprises only the amino-terminal 121 residues. This fragment (dl1137) retains the ability to interact with pRB and p107 but lacks the p53-binding domain. While loss of the p53-binding region results in loss of the capacity to induce lymphoid abnormalities, dl1137 retains the ability to induce choroid plexus tumors that are histologically indistinguishable from those induced by wild-type T antigen. Tumors induced by dl1137 develop much more slowly, however, reaching an end point at around 8 months of age rather than at 1 to 2 months. Analysis of tumor progression indicates that tumor induction by dl1137 does not require secondary genetic or epigenetic events. Rather, the tumor growth rate is significantly slowed, indicating that the T-antigen C-terminal region contributes to tumor progression in this cell type. In contrast, the pRB-binding region appears essential for tumorigenesis as mutation of residue 107, known to disrupt pRB and p107 binding to wild-type T antigen, abolishes the ability of the dl1137 protein to induce growth abnormalities in the brain.", 
"In this study, we used the ADE2 gene in a colony color assay to monitor transcription from the normally silent HMR mating-type locus in Saccharomyces cerevisiae. This sensitive assay reveals that some previously identified cis- and trans-acting mutations destabilize silencing, causing genetically identical cells to switch between repressed and derepressed transcriptional states. Deletion of the autonomously replicating sequence (ARS) consensus element at the HMR-E silencer or mutation of the silencer binding protein RAP1 (rap1s) results in the presence of large sectors within individual colonies of both repressed (Ade-, pink) and derepressed (Ade+, white) cells. These results suggest that both the ARS consensus element and the RA-P1 protein play a role in the establishment of repression at HMR. In diploid cells, the two copies of HMR appear to behave identically, suggesting that the switching event, though apparently stochastic, reflects some property of the cell rather than a specific event at each HMR locus. In the ADE2 assay system, silencing depends completely upon the function of the SIR genes, known trans-acting regulators of the silent loci, and is sensitive to the gene dosage of two SIR genes, SIR1 and SIR4. Using the ADE2 colony color assay in a genetic screen for suppressors of rap1s, silencer ARS element deletion double mutants, we have identified a large number of genes that may affect the establishment of repression at the HMR silent mating-type locus.", 
"We analysed de novo DNA methylation occurring in plants obtained from the transgenic petunia line R101-17. This line contains one copy of the maize Al gene that leads to the production of brick-red pelargonidin pigment in the flowers. Due to its integration into an unmethylated genomic region the A1 transgene is hypomethylated and transcriptionally active. Several epigenetic variants of line 17 were selected that exhibit characteristic and somatically stable pigmentation patterns, displaying fully coloured, marbled or colourless flowers. Analysis of the DNA methylation patterns revealed that the decrease in pigmentation among the epigenetic variants was correlated with an increase in methylation, specifically of the transgene DNA. No change in methylation of the hypomethylated integration region could be detected. A similar increase in methylation, specifically in the transgene region, was also observed among progeny of R101-17del, a deletion derivative of R101-17 that no longer produces pelargonidin pigments due to a deletion in the Al coding region. Again de novo methylation is specifically directed to the transgene, while the hypomethylated character of neighbouring regions is not affected. Possible mechanisms for transgene-specific methylation and its consequences for long-term use of transgenic material are discussed.", 
"Plant cells growing in an artificial culture environment make numerous genetic mistakes. These alterations are manifested as increased frequencies of single-gene mutations, chromosome breakages, transposable element activations, quantitative trait variations, and modifications of normal DNA methylation patterns. Evidence is presented that indicates a high frequency of DNA hypomethylation as the result of the tissue culture process. Fifteen percent of the methylation changes appear to have been homozygous in the original regenerated plants. A hypothesis is advanced that relates DNA methylation to the variety of genetic alterations found among maize tissue culture regenerants and their progenies. The epigenetic nature of DNA methylation raises questions concerning the stability of tissue culture-induced changes in self-pollinations and crosses.", 
"In this essay review I describe the relationship between plant organization, variation, and evolution. The hypothetical optimal design of an organism living in a given environment must be specified to test whether evolution is adaptive. In a variable environment this design should include a system for plasticity. Plasticity in plants is made possible by modular growth. In modular growth new parts can constantly be added to a body, resulting in a population-like structure of the organism. The developmental basis for modular growth is an open-ended ontogeny in which the formation of organs is not controlled by patterns of cell divisions and migrations and where there is no separation into germ-line and soma. Epigenetic pattern formation, late differentiation, and loose integration of the individual arising from a zygote (genet) are characteristic for plants. Because the phenotype is the result of an interaction between genotype and environment, the contributions of the two cannot be separated. Nevertheless, with appropriate experimental procedures it is possible to separate the variation among phenotypes into variation among genotypes, variation among environments, and covariation of genotypes and environments. Several genotypes must be grown in several environments. If the environments can be put on a scale, as in gradient experiments, reaction norms of genotypes may be quantified. Local specialization of genotypes in a heterogeneous environment may be recognized by transplant experiments. New results from both types of experiments indicate that the 'general-purpose genotype' is more common than originally thought and suggest that phenotypic plasticity in plants has relatively low costs. However, the critical experiments to measure these costs have not yet been published. The genetic analysis of phenotypic variation and the use of phenotypic characters for reconstructing phylogenies is hampered by the particular plant features mentioned above. Environment-induced phenotypic effects can be persistent and can be transmitted to clonal or even sexual progeny. Gene flow among taxa via sexual and possibly other mechanisms together with the loose integration of the individual make it difficult to recognize phylogenies for systematic groups in plants. Some of these problems are illustrated with species from the genera Solidago and Carex.", 
"Mating-type genes resident in the silent cassette HML at the left arm of chromosome III are repressed by the action of four SIR gene products, mediated independently through two cis-acting sites, termed the E and I silencers. We have found that in the absence of the I silencer, deletion of any one of three distinct elements within E yields partial derepression of the mating-type genes resident at HML, whereas deletion of any two yields full derepression. These elements correspond to a binding site for the abundant DNA-binding protein RAP1, an autonomous replicating sequence (ARS), and an as yet undistinguished region. From detailed deletion analysis of the E site we conclude that the ARS element contributes to silencer function in a capacity distinct from its role as an initiator of DNA replication. In addition, we find that strains deleted for any one of these elements comprise two genetically identical but phenotypically distinct types of cells: Those with HML apparently fully derepressed, and those with HML apparently completely repressed. These results reinforce the notion that epigenetic inheritance is an intrinsic characteristic of silencer action.", 
"Two instances of genetic transmission of spontaneous epimutation of the maize P-rr gene were identified. Transmission gave rise to two similar, moderately stable alleles, designated P-pr-1 and P-pr-2, that exhibited Mendelian behavior. Both isolates of P-pr conditioned a variable and variegated phenotype, unlike the uniform pigmentation conditioned by P-rr. Extensive genomic analysis failed to reveal insertions, deletions or restriction site polymorphisms between the new allele and its progenitor. However, methylation of the P gene was increased in P-pr relative to P-rr, and was greatly reduced (though not lost) in a revertant to uniform pigmentation. Variability in pigmentation conditioned by P-pr correlated with variability in transcript levels of the P gene, and both correlated inversely with variability in its methylation. Part of the variability in methylation could be accounted for by a developmental decrease in methylation in all tissues of plants carrying P-pr. We hypothesize that the variegated phenotype results from a general epigenetic pathway which causes a progressive decrease in methylation and increase in expression potential of the P gene as a function of cell divisions in each meristem of the plant. This renders all tissues chimeric for a functional gene; chimerism is visualized as variegation only in pericarp due to the tissue specificity of P gene expression. Therefore, this allele that originates from epimutation may exemplify an epigenetic mechanism for variegation in maize.", 
"Previous work has shown that two unlinked, partially homologous transgene loci can interact in plant nuclei, leading to reversible methylation and inactivation of one transgene locus in the presence of the second. To study whether the chromosomal location of a transgene influences its susceptibility to trans-inactivation, we retransformed four transgenic lines, which contained the same construct (H) integrated in different chromosomal locations, with a second, partially homologous construct (K). At least 50 double transformants (DTs) were regenerated from each single transformant (ST) and screened for inactivation of markers [chloramphenicol acetyltransferase (CAT); hygromycin resistance (HYG(R))] at the resident H locus. For two STs, H locus markers were inactivated in less than 1% of the DTs, suggesting that, at these integration sites, H was relatively resistant to trans-inactivation. In contrast, the other two STs appeared to be more sensitive to trans-inactivation: 4-10% of the DTs were CAT(-) and/or Hyg(s). Inactivation of H locus markers could be attributed to two distinct phenomena: 1. Regeneration from cells containing different epigenetic states of H, in which either both, one or none of the H alleles was active. This instability in the expression of the H locus, which was independent of K, was more pronounced in the homozygous state, and was associated with cellular mosaicism of expression and methylation. 2. The presence of an unlinked K locus could weaken the Hyg(R) phenotype by transcriptional inactivation and increased methylation of the hph gene at the H locus. These results indicated that a susceptible transgene locus is inherently unstable and partially methylated, and that these characteristics are exacerbated when the locus is homozygous for the transgene and/or when an unlinked homologous transgene is present.", 
"An illustrative list is presented of human and animal studies which each point to the existence of a third source, in addition to genetic and environmental factors, underlying phenotypic differences in development. It is argued that this third source may consist of nonlinear epigenetic processes that can create variability at all phenotypical-somatic and behavioral-levels. In a quantitative genetic analysis with human subjects, these processes are confounded with within-family environmental influences. A preliminary model to quantify these influences is introduced.", 
"Analysis of the developmental capacities of androgenetic and gynogenetic mouse embryos (bearing two paternal or two maternal pronuclei, respectively) revealed a defect in blastocyst formation of androgenetic, but not gynogenetic, embryos that was a function of the maternal genotype. Androgenetic embryos constructed using fertilized eggs from C57BL/6 or (B6D2)F1 mice developed to the blastocyst stage at frequencies similar to those previously reported, whereas androgenetic embryos constructed with fertilized eggs from DBA/2 mice developed poorly, the majority failing to progress beyond the 16-cell stage and unable to form a blastocoel-like cavity, regardless of whether the male pronuclei were of C57BL6 or DBA/2 origin. This impaired development was observed even in androgenetic embryos constructed by transplanting two male pronuclei from fertilized DBA/2 eggs to enucleated C57BL/6 eggs, indicating that the defect cannot be explained as the lack of some essential component in the DBA/2 cytoplasm that might otherwise compensate for androgeny. Rather, the DBA/2 egg cytoplasm apparently modifies the incoming male pronuclei differently than does C57BL/6 egg cytoplasm. Several specific alterations in the protein synthesis pattern of DBA/2 androgenones were observed that reflect a defect in the regulatory mechanisms that normally modulate the synthesis of these proteins between the 8-cell and blastocyst stages. These results are consistent with a model in which cytoplasmic factors present in the egg direct a strain-dependent modification of paternal genome function in response to epigenetic modifications (genomic imprinting) established during gametogenesis and indicate that preimplantation development can be affected by these modifications at both the morphological and biochemical levels.", 
"Functional consequences of either suppressing or intensifying spontaneous neuronal firing have been studied in developing rat cerebral cortex cultures using, respectively, tetrodotoxin (TTX) and picrotoxin (PTX) added chronically to the growth medium. Simple measures derived from the interspike interval histogram were able to powerfully discriminate between age and treatment groups. After return to control medium, most TTX-treated neurons spontaneously displayed stereotyped clustering of action potentials ('phasic' firing) which closely resembled the characteristic firing patterns seen acutely in the presence of PTX. The 'TTX-syndrome' thus suggests that GABAergic synaptic inhibition is ineffective in cortical networks grown under conditions which prevent the expression of bioelectric activity. In contrast, after return to control medium, neurons which had been partially disinhibited throughout development (by continuous exposure to PTX) had even less phasic firing than was measured in age-matched controls. Based upon these and previous findings, a two (main) factor model is put forth which can economically account for the major effects. The working hypothesis embodied in this model is that phasic neuronal discharges not only accelerate the maturation of excitatory connections within the neocortex but, even more important, are crucial for the development of adequate inhibitory synaptic transmission.", 
"Differences in the activity of maternally and paternally derived genomes in maize endosperm have been observed at three levels of genetic manipulation. When the balance of entire chromosome sets departs from the standard ratio of two of maternal origin to one of paternal origin, development is impaired, often leading to seed failure. At the level of individual chromosomes, absence of a paternal representative for 8 of the 19 chromosome arms tested causes a marked reduction in kernel size. Replacement of the missing arms by ones of maternal origin does not complement this defect. At the gene level, some alleles of R confer solid coloration on the aleurone layer when transmitted maternally but patchy coloration (mottled) when transmitted via pollen. In contrast with the endosperm, no effect of parentage on R phenotype has been detected in embryonic and seedling tissues. Furthermore, gynogenetic and androgenetic haploid plants are viable in maize and are similar in appearance. The detection of parental effects in the endosperm, but not the embryo, points to the few cell divisions of the gametophytes as a critical stage in imprinting. Chromosomally based epigenetic variation originating at this stage would be reflected as imprinting effects. A separate fertilization establishes a line of genetic descent in the embryo that appears to be relatively free of imprinted genes.", 
" ", 
"Herein we describe a general multivariate quantitative genetic model that incorporates two potentially important developmental phenomena, maternal effects and epigenetic effects. Maternal and epigenetic effects are defined as partial regression coefficients and phenotypic variances are derived in terms of age-specific genetic and environmental variances. As a starting point, the traditional quantitative genetic model of additive gene effects and random environmental effects is cast in a developmental time framework. From this framework, we first extend a maternal effects model to include multiple developmental ages for the occurrence of maternal effects. An example of maternal effects occurring at multiple developmental ages is prenatal and postnatal maternal effects in mammals. Subsequently, a model of intrinsic and epigenetic effects in the absence of maternal effects is described. It is shown that genetic correlations can arise through epigenetic effects, and in the absence of other developmental effects, epigenetic effects are in general confounded with age-specific intrinsic genetic effects. Finally, the two effects are incorporated into the basic quantitative genetic model. For this more biologically realistic model combining maternal and epigenetic effects, it is shown that the phenotypic regressions of offspring on mother and offspring on father can be used in some cases to estimate simultaneously maternal effects and epigenetic effects.", 
"Butylated hydroxyanisole (BHA) is a synthetic food antioxidant used to prevent oils, fats and shortenings from oxidative deterioration and rancidity. This review depicts the current knowledge on BHA. The physical and chemical characteristics of BHA are summarized and its function as a food antioxidant is made clear. The toxicological characteristics of BHA and its metabolic fate in man and animal are briefly reviewed. Special emphasis is laid on the carcinogenicity of BHA in the forestomach of rodents and to related events in the forestomach and other tissues in experimental animals. At present there is sufficient evidence for carcinogenicity of BHA, but there is hardly any indication that BHA is genotoxic. Therefore risk assessment for this epigenetic carcinogen is based on non-stochastic principles. However, the mechanism underlying the tumorigenicity of BHA is not known. In the last part of this review an attempt is made to unravel the unknown mechanism of carcinogenicity. It is hypothesized that BHA gives rise to tumor formation in rodent forestomach by inducing heritable changes in DNA. Evidence is being provided that reactive oxygen species, in particular hydroxylradicals, may play a crucial role. The key question with respect to risk assessment for BHA is whether or not the underlying mechanism is thresholded, which is important for the choice of the appropriate model to assess the risk, if any, for man and to manage any potential risk.", 
]



In [8]:
###import pandas as pd

###df = pd.read_excel (r'C:/Users/brady/Desktop/TM/Evo-Bio All Time/Test1.xlsx')
###print (df)

In [9]:
print(df)

['The American College of Medical Genetics and Genomics (ACMG) previously developed guidance for the interpretation of sequence variants. 1 In the past decade, sequencing technology has evolved -rapidly with the advent of high-throughput next-generation -sequencing. By adopting and leveraging next-generation sequencing, clinical laboratories are now performing an ever-increasing catalogue of genetic testing spanning genotyping, single genes, gene panels, exomes, genomes, transcriptomes, and epigenetic assays for genetic disorders. By virtue of increased complexity, this shift in genetic testing has been accompanied by new challenges in sequence interpretation. In this context the ACMG convened a workgroup in 2013 comprising representatives from the ACMG, the Association for Molecular Pathology (AMP), and the College of American Pathologists to revisit and revise the standards and guidelines for the interpretation of sequence variants. The group consisted of clinical laboratory director

In [10]:
###CLEAN DATA###

In [11]:
# Convert to list
data = [simple_preprocess(doc) for doc in df]

In [12]:
print(data)

[['the', 'american', 'college', 'of', 'medical', 'genetics', 'and', 'genomics', 'acmg', 'previously', 'developed', 'guidance', 'for', 'the', 'interpretation', 'of', 'sequence', 'variants', 'in', 'the', 'past', 'decade', 'sequencing', 'technology', 'has', 'evolved', 'rapidly', 'with', 'the', 'advent', 'of', 'high', 'throughput', 'next', 'generation', 'sequencing', 'by', 'adopting', 'and', 'leveraging', 'next', 'generation', 'sequencing', 'clinical', 'laboratories', 'are', 'now', 'performing', 'an', 'ever', 'increasing', 'catalogue', 'of', 'genetic', 'testing', 'spanning', 'genotyping', 'single', 'genes', 'gene', 'panels', 'exomes', 'genomes', 'transcriptomes', 'and', 'epigenetic', 'assays', 'for', 'genetic', 'disorders', 'by', 'virtue', 'of', 'increased', 'complexity', 'this', 'shift', 'in', 'genetic', 'testing', 'has', 'been', 'accompanied', 'by', 'new', 'challenges', 'in', 'sequence', 'interpretation', 'in', 'this', 'context', 'the', 'acmg', 'convened', 'workgroup', 'in', 'comprising'

In [13]:
###MAY NEED CLEANING STEP, MAY NOT BECAUSE ABSTRACTS ARE SO CLEAN###

In [14]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'american', 'college', 'of', 'medical', 'genetics', 'and', 'genomics', 'acmg', 'previously', 'developed', 'guidance', 'for', 'the', 'interpretation', 'of', 'sequence', 'variants', 'in', 'the', 'past', 'decade', 'sequencing', 'technology', 'has', 'evolved', 'rapidly', 'with', 'the', 'advent', 'of', 'high', 'throughput', 'next', 'generation', 'sequencing', 'by', 'adopting', 'and', 'leveraging', 'next', 'generation', 'sequencing', 'clinical', 'laboratories', 'are', 'now', 'performing', 'an', 'ever', 'increasing', 'catalogue', 'of', 'genetic', 'testing', 'spanning', 'genotyping', 'single', 'genes', 'gene', 'panels', 'exomes', 'genomes', 'transcriptomes', 'and', 'epigenetic', 'assays', 'for', 'genetic', 'disorders', 'by', 'virtue', 'of', 'increased', 'complexity', 'this', 'shift', 'in', 'genetic', 'testing', 'has', 'been', 'accompanied', 'by', 'new', 'challenges', 'in', 'sequence', 'interpretation', 'in', 'this', 'context', 'the', 'acmg', 'convened', 'workgroup', 'in', 'comprising'

In [15]:
###THIS STEP NEEDS TO BE CAREFULLY MONITORED TO MAKE SURE IT'S WORKING AS INTENDED###

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'american', 'college', 'of', 'medical', 'genetics', 'and', 'genomics', 'acmg', 'previously', 'developed', 'guidance', 'for', 'the', 'interpretation', 'of', 'sequence', 'variants', 'in', 'the', 'past_decade', 'sequencing', 'technology', 'has', 'evolved', 'rapidly', 'with', 'the', 'advent', 'of', 'high_throughput', 'next_generation_sequencing', 'by', 'adopting', 'and', 'leveraging', 'next_generation_sequencing', 'clinical', 'laboratories', 'are', 'now', 'performing', 'an', 'ever', 'increasing', 'catalogue', 'of', 'genetic', 'testing', 'spanning', 'genotyping', 'single', 'genes', 'gene', 'panels', 'exomes', 'genomes', 'transcriptomes', 'and', 'epigenetic', 'assays', 'for', 'genetic', 'disorders', 'by', 'virtue', 'of', 'increased', 'complexity', 'this', 'shift', 'in', 'genetic', 'testing', 'has_been', 'accompanied', 'by', 'new', 'challenges', 'in', 'sequence', 'interpretation', 'in', 'this', 'context', 'the', 'acmg', 'convened', 'workgroup', 'in', 'comprising', 'representatives', '

In [17]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
###en_core_web_sm needs to be explicitly loaded and I believe you need to use "python -m spacy download en" in cmd.exe to properly load everything.###

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['genomic', 'previously', 'develop', 'guidance', 'interpretation', 'sequence', 'variant', 'past_decade', 'sequence', 'technology', 'evolve', 'rapidly', 'advent', 'high_throughput', 'next_generation', 'sequence', 'adopt', 'leverage', 'next_generation', 'sequence', 'clinical', 'laboratory', 'perform', 'ever', 'increase', 'catalogue', 'genetic', 'testing', 'span', 'genotype', 'single', 'gene', 'gene', 'panel', 'exome', 'genome', 'transcriptome', 'epigenetic', 'assay', 'genetic', 'disorder', 'virtue', 'increase', 'complexity', 'shift', 'genetic', 'testing', 'accompany', 'new', 'challenge', 'sequence', 'interpretation', 'context', 'convene', 'comprise', 'representative', 'standard', 'guideline', 'interpretation', 'group', 'consist', 'clinical', 'laboratory', 'director', 'clinician', 'report', 'represent', 'expert', 'opinion', 'workgroup', 'input', 'pathologist', 'stakeholder', 'recommendation', 'primarily', 'apply', 'breadth', 'genetic', 'test', 'use', 'clinical', 'laboratory', 'include', 

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 7), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 4), (28, 3), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 4), (41, 7), (42, 1), (43, 2), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 3), (54, 1), (55, 1), (56, 4), (57, 5), (58, 1), (59, 2), (60, 1), (61, 3), (62, 1), (63, 2), (64, 1), (65, 2), (66, 1), (67, 2), (68, 2), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 2), (77, 3), (78, 1), (79, 1), (80, 1), (81, 1), (82, 5), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 4), (95, 1), (96, 1), (97, 1), (98, 1), (99, 3), (100, 4), (101, 1), (102, 1)]]


In [21]:
#to call an individual dictionary term switch value as appropriate
id2word[0]

'accompany'

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('accompany', 1),
  ('adopt', 1),
  ('advent', 1),
  ('amendment', 1),
  ('analysis', 1),
  ('apply', 1),
  ('approve', 1),
  ('assay', 1),
  ('base', 1),
  ('benign', 2),
  ('board', 1),
  ('breadth', 1),
  ('catalogue', 1),
  ('category', 1),
  ('cause', 1),
  ('certify', 1),
  ('challenge', 1),
  ('classify', 1),
  ('clinical', 7),
  ('clinician', 1),
  ('complexity', 2),
  ('comprise', 1),
  ('computational', 1),
  ('consist', 1),
  ('context', 1),
  ('convene', 1),
  ('criterion', 1),
  ('datum', 4),
  ('describe', 3),
  ('develop', 1),
  ('director', 1),
  ('disorder', 2),
  ('epigenetic', 1),
  ('equivalent', 1),
  ('ever', 1),
  ('evidence', 1),
  ('evolve', 1),
  ('exome', 2),
  ('expert', 1),
  ('functional', 1),
  ('gene', 4),
  ('genetic', 7),
  ('geneticist', 1),
  ('genome', 2),
  ('genomic', 1),
  ('genotype', 2),
  ('group', 1),
  ('guidance', 1),
  ('guideline', 1),
  ('high_throughput', 1),
  ('identify', 1),
  ('improvement', 1),
  ('include', 1),
  ('increase', 3)

In [23]:
###Loads of room for customization at this step. Read carefully and adjust accordingly.###

In [24]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.019*"locus" + 0.014*"study" + 0.013*"complex" + 0.012*"mechanism" + '
  '0.011*"gene" + 0.011*"growth" + 0.011*"cell" + 0.009*"induce" + '
  '0.009*"appear" + 0.009*"bind"'),
 (1,
  '0.018*"age" + 0.011*"mortality" + 0.010*"cyclin" + 0.007*"blood" + '
  '0.007*"cki" + 0.006*"chronological" + 0.006*"apparent" + 0.006*"improve" + '
  '0.006*"estimate" + 0.004*"racial"'),
 (2,
  '0.019*"cardiac" + 0.013*"localization" + 0.012*"hypertrophy" + '
  '0.010*"fibrosis" + 0.009*"partition" + 0.008*"heart" + 0.008*"pathological" '
  '+ 0.007*"spectrometry" + 0.006*"supply" + 0.006*"physiological"'),
 (3,
  '0.042*"methylation" + 0.032*"dna" + 0.031*"gene" + 0.023*"imprint" + '
  '0.021*"epigenetic" + 0.018*"chromosome" + 0.015*"modification" + '
  '0.015*"silence" + 0.011*"development" + 0.011*"maternal"'),
 (4,
  '0.123*"cell" + 0.030*"stem" + 0.016*"somatic" + 0.015*"embryonic" + '
  '0.014*"early" + 0.012*"development" + 0.012*"pluripotent" + 0.012*"adult" + '
  '0.011*"differentiate

In [26]:
###Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
###In my experience, topic coherence score, in particular, has been more helpful.

###Need to learn more about these values.

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.95312963779006

Coherence Score:  0.3903205723456517


In [28]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\brady\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.256091 -0.088141       1        1  48.367612
3      0.305615 -0.003207       2        1  19.184035
0      0.174082 -0.093583       3        1  11.080312
10     0.151473  0.283216       4        1   7.554944
4      0.024776 -0.055475       5        1   3.843860
9     -0.036937 -0.042534       6        1   2.983950
7     -0.052501 -0.049104       7        1   2.940217
8     -0.116980  0.014150       8        1   1.524989
13    -0.126713  0.011294       9        1   1.181064
2     -0.120085  0.007117      10        1   0.416468
5     -0.122101  0.007234      11        1   0.325019
1     -0.113988  0.000230      12        1   0.248758
12    -0.111729  0.004433      13        1   0.183537
11    -0.111005  0.004370      14        1   0.165235, topic_info=                 Term        Freq       Total Category  logprob  loglift
165              cell  427.000000  427.000000  Default  30.0000  30.0000
354       methylation  437.000000  437.000000  Default  29.0000  29.0000
292         chromatin  151.000000  151.000000  Default  28.0000  28.0000
108            cancer  258.000000  258.000000  Default  27.0000  27.0000
114               dna  341.000000  341.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
3911              bha    0.010733    0.669137  Topic14  -8.3189   2.2729
3916      forestomach    0.010370    0.660959  Topic14  -8.3533   2.2508
3913  carcinogenicity    0.010370    0.660959  Topic14  -8.3533   2.2508
687             adult    0.104475   19.973828  Topic14  -6.0433   1.1523
405          hallmark    0.012386    7.213320  Topic14  -8.1756   0.0385

[691 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
503       8  0.674678      absent
1855     11  0.807543   abundance
1089      4  0.671825  accomplish
727       4  0.200359      acetyl
727       6  0.801436      acetyl
...     ...       ...         ...
467       4  0.441940        vivo
467       7  0.176776        vivo
285       8  0.482353       xlink
2532      6  0.973540       yeast
2769      2  0.939555      zygote

[929 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 1, 11, 5, 10, 8, 9, 14, 3, 6, 2, 13, 12])

In [ ]:
###rename file.html before running command. Search computer and it should be in the user directory.
p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(p, 'file.html')